# Data quality
In this notebook we analyze the data quality refered to COVID19 in Mexico. This information in obtained thought the Secretariat of Health and published by *Dirección General de Epidemiología* the mexican CDC. https://www.gob.mx/salud/documentos/datos-abiertos-152127. 

In [142]:

import glob
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

import datetime
from IPython.display import Markdown as md

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib 

Using matplotlib backend: MacOSX


In [161]:
dict_ORIGEN = {'1': "USMER", '2': "FUERA DE USMER", '99': 'NO ESPECIFICADO'}
dict_SECTOR = {'1': 'CRUZ ROJA', '2': 'DIF', '3': 'ESTATAL', '4': 'IMSS', '5': 'IMSS-BIENESTAR', '6': 'ISSSTE', '7': 'MUNICIPAL', '8': 'PEMEX', '9': 'PRIVADA', '10': 'SEDENA', '11': 'SEMAR', '12': 'SSA', '13': 'UNIVERSITARIO', '99': 'NO ESPECIFICADO', }
dict_ENTIDAD = {'01': 'AGUASCALIENTES', '02': 'BAJA CALIFORNIA', '03': 'BAJA CALIFORNIA SUR', '04': 'CAMPECHE', '05': 'COAHUILA DE ZARAGOZA', '06': 'COLIMA', '07': 'CHIAPAS', '08': 'CHIHUAHUA', '09': 'CIUDAD DE MÉXICO', '10': 'DURANGO', '11': 'GUANAJUATO', '12': 'GUERRERO', '13': 'HIDALGO', '14': 'JALISCO', '15': 'MÉXICO', '16': 'MICHOACÁN DE OCAMPO', '17': 'MORELOS', '18': 'NAYARIT', '19': 'NUEVO LEÓN', '20': 'OAXACA', '21': 'PUEBLA', '22': 'QUERÉTARO', '23': 'QUINTANA ROO', '24': 'SAN LUIS POTOSÍ', '25': 'SINALOA', '26': 'SONORA', '27': 'TABASCO', '28': 'TAMAULIPAS', '29': 'TLAXCALA', '30': 'VERACRUZ DE IGNACIO DE LA LLAVE', '31': 'YUCATÁN', '32': 'ZACATECAS', '36': 'ESTADOS UNIDOS MEXICANOS', '97': 'NO APLICA', '98': 'SE IGNORA', '99': 'NO ESPECIFICADO',} 
dict_SEXO = {'1': "MUJER", '2': "HOMBRE", '99': 'NO ESPECIFICADO'}
dict_MUNICIPIO = {'001': 'AGUASCALIENTES', '002': 'ASIENTOS', '003': 'CALVILLO', '004': 'COSÍO', '005': 'JESÚS MARÍA', '006': 'PABELLÓN DE ARTEAGA', '007': 'RINCÓN DE ROMOS', '008': 'SAN JOSÉ DE GRACIA', '009': 'TEPEZALÁ', '010': 'EL LLANO', '011': 'SAN FRANCISCO DE LOS ROMO', '999': 'NO ESPECIFICADO', '001': 'ENSENADA', '002': 'MEXICALI', '003': 'TECATE', '004': 'TIJUANA', '005': 'PLAYAS DE ROSARITO', '999': 'NO ESPECIFICADO', '001': 'COMONDÚ', '002': 'MULEGÉ', '003': 'LA PAZ', '008': 'LOS CABOS', '009': 'LORETO', '999': 'NO ESPECIFICADO', '001': 'CALKINÍ', '002': 'CAMPECHE', '003': 'CARMEN', '004': 'CHAMPOTÓN', '005': 'HECELCHAKÁN', '006': 'HOPELCHÉN', '007': 'PALIZADA', '008': 'TENABO', '009': 'ESCÁRCEGA', '010': 'CALAKMUL', '011': 'CANDELARIA', '999': 'NO ESPECIFICADO', '001': 'ABASOLO', '002': 'ACUÑA', '003': 'ALLENDE', '004': 'ARTEAGA', '005': 'CANDELA', '006': 'CASTAÑOS', '007': 'CUATRO CIÉNEGAS', '008': 'ESCOBEDO', '009': 'FRANCISCO I. MADERO', '010': 'FRONTERA', '011': 'GENERAL CEPEDA', '012': 'GUERRERO', '013': 'HIDALGO', '014': 'JIMÉNEZ', '015': 'JUÁREZ', '016': 'LAMADRID', '017': 'MATAMOROS', '018': 'MONCLOVA', '019': 'MORELOS', '020': 'MÚZQUIZ', '021': 'NADADORES', '022': 'NAVA', '023': 'OCAMPO', '024': 'PARRAS', '025': 'PIEDRAS NEGRAS', '026': 'PROGRESO', '027': 'RAMOS ARIZPE', '028': 'SABINAS', '029': 'SACRAMENTO', '030': 'SALTILLO', '031': 'SAN BUENAVENTURA', '032': 'SAN JUAN DE SABINAS', '033': 'SAN PEDRO', '034': 'SIERRA MOJADA', '035': 'TORREÓN', '036': 'VIESCA', '037': 'VILLA UNIÓN', '038': 'ZARAGOZA', '999': 'NO ESPECIFICADO', '001': 'ARMERÍA', '002': 'COLIMA', '003': 'COMALA', '004': 'COQUIMATLÁN', '005': 'CUAUHTÉMOC', '006': 'IXTLAHUACÁN', '007': 'MANZANILLO', '008': 'MINATITLÁN', '009': 'TECOMÁN', '010': 'VILLA DE ÁLVAREZ', '999': 'NO ESPECIFICADO', '001': 'ACACOYAGUA', '002': 'ACALA', '003': 'ACAPETAHUA', '004': 'ALTAMIRANO', '005': 'AMATÁN', '006': 'AMATENANGO DE LA FRONTERA', '007': 'AMATENANGO DEL VALLE', '008': 'ANGEL ALBINO CORZO', '009': 'ARRIAGA', '010': 'BEJUCAL DE OCAMPO', '011': 'BELLA VISTA', '012': 'BERRIOZÁBAL', '013': 'BOCHIL', '014': 'EL BOSQUE', '015': 'CACAHOATÁN', '016': 'CATAZAJÁ', '017': 'CINTALAPA', '018': 'COAPILLA', '019': 'COMITÁN DE DOMÍNGUEZ', '020': 'LA CONCORDIA', '021': 'COPAINALÁ', '022': 'CHALCHIHUITÁN', '023': 'CHAMULA', '024': 'CHANAL', '025': 'CHAPULTENANGO', '026': 'CHENALHÓ', '027': 'CHIAPA DE CORZO', '028': 'CHIAPILLA', '029': 'CHICOASÉN', '030': 'CHICOMUSELO', '031': 'CHILÓN', '032': 'ESCUINTLA', '033': 'FRANCISCO LEÓN', '034': 'FRONTERA COMALAPA', '035': 'FRONTERA HIDALGO', '036': 'LA GRANDEZA', '037': 'HUEHUETÁN', '038': 'HUIXTÁN', '039': 'HUITIUPÁN', '040': 'HUIXTLA', '041': 'LA INDEPENDENCIA', '042': 'IXHUATÁN', '043': 'IXTACOMITÁN', '044': 'IXTAPA', '045': 'IXTAPANGAJOYA', '046': 'JIQUIPILAS', '047': 'JITOTOL', '048': 'JUÁREZ', '049': 'LARRÁINZAR', '050': 'LA LIBERTAD', '051': 'MAPASTEPEC', '052': 'LAS MARGARITAS', '053': 'MAZAPA DE MADERO', '054': 'MAZATÁN', '055': 'METAPA', '056': 'MITONTIC', '057': 'MOTOZINTLA', '058': 'NICOLÁS RUÍZ', '059': 'OCOSINGO', '060': 'OCOTEPEC', '061': 'OCOZOCOAUTLA DE ESPINOSA', '062': 'OSTUACÁN', '063': 'OSUMACINTA', '064': 'OXCHUC', '065': 'PALENQUE', '066': 'PANTELHÓ', '067': 'PANTEPEC', '068': 'PICHUCALCO', '069': 'PIJIJIAPAN', '070': 'EL PORVENIR', '071': 'VILLA COMALTITLÁN', '072': 'PUEBLO NUEVO SOLISTAHUACÁN', '073': 'RAYÓN', '074': 'REFORMA', '075': 'LAS ROSAS', '076': 'SABANILLA', '077': 'SALTO DE AGUA', '078': 'SAN CRISTÓBAL DE LAS CASAS', '079': 'SAN FERNANDO', '080': 'SILTEPEC', '081': 'SIMOJOVEL', '082': 'SITALÁ', '083': 'SOCOLTENANGO', '084': 'SOLOSUCHIAPA', '085': 'SOYALÓ', '086': 'SUCHIAPA', '087': 'SUCHIATE', '088': 'SUNUAPA', '089': 'TAPACHULA', '090': 'TAPALAPA', '091': 'TAPILULA', '092': 'TECPATÁN', '093': 'TENEJAPA', '094': 'TEOPISCA', '096': 'TILA', '097': 'TONALÁ', '098': 'TOTOLAPA', '099': 'LA TRINITARIA', '100': 'TUMBALÁ', '101': 'TUXTLA GUTIÉRREZ', '102': 'TUXTLA CHICO', '103': 'TUZANTÁN', '104': 'TZIMOL', '105': 'UNIÓN JUÁREZ', '106': 'VENUSTIANO CARRANZA', '107': 'VILLA CORZO', '108': 'VILLAFLORES', '109': 'YAJALÓN', '110': 'SAN LUCAS', '111': 'ZINACANTÁN', '112': 'SAN JUAN CANCUC', '113': 'ALDAMA', '114': 'BENEMÉRITO DE LAS AMÉRICAS', '115': 'MARAVILLA TENEJAPA', '116': 'MARQUÉS DE COMILLAS', '117': 'MONTECRISTO DE GUERRERO', '118': 'SAN ANDRÉS DURAZNAL', '119': 'SANTIAGO EL PINAR', '120': 'CAPITÁN LUIS ÁNGEL VIDAL', '121': 'RINCÓN CHAMULA SAN PEDRO', '122': 'EL PARRAL', '123': 'EMILIANO ZAPATA', '124': 'MEZCALAPA', '999': 'NO ESPECIFICADO', '001': 'AHUMADA', '002': 'ALDAMA', '003': 'ALLENDE', '004': 'AQUILES SERDÁN', '005': 'ASCENSIÓN', '006': 'BACHÍNIVA', '007': 'BALLEZA', '008': 'BATOPILAS DE MANUEL GÓMEZ MORÍN', '009': 'BOCOYNA', '010': 'BUENAVENTURA', '011': 'CAMARGO', '012': 'CARICHÍ', '013': 'CASAS GRANDES', '014': 'CORONADO', '015': 'COYAME DEL SOTOL', '016': 'LA CRUZ', '017': 'CUAUHTÉMOC', '018': 'CUSIHUIRIACHI', '019': 'CHIHUAHUA', '020': 'CHÍNIPAS', '021': 'DELICIAS', '022': 'DR. BELISARIO DOMÍNGUEZ', '023': 'GALEANA', '024': 'SANTA ISABEL', '025': 'GÓMEZ FARÍAS', '026': 'GRAN MORELOS', '027': 'GUACHOCHI', '028': 'GUADALUPE', '029': 'GUADALUPE Y CALVO', '030': 'GUAZAPARES', '031': 'GUERRERO', '032': 'HIDALGO DEL PARRAL', '033': 'HUEJOTITÁN', '034': 'IGNACIO ZARAGOZA', '035': 'JANOS', '036': 'JIMÉNEZ', '037': 'JUÁREZ', '038': 'JULIMES', '039': 'LÓPEZ', '040': 'MADERA', '041': 'MAGUARICHI', '042': 'MANUEL BENAVIDES', '043': 'MATACHÍ', '044': 'MATAMOROS', '045': 'MEOQUI', '046': 'MORELOS', '047': 'MORIS', '048': 'NAMIQUIPA', '049': 'NONOAVA', '050': 'NUEVO CASAS GRANDES', '051': 'OCAMPO', '052': 'OJINAGA', '053': 'PRAXEDIS G. GUERRERO', '054': 'RIVA PALACIO', '055': 'ROSALES', '056': 'ROSARIO', '057': 'SAN FRANCISCO DE BORJA', '058': 'SAN FRANCISCO DE CONCHOS', '059': 'SAN FRANCISCO DEL ORO', '060': 'SANTA BÁRBARA', '061': 'SATEVÓ', '062': 'SAUCILLO', '063': 'TEMÓSACHIC', '064': 'EL TULE', '065': 'URIQUE', '066': 'URUACHI', '067': 'VALLE DE ZARAGOZA', '999': 'NO ESPECIFICADO', '002': 'AZCAPOTZALCO', '003': 'COYOACÁN', '004': 'CUAJIMALPA DE MORELOS', '005': 'GUSTAVO A. MADERO', '006': 'IZTACALCO', '007': 'IZTAPALAPA', '008': 'LA MAGDALENA CONTRERAS', '009': 'MILPA ALTA', '010': 'ÁLVARO OBREGÓN', '011': 'TLÁHUAC', '012': 'TLALPAN', '013': 'XOCHIMILCO', '014': 'BENITO JUÁREZ', '015': 'CUAUHTÉMOC', '016': 'MIGUEL HIDALGO', '017': 'VENUSTIANO CARRANZA', '999': 'NO ESPECIFICADO', '001': 'CANATLÁN', '002': 'CANELAS', '003': 'CONETO DE COMONFORT', '004': 'CUENCAMÉ', '005': 'DURANGO', '006': 'GENERAL SIMÓN BOLÍVAR', '007': 'GÓMEZ PALACIO', '008': 'GUADALUPE VICTORIA', '009': 'GUANACEVÍ', '010': 'HIDALGO', '011': 'INDÉ', '012': 'LERDO', '013': 'MAPIMÍ', '014': 'MEZQUITAL', '015': 'NAZAS', '016': 'NOMBRE DE DIOS', '017': 'OCAMPO', '018': 'EL ORO', '019': 'OTÁEZ', '020': 'PÁNUCO DE CORONADO', '021': 'PEÑÓN BLANCO', '022': 'POANAS', '023': 'PUEBLO NUEVO', '024': 'RODEO', '025': 'SAN BERNARDO', '026': 'SAN DIMAS', '027': 'SAN JUAN DE GUADALUPE', '028': 'SAN JUAN DEL RÍO', '029': 'SAN LUIS DEL CORDERO', '030': 'SAN PEDRO DEL GALLO', '031': 'SANTA CLARA', '032': 'SANTIAGO PAPASQUIARO', '033': 'SÚCHIL', '034': 'TAMAZULA', '035': 'TEPEHUANES', '036': 'TLAHUALILO', '037': 'TOPIA', '038': 'VICENTE GUERRERO', '039': 'NUEVO IDEAL', '999': 'NO ESPECIFICADO', '001': 'ABASOLO', '002': 'ACÁMBARO', '003': 'SAN MIGUEL DE ALLENDE', '004': 'APASEO EL ALTO', '005': 'APASEO EL GRANDE', '006': 'ATARJEA', '007': 'CELAYA', '008': 'MANUEL DOBLADO', '009': 'COMONFORT', '010': 'CORONEO', '011': 'CORTAZAR', '012': 'CUERÁMARO', '013': 'DOCTOR MORA', '014': 'DOLORES HIDALGO CUNA DE LA INDEPENDENCIA NACIONAL', '015': 'GUANAJUATO', '016': 'HUANÍMARO', '017': 'IRAPUATO', '018': 'JARAL DEL PROGRESO', '019': 'JERÉCUARO', '020': 'LEÓN', '021': 'MOROLEÓN', '022': 'OCAMPO', '023': 'PÉNJAMO', '024': 'PUEBLO NUEVO', '025': 'PURÍSIMA DEL RINCÓN', '026': 'ROMITA', '027': 'SALAMANCA', '028': 'SALVATIERRA', '029': 'SAN DIEGO DE LA UNIÓN', '030': 'SAN FELIPE', '031': 'SAN FRANCISCO DEL RINCÓN', '032': 'SAN JOSÉ ITURBIDE', '033': 'SAN LUIS DE LA PAZ', '034': 'SANTA CATARINA', '035': 'SANTA CRUZ DE JUVENTINO ROSAS', '036': 'SANTIAGO MARAVATÍO', '037': 'SILAO DE LA VICTORIA', '038': 'TARANDACUAO', '039': 'TARIMORO', '040': 'TIERRA BLANCA', '041': 'URIANGATO', '042': 'VALLE DE SANTIAGO', '043': 'VICTORIA', '044': 'VILLAGRÁN', '045': 'XICHÚ', '046': 'YURIRIA', '999': 'NO ESPECIFICADO', '001': 'ACAPULCO DE JUÁREZ', '002': 'AHUACUOTZINGO', '003': 'AJUCHITLÁN DEL PROGRESO', '004': 'ALCOZAUCA DE GUERRERO', '005': 'ALPOYECA', '006': 'APAXTLA', '007': 'ARCELIA', '008': 'ATENANGO DEL RÍO', '009': 'ATLAMAJALCINGO DEL MONTE', '010': 'ATLIXTAC', '011': 'ATOYAC DE ÁLVAREZ', '012': 'AYUTLA DE LOS LIBRES', '013': 'AZOYÚ', '014': 'BENITO JUÁREZ', '015': 'BUENAVISTA DE CUÉLLAR', '016': 'COAHUAYUTLA DE JOSÉ MARÍA IZAZAGA', '017': 'COCULA', '018': 'COPALA', '019': 'COPALILLO', '020': 'COPANATOYAC', '021': 'COYUCA DE BENÍTEZ', '022': 'COYUCA DE CATALÁN', '023': 'CUAJINICUILAPA', '024': 'CUALÁC', '025': 'CUAUTEPEC', '026': 'CUETZALA DEL PROGRESO', '027': 'CUTZAMALA DE PINZÓN', '028': 'CHILAPA DE ÁLVAREZ', '029': 'CHILPANCINGO DE LOS BRAVO', '030': 'FLORENCIO VILLARREAL', '031': 'GENERAL CANUTO A. NERI', '032': 'GENERAL HELIODORO CASTILLO', '033': 'HUAMUXTITLÁN', '034': 'HUITZUCO DE LOS FIGUEROA', '035': 'IGUALA DE LA INDEPENDENCIA', '036': 'IGUALAPA', '037': 'IXCATEOPAN DE CUAUHTÉMOC', '038': 'ZIHUATANEJO DE AZUETA', '039': 'JUAN R. ESCUDERO', '040': 'LEONARDO BRAVO', '041': 'MALINALTEPEC', '042': 'MÁRTIR DE CUILAPAN', '043': 'METLATÓNOC', '044': 'MOCHITLÁN', '045': 'OLINALÁ', '046': 'OMETEPEC', '047': 'PEDRO ASCENCIO ALQUISIRAS', '048': 'PETATLÁN', '049': 'PILCAYA', '050': 'PUNGARABATO', '051': 'QUECHULTENANGO', '052': 'SAN LUIS ACATLÁN', '053': 'SAN MARCOS', '054': 'SAN MIGUEL TOTOLAPAN', '055': 'TAXCO DE ALARCÓN', '056': 'TECOANAPA', '057': 'TÉCPAN DE GALEANA', '058': 'TELOLOAPAN', '059': 'TEPECOACUILCO DE TRUJANO', '060': 'TETIPAC', '061': 'TIXTLA DE GUERRERO', '062': 'TLACOACHISTLAHUACA', '063': 'TLACOAPA', '064': 'TLALCHAPA', '065': 'TLALIXTAQUILLA DE MALDONADO', '066': 'TLAPA DE COMONFORT', '067': 'TLAPEHUALA', '068': 'LA UNIÓN DE ISIDORO MONTES DE OCA', '069': 'XALPATLÁHUAC', '070': 'XOCHIHUEHUETLÁN', '071': 'XOCHISTLAHUACA', '072': 'ZAPOTITLÁN TABLAS', '073': 'ZIRÁNDARO', '074': 'ZITLALA', '075': 'EDUARDO NERI', '076': 'ACATEPEC', '077': 'MARQUELIA', '078': 'COCHOAPA EL GRANDE', '079': 'JOSÉ JOAQUÍN DE HERRERA', '080': 'JUCHITÁN', '081': 'ILIATENCO', '999': 'NO ESPECIFICADO', '001': 'ACATLÁN', '002': 'ACAXOCHITLÁN', '003': 'ACTOPAN', '004': 'AGUA BLANCA DE ITURBIDE', '005': 'AJACUBA', '006': 'ALFAJAYUCAN', '007': 'ALMOLOYA', '008': 'APAN', '009': 'EL ARENAL', '010': 'ATITALAQUIA', '011': 'ATLAPEXCO', '012': 'ATOTONILCO EL GRANDE', '013': 'ATOTONILCO DE TULA', '014': 'CALNALI', '015': 'CARDONAL', '016': 'CUAUTEPEC DE HINOJOSA', '017': 'CHAPANTONGO', '018': 'CHAPULHUACÁN', '019': 'CHILCUAUTLA', '020': 'ELOXOCHITLÁN', '021': 'EMILIANO ZAPATA', '022': 'EPAZOYUCAN', '023': 'FRANCISCO I. MADERO', '024': 'HUASCA DE OCAMPO', '025': 'HUAUTLA', '026': 'HUAZALINGO', '027': 'HUEHUETLA', '028': 'HUEJUTLA DE REYES', '029': 'HUICHAPAN', '030': 'IXMIQUILPAN', '031': 'JACALA DE LEDEZMA', '032': 'JALTOCÁN', '033': 'JUÁREZ HIDALGO', '034': 'LOLOTLA', '035': 'METEPEC', '036': 'SAN AGUSTÍN METZQUITITLÁN', '037': 'METZTITLÁN', '038': 'MINERAL DEL CHICO', '039': 'MINERAL DEL MONTE', '040': 'LA MISIÓN', '041': 'MIXQUIAHUALA DE JUÁREZ', '042': 'MOLANGO DE ESCAMILLA', '043': 'NICOLÁS FLORES', '044': 'NOPALA DE VILLAGRÁN', '045': 'OMITLÁN DE JUÁREZ', '046': 'SAN FELIPE ORIZATLÁN', '047': 'PACULA', '048': 'PACHUCA DE SOTO', '049': 'PISAFLORES', '050': 'PROGRESO DE OBREGÓN', '051': 'MINERAL DE LA REFORMA', '052': 'SAN AGUSTÍN TLAXIACA', '053': 'SAN BARTOLO TUTOTEPEC', '054': 'SAN SALVADOR', '055': 'SANTIAGO DE ANAYA', '056': 'SANTIAGO TULANTEPEC DE LUGO GUERRERO', '057': 'SINGUILUCAN', '058': 'TASQUILLO', '059': 'TECOZAUTLA', '060': 'TENANGO DE DORIA', '061': 'TEPEAPULCO', '062': 'TEPEHUACÁN DE GUERRERO', '063': 'TEPEJI DEL RÍO DE OCAMPO', '064': 'TEPETITLÁN', '065': 'TETEPANGO', '066': 'VILLA DE TEZONTEPEC', '067': 'TEZONTEPEC DE ALDAMA', '068': 'TIANGUISTENGO', '069': 'TIZAYUCA', '070': 'TLAHUELILPAN', '071': 'TLAHUILTEPA', '072': 'TLANALAPA', '073': 'TLANCHINOL', '074': 'TLAXCOAPAN', '075': 'TOLCAYUCA', '076': 'TULA DE ALLENDE', '077': 'TULANCINGO DE BRAVO', '078': 'XOCHIATIPAN', '079': 'XOCHICOATLÁN', '080': 'YAHUALICA', '081': 'ZACUALTIPÁN DE ÁNGELES', '082': 'ZAPOTLÁN DE JUÁREZ', '083': 'ZEMPOALA', '084': 'ZIMAPÁN', '999': 'NO ESPECIFICADO', '001': 'ACATIC', '002': 'ACATLÁN DE JUÁREZ', '003': 'AHUALULCO DE MERCADO', '004': 'AMACUECA', '005': 'AMATITÁN', '006': 'AMECA', '007': 'SAN JUANITO DE ESCOBEDO', '008': 'ARANDAS', '009': 'EL ARENAL', '010': 'ATEMAJAC DE BRIZUELA', '011': 'ATENGO', '012': 'ATENGUILLO', '013': 'ATOTONILCO EL ALTO', '014': 'ATOYAC', '015': 'AUTLÁN DE NAVARRO', '016': 'AYOTLÁN', '017': 'AYUTLA', '018': 'LA BARCA', '019': 'BOLAÑOS', '020': 'CABO CORRIENTES', '021': 'CASIMIRO CASTILLO', '022': 'CIHUATLÁN', '023': 'ZAPOTLÁN EL GRANDE', '024': 'COCULA', '025': 'COLOTLÁN', '026': 'CONCEPCIÓN DE BUENOS AIRES', '027': 'CUAUTITLÁN DE GARCÍA BARRAGÁN', '028': 'CUAUTLA', '029': 'CUQUÍO', '030': 'CHAPALA', '031': 'CHIMALTITÁN', '032': 'CHIQUILISTLÁN', '033': 'DEGOLLADO', '034': 'EJUTLA', '035': 'ENCARNACIÓN DE DÍAZ', '036': 'ETZATLÁN', '037': 'EL GRULLO', '038': 'GUACHINANGO', '039': 'GUADALAJARA', '040': 'HOSTOTIPAQUILLO', '041': 'HUEJÚCAR', '042': 'HUEJUQUILLA EL ALTO', '043': 'LA HUERTA', '044': 'IXTLAHUACÁN DE LOS MEMBRILLOS', '045': 'IXTLAHUACÁN DEL RÍO', '046': 'JALOSTOTITLÁN', '047': 'JAMAY', '048': 'JESÚS MARÍA', '049': 'JILOTLÁN DE LOS DOLORES', '050': 'JOCOTEPEC', '051': 'JUANACATLÁN', '052': 'JUCHITLÁN', '053': 'LAGOS DE MORENO', '054': 'EL LIMÓN', '055': 'MAGDALENA', '056': 'SANTA MARÍA DEL ORO', '057': 'LA MANZANILLA DE LA PAZ', '058': 'MASCOTA', '059': 'MAZAMITLA', '060': 'MEXTICACÁN', '061': 'MEZQUITIC', '062': 'MIXTLÁN', '063': 'OCOTLÁN', '064': 'OJUELOS DE JALISCO', '065': 'PIHUAMO', '066': 'PONCITLÁN', '067': 'PUERTO VALLARTA', '068': 'VILLA PURIFICACIÓN', '069': 'QUITUPAN', '070': 'EL SALTO', '071': 'SAN CRISTÓBAL DE LA BARRANCA', '072': 'SAN DIEGO DE ALEJANDRÍA', '073': 'SAN JUAN DE LOS LAGOS', '074': 'SAN JULIÁN', '075': 'SAN MARCOS', '076': 'SAN MARTÍN DE BOLAÑOS', '077': 'SAN MARTÍN HIDALGO', '078': 'SAN MIGUEL EL ALTO', '079': 'GÓMEZ FARÍAS', '080': 'SAN SEBASTIÁN DEL OESTE', '081': 'SANTA MARÍA DE LOS ÁNGELES', '082': 'SAYULA', '083': 'TALA', '084': 'TALPA DE ALLENDE', '085': 'TAMAZULA DE GORDIANO', '086': 'TAPALPA', '087': 'TECALITLÁN', '088': 'TECOLOTLÁN', '089': 'TECHALUTA DE MONTENEGRO', '090': 'TENAMAXTLÁN', '091': 'TEOCALTICHE', '092': 'TEOCUITATLÁN DE CORONA', '093': 'TEPATITLÁN DE MORELOS', '094': 'TEQUILA', '095': 'TEUCHITLÁN', '096': 'TIZAPÁN EL ALTO', '097': 'TLAJOMULCO DE ZÚÑIGA', '098': 'SAN PEDRO TLAQUEPAQUE', '099': 'TOLIMÁN', '100': 'TOMATLÁN', '101': 'TONALÁ', '102': 'TONAYA', '103': 'TONILA', '104': 'TOTATICHE', '105': 'TOTOTLÁN', '106': 'TUXCACUESCO', '107': 'TUXCUECA', '108': 'TUXPAN', '109': 'UNIÓN DE SAN ANTONIO', '110': 'UNIÓN DE TULA', '111': 'VALLE DE GUADALUPE', '112': 'VALLE DE JUÁREZ', '113': 'SAN GABRIEL', '114': 'VILLA CORONA', '115': 'VILLA GUERRERO', '116': 'VILLA HIDALGO', '117': 'CAÑADAS DE OBREGÓN', '118': 'YAHUALICA DE GONZÁLEZ GALLO', '119': 'ZACOALCO DE TORRES', '120': 'ZAPOPAN', '121': 'ZAPOTILTIC', '122': 'ZAPOTITLÁN DE VADILLO', '123': 'ZAPOTLÁN DEL REY', '124': 'ZAPOTLANEJO', '125': 'SAN IGNACIO CERRO GORDO', '999': 'NO ESPECIFICADO', '001': 'ACAMBAY DE RUÍZ CASTAÑEDA', '002': 'ACOLMAN', '003': 'ACULCO', '004': 'ALMOLOYA DE ALQUISIRAS', '005': 'ALMOLOYA DE JUÁREZ', '006': 'ALMOLOYA DEL RÍO', '007': 'AMANALCO', '008': 'AMATEPEC', '009': 'AMECAMECA', '010': 'APAXCO', '011': 'ATENCO', '012': 'ATIZAPÁN', '013': 'ATIZAPÁN DE ZARAGOZA', '014': 'ATLACOMULCO', '015': 'ATLAUTLA', '016': 'AXAPUSCO', '017': 'AYAPANGO', '018': 'CALIMAYA', '019': 'CAPULHUAC', '020': 'COACALCO DE BERRIOZÁBAL', '021': 'COATEPEC HARINAS', '022': 'COCOTITLÁN', '023': 'COYOTEPEC', '024': 'CUAUTITLÁN', '025': 'CHALCO', '026': 'CHAPA DE MOTA', '027': 'CHAPULTEPEC', '028': 'CHIAUTLA', '029': 'CHICOLOAPAN', '030': 'CHICONCUAC', '031': 'CHIMALHUACÁN', '032': 'DONATO GUERRA', '033': 'ECATEPEC DE MORELOS', '034': 'ECATZINGO', '035': 'HUEHUETOCA', '036': 'HUEYPOXTLA', '037': 'HUIXQUILUCAN', '038': 'ISIDRO FABELA', '039': 'IXTAPALUCA', '040': 'IXTAPAN DE LA SAL', '041': 'IXTAPAN DEL ORO', '042': 'IXTLAHUACA', '043': 'XALATLACO', '044': 'JALTENCO', '045': 'JILOTEPEC', '046': 'JILOTZINGO', '047': 'JIQUIPILCO', '048': 'JOCOTITLÁN', '049': 'JOQUICINGO', '050': 'JUCHITEPEC', '051': 'LERMA', '052': 'MALINALCO', '053': 'MELCHOR OCAMPO', '054': 'METEPEC', '055': 'MEXICALTZINGO', '056': 'MORELOS', '057': 'NAUCALPAN DE JUÁREZ', '058': 'NEZAHUALCÓYOTL', '059': 'NEXTLALPAN', '060': 'NICOLÁS ROMERO', '061': 'NOPALTEPEC', '062': 'OCOYOACAC', '063': 'OCUILAN', '064': 'EL ORO', '065': 'OTUMBA', '066': 'OTZOLOAPAN', '067': 'OTZOLOTEPEC', '068': 'OZUMBA', '069': 'PAPALOTLA', '070': 'LA PAZ', '071': 'POLOTITLÁN', '072': 'RAYÓN', '073': 'SAN ANTONIO LA ISLA', '074': 'SAN FELIPE DEL PROGRESO', '075': 'SAN MARTÍN DE LAS PIRÁMIDES', '076': 'SAN MATEO ATENCO', '077': 'SAN SIMÓN DE GUERRERO', '078': 'SANTO TOMÁS', '079': 'SOYANIQUILPAN DE JUÁREZ', '080': 'SULTEPEC', '081': 'TECÁMAC', '082': 'TEJUPILCO', '083': 'TEMAMATLA', '084': 'TEMASCALAPA', '085': 'TEMASCALCINGO', '086': 'TEMASCALTEPEC', '087': 'TEMOAYA', '088': 'TENANCINGO', '089': 'TENANGO DEL AIRE', '090': 'TENANGO DEL VALLE', '091': 'TEOLOYUCAN', '092': 'TEOTIHUACÁN', '093': 'TEPETLAOXTOC', '094': 'TEPETLIXPA', '095': 'TEPOTZOTLÁN', '096': 'TEQUIXQUIAC', '097': 'TEXCALTITLÁN', '098': 'TEXCALYACAC', '099': 'TEXCOCO', '100': 'TEZOYUCA', '101': 'TIANGUISTENCO', '102': 'TIMILPAN', '103': 'TLALMANALCO', '104': 'TLALNEPANTLA DE BAZ', '105': 'TLATLAYA', '106': 'TOLUCA', '107': 'TONATICO', '108': 'TULTEPEC', '109': 'TULTITLÁN', '110': 'VALLE DE BRAVO', '111': 'VILLA DE ALLENDE', '112': 'VILLA DEL CARBÓN', '113': 'VILLA GUERRERO', '114': 'VILLA VICTORIA', '115': 'XONACATLÁN', '116': 'ZACAZONAPAN', '117': 'ZACUALPAN', '118': 'ZINACANTEPEC', '119': 'ZUMPAHUACÁN', '120': 'ZUMPANGO', '121': 'CUAUTITLÁN IZCALLI', '122': 'VALLE DE CHALCO SOLIDARIDAD', '123': 'LUVIANOS', '124': 'SAN JOSÉ DEL RINCÓN', '125': 'TONANITLA', '999': 'NO ESPECIFICADO', '001': 'ACUITZIO', '002': 'AGUILILLA', '003': 'ÁLVARO OBREGÓN', '004': 'ANGAMACUTIRO', '005': 'ANGANGUEO', '006': 'APATZINGÁN', '007': 'APORO', '008': 'AQUILA', '009': 'ARIO', '010': 'ARTEAGA', '011': 'BRISEÑAS', '012': 'BUENAVISTA', '013': 'CARÁCUARO', '014': 'COAHUAYANA', '015': 'COALCOMÁN DE VÁZQUEZ PALLARES', '016': 'COENEO', '017': 'CONTEPEC', '018': 'COPÁNDARO', '019': 'COTIJA', '020': 'CUITZEO', '021': 'CHARAPAN', '022': 'CHARO', '023': 'CHAVINDA', '024': 'CHERÁN', '025': 'CHILCHOTA', '026': 'CHINICUILA', '027': 'CHUCÁNDIRO', '028': 'CHURINTZIO', '029': 'CHURUMUCO', '030': 'ECUANDUREO', '031': 'EPITACIO HUERTA', '032': 'ERONGARÍCUARO', '033': 'GABRIEL ZAMORA', '034': 'HIDALGO', '035': 'LA HUACANA', '036': 'HUANDACAREO', '037': 'HUANIQUEO', '038': 'HUETAMO', '039': 'HUIRAMBA', '040': 'INDAPARAPEO', '041': 'IRIMBO', '042': 'IXTLÁN', '043': 'JACONA', '044': 'JIMÉNEZ', '045': 'JIQUILPAN', '046': 'JUÁREZ', '047': 'JUNGAPEO', '048': 'LAGUNILLAS', '049': 'MADERO', '050': 'MARAVATÍO', '051': 'MARCOS CASTELLANOS', '052': 'LÁZARO CÁRDENAS', '053': 'MORELIA', '054': 'MORELOS', '055': 'MÚGICA', '056': 'NAHUATZEN', '057': 'NOCUPÉTARO', '058': 'NUEVO PARANGARICUTIRO', '059': 'NUEVO URECHO', '060': 'NUMARÁN', '061': 'OCAMPO', '062': 'PAJACUARÁN', '063': 'PANINDÍCUARO', '064': 'PARÁCUARO', '065': 'PARACHO', '066': 'PÁTZCUARO', '067': 'PENJAMILLO', '068': 'PERIBÁN', '069': 'LA PIEDAD', '070': 'PURÉPERO', '071': 'PURUÁNDIRO', '072': 'QUERÉNDARO', '073': 'QUIROGA', '074': 'COJUMATLÁN DE RÉGULES', '075': 'LOS REYES', '076': 'SAHUAYO', '077': 'SAN LUCAS', '078': 'SANTA ANA MAYA', '079': 'SALVADOR ESCALANTE', '080': 'SENGUIO', '081': 'SUSUPUATO', '082': 'TACÁMBARO', '083': 'TANCÍTARO', '084': 'TANGAMANDAPIO', '085': 'TANGANCÍCUARO', '086': 'TANHUATO', '087': 'TARETAN', '088': 'TARÍMBARO', '089': 'TEPALCATEPEC', '090': 'TINGAMBATO', '091': 'TINGÜINDÍN', '092': 'TIQUICHEO DE NICOLÁS ROMERO', '093': 'TLALPUJAHUA', '094': 'TLAZAZALCA', '095': 'TOCUMBO', '096': 'TUMBISCATÍO', '097': 'TURICATO', '098': 'TUXPAN', '099': 'TUZANTLA', '100': 'TZINTZUNTZAN', '101': 'TZITZIO', '102': 'URUAPAN', '103': 'VENUSTIANO CARRANZA', '104': 'VILLAMAR', '105': 'VISTA HERMOSA', '106': 'YURÉCUARO', '107': 'ZACAPU', '108': 'ZAMORA', '109': 'ZINÁPARO', '110': 'ZINAPÉCUARO', '111': 'ZIRACUARETIRO', '112': 'ZITÁCUARO', '113': 'JOSÉ SIXTO VERDUZCO', '999': 'NO ESPECIFICADO', '001': 'AMACUZAC', '002': 'ATLATLAHUCAN', '003': 'AXOCHIAPAN', '004': 'AYALA', '005': 'COATLÁN DEL RÍO', '006': 'CUAUTLA', '007': 'CUERNAVACA', '008': 'EMILIANO ZAPATA', '009': 'HUITZILAC', '010': 'JANTETELCO', '011': 'JIUTEPEC', '012': 'JOJUTLA', '013': 'JONACATEPEC DE LEANDRO VALLE', '014': 'MAZATEPEC', '015': 'MIACATLÁN', '016': 'OCUITUCO', '017': 'PUENTE DE IXTLA', '018': 'TEMIXCO', '019': 'TEPALCINGO', '020': 'TEPOZTLÁN', '021': 'TETECALA', '022': 'TETELA DEL VOLCÁN', '023': 'TLALNEPANTLA', '024': 'TLALTIZAPÁN DE ZAPATA', '025': 'TLAQUILTENANGO', '026': 'TLAYACAPAN', '027': 'TOTOLAPAN', '028': 'XOCHITEPEC', '029': 'YAUTEPEC', '030': 'YECAPIXTLA', '031': 'ZACATEPEC', '032': 'ZACUALPAN DE AMILPAS', '033': 'TEMOAC', '034': 'COATETELCO', '035': 'XOXOCOTLA', '999': 'NO ESPECIFICADO', '001': 'ACAPONETA', '002': 'AHUACATLÁN', '003': 'AMATLÁN DE CAÑAS', '004': 'COMPOSTELA', '005': 'HUAJICORI', '006': 'IXTLÁN DEL RÍO', '007': 'JALA', '008': 'XALISCO', '009': 'DEL NAYAR', '010': 'ROSAMORADA', '011': 'RUÍZ', '012': 'SAN BLAS', '013': 'SAN PEDRO LAGUNILLAS', '014': 'SANTA MARÍA DEL ORO', '015': 'SANTIAGO IXCUINTLA', '016': 'TECUALA', '017': 'TEPIC', '018': 'TUXPAN', '019': 'LA YESCA', '020': 'BAHÍA DE BANDERAS', '999': 'NO ESPECIFICADO', '001': 'ABASOLO', '002': 'AGUALEGUAS', '003': 'LOS ALDAMAS', '004': 'ALLENDE', '005': 'ANÁHUAC', '006': 'APODACA', '007': 'ARAMBERRI', '008': 'BUSTAMANTE', '009': 'CADEREYTA JIMÉNEZ', '010': 'EL CARMEN', '011': 'CERRALVO', '012': 'CIÉNEGA DE FLORES', '013': 'CHINA', '014': 'DOCTOR ARROYO', '015': 'DOCTOR COSS', '016': 'DOCTOR GONZÁLEZ', '017': 'GALEANA', '018': 'GARCÍA', '019': 'SAN PEDRO GARZA GARCÍA', '020': 'GENERAL BRAVO', '021': 'GENERAL ESCOBEDO', '022': 'GENERAL TERÁN', '023': 'GENERAL TREVIÑO', '024': 'GENERAL ZARAGOZA', '025': 'GENERAL ZUAZUA', '026': 'GUADALUPE', '027': 'LOS HERRERAS', '028': 'HIGUERAS', '029': 'HUALAHUISES', '030': 'ITURBIDE', '031': 'JUÁREZ', '032': 'LAMPAZOS DE NARANJO', '033': 'LINARES', '034': 'MARÍN', '035': 'MELCHOR OCAMPO', '036': 'MIER Y NORIEGA', '037': 'MINA', '038': 'MONTEMORELOS', '039': 'MONTERREY', '040': 'PARÁS', '041': 'PESQUERÍA', '042': 'LOS RAMONES', '043': 'RAYONES', '044': 'SABINAS HIDALGO', '045': 'SALINAS VICTORIA', '046': 'SAN NICOLÁS DE LOS GARZA', '047': 'HIDALGO', '048': 'SANTA CATARINA', '049': 'SANTIAGO', '050': 'VALLECILLO', '051': 'VILLALDAMA', '999': 'NO ESPECIFICADO', '001': 'ABEJONES', '002': 'ACATLÁN DE PÉREZ FIGUEROA', '003': 'ASUNCIÓN CACALOTEPEC', '004': 'ASUNCIÓN CUYOTEPEJI', '005': 'ASUNCIÓN IXTALTEPEC', '006': 'ASUNCIÓN NOCHIXTLÁN', '007': 'ASUNCIÓN OCOTLÁN', '008': 'ASUNCIÓN TLACOLULITA', '009': 'AYOTZINTEPEC', '010': 'EL BARRIO DE LA SOLEDAD', '011': 'CALIHUALÁ', '012': 'CANDELARIA LOXICHA', '013': 'CIÉNEGA DE ZIMATLÁN', '014': 'CIUDAD IXTEPEC', '015': 'COATECAS ALTAS', '016': 'COICOYÁN DE LAS FLORES', '017': 'LA COMPAÑÍA', '018': 'CONCEPCIÓN BUENAVISTA', '019': 'CONCEPCIÓN PÁPALO', '020': 'CONSTANCIA DEL ROSARIO', '021': 'COSOLAPA', '022': 'COSOLTEPEC', '023': 'CUILÁPAM DE GUERRERO', '024': 'CUYAMECALCO VILLA DE ZARAGOZA', '025': 'CHAHUITES', '026': 'CHALCATONGO DE HIDALGO', '027': 'CHIQUIHUITLÁN DE BENITO JUÁREZ', '028': 'HEROICA CIUDAD DE EJUTLA DE CRESPO', '029': 'ELOXOCHITLÁN DE FLORES MAGÓN', '030': 'EL ESPINAL', '031': 'TAMAZULÁPAM DEL ESPÍRITU SANTO', '032': 'FRESNILLO DE TRUJANO', '033': 'GUADALUPE ETLA', '034': 'GUADALUPE DE RAMÍREZ', '035': 'GUELATAO DE JUÁREZ', '036': 'GUEVEA DE HUMBOLDT', '037': 'MESONES HIDALGO', '038': 'VILLA HIDALGO', '039': 'HEROICA CIUDAD DE HUAJUAPAN DE LEÓN', '040': 'HUAUTEPEC', '041': 'HUAUTLA DE JIMÉNEZ', '042': 'IXTLÁN DE JUÁREZ', '043': 'JUCHITÁN DE ZARAGOZA', '044': 'LOMA BONITA', '045': 'MAGDALENA APASCO', '046': 'MAGDALENA JALTEPEC', '047': 'SANTA MAGDALENA JICOTLÁN', '048': 'MAGDALENA MIXTEPEC', '049': 'MAGDALENA OCOTLÁN', '050': 'MAGDALENA PEÑASCO', '051': 'MAGDALENA TEITIPAC', '052': 'MAGDALENA TEQUISISTLÁN', '053': 'MAGDALENA TLACOTEPEC', '054': 'MAGDALENA ZAHUATLÁN', '055': 'MARISCALA DE JUÁREZ', '056': 'MÁRTIRES DE TACUBAYA', '057': 'MATÍAS ROMERO AVENDAÑO', '058': 'MAZATLÁN VILLA DE FLORES', '059': 'MIAHUATLÁN DE PORFIRIO DÍAZ', '060': 'MIXISTLÁN DE LA REFORMA', '061': 'MONJAS', '062': 'NATIVIDAD', '063': 'NAZARENO ETLA', '064': 'NEJAPA DE MADERO', '065': 'IXPANTEPEC NIEVES', '066': 'SANTIAGO NILTEPEC', '067': 'OAXACA DE JUÁREZ', '068': 'OCOTLÁN DE MORELOS', '069': 'LA PE', '070': 'PINOTEPA DE DON LUIS', '071': 'PLUMA HIDALGO', '072': 'SAN JOSÉ DEL PROGRESO', '073': 'PUTLA VILLA DE GUERRERO', '074': 'SANTA CATARINA QUIOQUITANI', '075': 'REFORMA DE PINEDA', '076': 'LA REFORMA', '077': 'REYES ETLA', '078': 'ROJAS DE CUAUHTÉMOC', '079': 'SALINA CRUZ', '080': 'SAN AGUSTÍN AMATENGO', '081': 'SAN AGUSTÍN ATENANGO', '082': 'SAN AGUSTÍN CHAYUCO', '083': 'SAN AGUSTÍN DE LAS JUNTAS', '084': 'SAN AGUSTÍN ETLA', '085': 'SAN AGUSTÍN LOXICHA', '086': 'SAN AGUSTÍN TLACOTEPEC', '087': 'SAN AGUSTÍN YATARENI', '088': 'SAN ANDRÉS CABECERA NUEVA', '089': 'SAN ANDRÉS DINICUITI', '090': 'SAN ANDRÉS HUAXPALTEPEC', '091': 'SAN ANDRÉS HUAYÁPAM', '092': 'SAN ANDRÉS IXTLAHUACA', '093': 'SAN ANDRÉS LAGUNAS', '094': 'SAN ANDRÉS NUXIÑO', '095': 'SAN ANDRÉS PAXTLÁN', '096': 'SAN ANDRÉS SINAXTLA', '097': 'SAN ANDRÉS SOLAGA', '098': 'SAN ANDRÉS TEOTILÁLPAM', '099': 'SAN ANDRÉS TEPETLAPA', '100': 'SAN ANDRÉS YAÁ', '101': 'SAN ANDRÉS ZABACHE', '102': 'SAN ANDRÉS ZAUTLA', '103': 'SAN ANTONINO CASTILLO VELASCO', '104': 'SAN ANTONINO EL ALTO', '105': 'SAN ANTONINO MONTE VERDE', '106': 'SAN ANTONIO ACUTLA', '107': 'SAN ANTONIO DE LA CAL', '108': 'SAN ANTONIO HUITEPEC', '109': 'SAN ANTONIO NANAHUATÍPAM', '110': 'SAN ANTONIO SINICAHUA', '111': 'SAN ANTONIO TEPETLAPA', '112': 'SAN BALTAZAR CHICHICÁPAM', '113': 'SAN BALTAZAR LOXICHA', '114': 'SAN BALTAZAR YATZACHI EL BAJO', '115': 'SAN BARTOLO COYOTEPEC', '116': 'SAN BARTOLOMÉ AYAUTLA', '117': 'SAN BARTOLOMÉ LOXICHA', '118': 'SAN BARTOLOMÉ QUIALANA', '119': 'SAN BARTOLOMÉ YUCUAÑE', '120': 'SAN BARTOLOMÉ ZOOGOCHO', '121': 'SAN BARTOLO SOYALTEPEC', '122': 'SAN BARTOLO YAUTEPEC', '123': 'SAN BERNARDO MIXTEPEC', '124': 'SAN BLAS ATEMPA', '125': 'SAN CARLOS YAUTEPEC', '126': 'SAN CRISTÓBAL AMATLÁN', '127': 'SAN CRISTÓBAL AMOLTEPEC', '128': 'SAN CRISTÓBAL LACHIRIOAG', '129': 'SAN CRISTÓBAL SUCHIXTLAHUACA', '130': 'SAN DIONISIO DEL MAR', '131': 'SAN DIONISIO OCOTEPEC', '132': 'SAN DIONISIO OCOTLÁN', '133': 'SAN ESTEBAN ATATLAHUCA', '134': 'SAN FELIPE JALAPA DE DÍAZ', '135': 'SAN FELIPE TEJALÁPAM', '136': 'SAN FELIPE USILA', '137': 'SAN FRANCISCO CAHUACUÁ', '138': 'SAN FRANCISCO CAJONOS', '139': 'SAN FRANCISCO CHAPULAPA', '140': 'SAN FRANCISCO CHINDÚA', '141': 'SAN FRANCISCO DEL MAR', '142': 'SAN FRANCISCO HUEHUETLÁN', '143': 'SAN FRANCISCO IXHUATÁN', '144': 'SAN FRANCISCO JALTEPETONGO', '145': 'SAN FRANCISCO LACHIGOLÓ', '146': 'SAN FRANCISCO LOGUECHE', '147': 'SAN FRANCISCO NUXAÑO', '148': 'SAN FRANCISCO OZOLOTEPEC', '149': 'SAN FRANCISCO SOLA', '150': 'SAN FRANCISCO TELIXTLAHUACA', '151': 'SAN FRANCISCO TEOPAN', '152': 'SAN FRANCISCO TLAPANCINGO', '153': 'SAN GABRIEL MIXTEPEC', '154': 'SAN ILDEFONSO AMATLÁN', '155': 'SAN ILDEFONSO SOLA', '156': 'SAN ILDEFONSO VILLA ALTA', '157': 'SAN JACINTO AMILPAS', '158': 'SAN JACINTO TLACOTEPEC', '159': 'SAN JERÓNIMO COATLÁN', '160': 'SAN JERÓNIMO SILACAYOAPILLA', '161': 'SAN JERÓNIMO SOSOLA', '162': 'SAN JERÓNIMO TAVICHE', '163': 'SAN JERÓNIMO TECÓATL', '164': 'SAN JORGE NUCHITA', '165': 'SAN JOSÉ AYUQUILA', '166': 'SAN JOSÉ CHILTEPEC', '167': 'SAN JOSÉ DEL PEÑASCO', '168': 'SAN JOSÉ ESTANCIA GRANDE', '169': 'SAN JOSÉ INDEPENDENCIA', '170': 'SAN JOSÉ LACHIGUIRI', '171': 'SAN JOSÉ TENANGO', '172': 'SAN JUAN ACHIUTLA', '173': 'SAN JUAN ATEPEC', '174': 'ÁNIMAS TRUJANO', '175': 'SAN JUAN BAUTISTA ATATLAHUCA', '176': 'SAN JUAN BAUTISTA COIXTLAHUACA', '177': 'SAN JUAN BAUTISTA CUICATLÁN', '178': 'SAN JUAN BAUTISTA GUELACHE', '179': 'SAN JUAN BAUTISTA JAYACATLÁN', '180': 'SAN JUAN BAUTISTA LO DE SOTO', '181': 'SAN JUAN BAUTISTA SUCHITEPEC', '182': 'SAN JUAN BAUTISTA TLACOATZINTEPEC', '183': 'SAN JUAN BAUTISTA TLACHICHILCO', '184': 'SAN JUAN BAUTISTA TUXTEPEC', '185': 'SAN JUAN CACAHUATEPEC', '186': 'SAN JUAN CIENEGUILLA', '187': 'SAN JUAN COATZÓSPAM', '188': 'SAN JUAN COLORADO', '189': 'SAN JUAN COMALTEPEC', '190': 'SAN JUAN COTZOCÓN', '191': 'SAN JUAN CHICOMEZÚCHIL', '192': 'SAN JUAN CHILATECA', '193': 'SAN JUAN DEL ESTADO', '194': 'SAN JUAN DEL RÍO', '195': 'SAN JUAN DIUXI', '196': 'SAN JUAN EVANGELISTA ANALCO', '197': 'SAN JUAN GUELAVÍA', '198': 'SAN JUAN GUICHICOVI', '199': 'SAN JUAN IHUALTEPEC', '200': 'SAN JUAN JUQUILA MIXES', '201': 'SAN JUAN JUQUILA VIJANOS', '202': 'SAN JUAN LACHAO', '203': 'SAN JUAN LACHIGALLA', '204': 'SAN JUAN LAJARCIA', '205': 'SAN JUAN LALANA', '206': 'SAN JUAN DE LOS CUÉS', '207': 'SAN JUAN MAZATLÁN', '208': 'SAN JUAN MIXTEPEC', '209': 'SAN JUAN MIXTEPEC', '210': 'SAN JUAN ÑUMÍ', '211': 'SAN JUAN OZOLOTEPEC', '212': 'SAN JUAN PETLAPA', '213': 'SAN JUAN QUIAHIJE', '214': 'SAN JUAN QUIOTEPEC', '215': 'SAN JUAN SAYULTEPEC', '216': 'SAN JUAN TABAÁ', '217': 'SAN JUAN TAMAZOLA', '218': 'SAN JUAN TEITA', '219': 'SAN JUAN TEITIPAC', '220': 'SAN JUAN TEPEUXILA', '221': 'SAN JUAN TEPOSCOLULA', '222': 'SAN JUAN YAEÉ', '223': 'SAN JUAN YATZONA', '224': 'SAN JUAN YUCUITA', '225': 'SAN LORENZO', '226': 'SAN LORENZO ALBARRADAS', '227': 'SAN LORENZO CACAOTEPEC', '228': 'SAN LORENZO CUAUNECUILTITLA', '229': 'SAN LORENZO TEXMELÚCAN', '230': 'SAN LORENZO VICTORIA', '231': 'SAN LUCAS CAMOTLÁN', '232': 'SAN LUCAS OJITLÁN', '233': 'SAN LUCAS QUIAVINÍ', '234': 'SAN LUCAS ZOQUIÁPAM', '235': 'SAN LUIS AMATLÁN', '236': 'SAN MARCIAL OZOLOTEPEC', '237': 'SAN MARCOS ARTEAGA', '238': 'SAN MARTÍN DE LOS CANSECOS', '239': 'SAN MARTÍN HUAMELÚLPAM', '240': 'SAN MARTÍN ITUNYOSO', '241': 'SAN MARTÍN LACHILÁ', '242': 'SAN MARTÍN PERAS', '243': 'SAN MARTÍN TILCAJETE', '244': 'SAN MARTÍN TOXPALAN', '245': 'SAN MARTÍN ZACATEPEC', '246': 'SAN MATEO CAJONOS', '247': 'CAPULÁLPAM DE MÉNDEZ', '248': 'SAN MATEO DEL MAR', '249': 'SAN MATEO YOLOXOCHITLÁN', '250': 'SAN MATEO ETLATONGO', '251': 'SAN MATEO NEJÁPAM', '252': 'SAN MATEO PEÑASCO', '253': 'SAN MATEO PIÑAS', '254': 'SAN MATEO RÍO HONDO', '255': 'SAN MATEO SINDIHUI', '256': 'SAN MATEO TLAPILTEPEC', '257': 'SAN MELCHOR BETAZA', '258': 'SAN MIGUEL ACHIUTLA', '259': 'SAN MIGUEL AHUEHUETITLÁN', '260': 'SAN MIGUEL ALOÁPAM', '261': 'SAN MIGUEL AMATITLÁN', '262': 'SAN MIGUEL AMATLÁN', '263': 'SAN MIGUEL COATLÁN', '264': 'SAN MIGUEL CHICAHUA', '265': 'SAN MIGUEL CHIMALAPA', '266': 'SAN MIGUEL DEL PUERTO', '267': 'SAN MIGUEL DEL RÍO', '268': 'SAN MIGUEL EJUTLA', '269': 'SAN MIGUEL EL GRANDE', '270': 'SAN MIGUEL HUAUTLA', '271': 'SAN MIGUEL MIXTEPEC', '272': 'SAN MIGUEL PANIXTLAHUACA', '273': 'SAN MIGUEL PERAS', '274': 'SAN MIGUEL PIEDRAS', '275': 'SAN MIGUEL QUETZALTEPEC', '276': 'SAN MIGUEL SANTA FLOR', '277': 'VILLA SOLA DE VEGA', '278': 'SAN MIGUEL SOYALTEPEC', '279': 'SAN MIGUEL SUCHIXTEPEC', '280': 'VILLA TALEA DE CASTRO', '281': 'SAN MIGUEL TECOMATLÁN', '282': 'SAN MIGUEL TENANGO', '283': 'SAN MIGUEL TEQUIXTEPEC', '284': 'SAN MIGUEL TILQUIÁPAM', '285': 'SAN MIGUEL TLACAMAMA', '286': 'SAN MIGUEL TLACOTEPEC', '287': 'SAN MIGUEL TULANCINGO', '288': 'SAN MIGUEL YOTAO', '289': 'SAN NICOLÁS', '290': 'SAN NICOLÁS HIDALGO', '291': 'SAN PABLO COATLÁN', '292': 'SAN PABLO CUATRO VENADOS', '293': 'SAN PABLO ETLA', '294': 'SAN PABLO HUITZO', '295': 'SAN PABLO HUIXTEPEC', '296': 'SAN PABLO MACUILTIANGUIS', '297': 'SAN PABLO TIJALTEPEC', '298': 'SAN PABLO VILLA DE MITLA', '299': 'SAN PABLO YAGANIZA', '300': 'SAN PEDRO AMUZGOS', '301': 'SAN PEDRO APÓSTOL', '302': 'SAN PEDRO ATOYAC', '303': 'SAN PEDRO CAJONOS', '304': 'SAN PEDRO COXCALTEPEC CÁNTAROS', '305': 'SAN PEDRO COMITANCILLO', '306': 'SAN PEDRO EL ALTO', '307': 'SAN PEDRO HUAMELULA', '308': 'SAN PEDRO HUILOTEPEC', '309': 'SAN PEDRO IXCATLÁN', '310': 'SAN PEDRO IXTLAHUACA', '311': 'SAN PEDRO JALTEPETONGO', '312': 'SAN PEDRO JICAYÁN', '313': 'SAN PEDRO JOCOTIPAC', '314': 'SAN PEDRO JUCHATENGO', '315': 'SAN PEDRO MÁRTIR', '316': 'SAN PEDRO MÁRTIR QUIECHAPA', '317': 'SAN PEDRO MÁRTIR YUCUXACO', '318': 'SAN PEDRO MIXTEPEC', '319': 'SAN PEDRO MIXTEPEC', '320': 'SAN PEDRO MOLINOS', '321': 'SAN PEDRO NOPALA', '322': 'SAN PEDRO OCOPETATILLO', '323': 'SAN PEDRO OCOTEPEC', '324': 'SAN PEDRO POCHUTLA', '325': 'SAN PEDRO QUIATONI', '326': 'SAN PEDRO SOCHIÁPAM', '327': 'SAN PEDRO TAPANATEPEC', '328': 'SAN PEDRO TAVICHE', '329': 'SAN PEDRO TEOZACOALCO', '330': 'SAN PEDRO TEUTILA', '331': 'SAN PEDRO TIDAÁ', '332': 'SAN PEDRO TOPILTEPEC', '333': 'SAN PEDRO TOTOLÁPAM', '334': 'VILLA DE TUTUTEPEC', '335': 'SAN PEDRO YANERI', '336': 'SAN PEDRO YÓLOX', '337': 'SAN PEDRO Y SAN PABLO AYUTLA', '338': 'VILLA DE ETLA', '339': 'SAN PEDRO Y SAN PABLO TEPOSCOLULA', '340': 'SAN PEDRO Y SAN PABLO TEQUIXTEPEC', '341': 'SAN PEDRO YUCUNAMA', '342': 'SAN RAYMUNDO JALPAN', '343': 'SAN SEBASTIÁN ABASOLO', '344': 'SAN SEBASTIÁN COATLÁN', '345': 'SAN SEBASTIÁN IXCAPA', '346': 'SAN SEBASTIÁN NICANANDUTA', '347': 'SAN SEBASTIÁN RÍO HONDO', '348': 'SAN SEBASTIÁN TECOMAXTLAHUACA', '349': 'SAN SEBASTIÁN TEITIPAC', '350': 'SAN SEBASTIÁN TUTLA', '351': 'SAN SIMÓN ALMOLONGAS', '352': 'SAN SIMÓN ZAHUATLÁN', '353': 'SANTA ANA', '354': 'SANTA ANA ATEIXTLAHUACA', '355': 'SANTA ANA CUAUHTÉMOC', '356': 'SANTA ANA DEL VALLE', '357': 'SANTA ANA TAVELA', '358': 'SANTA ANA TLAPACOYAN', '359': 'SANTA ANA YARENI', '360': 'SANTA ANA ZEGACHE', '361': 'SANTA CATALINA QUIERÍ', '362': 'SANTA CATARINA CUIXTLA', '363': 'SANTA CATARINA IXTEPEJI', '364': 'SANTA CATARINA JUQUILA', '365': 'SANTA CATARINA LACHATAO', '366': 'SANTA CATARINA LOXICHA', '367': 'SANTA CATARINA MECHOACÁN', '368': 'SANTA CATARINA MINAS', '369': 'SANTA CATARINA QUIANÉ', '370': 'SANTA CATARINA TAYATA', '371': 'SANTA CATARINA TICUÁ', '372': 'SANTA CATARINA YOSONOTÚ', '373': 'SANTA CATARINA ZAPOQUILA', '374': 'SANTA CRUZ ACATEPEC', '375': 'SANTA CRUZ AMILPAS', '376': 'SANTA CRUZ DE BRAVO', '377': 'SANTA CRUZ ITUNDUJIA', '378': 'SANTA CRUZ MIXTEPEC', '379': 'SANTA CRUZ NUNDACO', '380': 'SANTA CRUZ PAPALUTLA', '381': 'SANTA CRUZ TACACHE DE MINA', '382': 'SANTA CRUZ TACAHUA', '383': 'SANTA CRUZ TAYATA', '384': 'SANTA CRUZ XITLA', '385': 'SANTA CRUZ XOXOCOTLÁN', '386': 'SANTA CRUZ ZENZONTEPEC', '387': 'SANTA GERTRUDIS', '388': 'SANTA INÉS DEL MONTE', '389': 'SANTA INÉS YATZECHE', '390': 'SANTA LUCÍA DEL CAMINO', '391': 'SANTA LUCÍA MIAHUATLÁN', '392': 'SANTA LUCÍA MONTEVERDE', '393': 'SANTA LUCÍA OCOTLÁN', '394': 'SANTA MARÍA ALOTEPEC', '395': 'SANTA MARÍA APAZCO', '396': 'SANTA MARÍA LA ASUNCIÓN', '397': 'HEROICA CIUDAD DE TLAXIACO', '398': 'AYOQUEZCO DE ALDAMA', '399': 'SANTA MARÍA ATZOMPA', '400': 'SANTA MARÍA CAMOTLÁN', '401': 'SANTA MARÍA COLOTEPEC', '402': 'SANTA MARÍA CORTIJO', '403': 'SANTA MARÍA COYOTEPEC', '404': 'SANTA MARÍA CHACHOÁPAM', '405': 'VILLA DE CHILAPA DE DÍAZ', '406': 'SANTA MARÍA CHILCHOTLA', '407': 'SANTA MARÍA CHIMALAPA', '408': 'SANTA MARÍA DEL ROSARIO', '409': 'SANTA MARÍA DEL TULE', '410': 'SANTA MARÍA ECATEPEC', '411': 'SANTA MARÍA GUELACÉ', '412': 'SANTA MARÍA GUIENAGATI', '413': 'SANTA MARÍA HUATULCO', '414': 'SANTA MARÍA HUAZOLOTITLÁN', '415': 'SANTA MARÍA IPALAPA', '416': 'SANTA MARÍA IXCATLÁN', '417': 'SANTA MARÍA JACATEPEC', '418': 'SANTA MARÍA JALAPA DEL MARQUÉS', '419': 'SANTA MARÍA JALTIANGUIS', '420': 'SANTA MARÍA LACHIXÍO', '421': 'SANTA MARÍA MIXTEQUILLA', '422': 'SANTA MARÍA NATIVITAS', '423': 'SANTA MARÍA NDUAYACO', '424': 'SANTA MARÍA OZOLOTEPEC', '425': 'SANTA MARÍA PÁPALO', '426': 'SANTA MARÍA PEÑOLES', '427': 'SANTA MARÍA PETAPA', '428': 'SANTA MARÍA QUIEGOLANI', '429': 'SANTA MARÍA SOLA', '430': 'SANTA MARÍA TATALTEPEC', '431': 'SANTA MARÍA TECOMAVACA', '432': 'SANTA MARÍA TEMAXCALAPA', '433': 'SANTA MARÍA TEMAXCALTEPEC', '434': 'SANTA MARÍA TEOPOXCO', '435': 'SANTA MARÍA TEPANTLALI', '436': 'SANTA MARÍA TEXCATITLÁN', '437': 'SANTA MARÍA TLAHUITOLTEPEC', '438': 'SANTA MARÍA TLALIXTAC', '439': 'SANTA MARÍA TONAMECA', '440': 'SANTA MARÍA TOTOLAPILLA', '441': 'SANTA MARÍA XADANI', '442': 'SANTA MARÍA YALINA', '443': 'SANTA MARÍA YAVESÍA', '444': 'SANTA MARÍA YOLOTEPEC', '445': 'SANTA MARÍA YOSOYÚA', '446': 'SANTA MARÍA YUCUHITI', '447': 'SANTA MARÍA ZACATEPEC', '448': 'SANTA MARÍA ZANIZA', '449': 'SANTA MARÍA ZOQUITLÁN', '450': 'SANTIAGO AMOLTEPEC', '451': 'SANTIAGO APOALA', '452': 'SANTIAGO APÓSTOL', '453': 'SANTIAGO ASTATA', '454': 'SANTIAGO ATITLÁN', '455': 'SANTIAGO AYUQUILILLA', '456': 'SANTIAGO CACALOXTEPEC', '457': 'SANTIAGO CAMOTLÁN', '458': 'SANTIAGO COMALTEPEC', '459': 'VILLA DE SANTIAGO CHAZUMBA', '460': 'SANTIAGO CHOÁPAM', '461': 'SANTIAGO DEL RÍO', '462': 'SANTIAGO HUAJOLOTITLÁN', '463': 'SANTIAGO HUAUCLILLA', '464': 'SANTIAGO IHUITLÁN PLUMAS', '465': 'SANTIAGO IXCUINTEPEC', '466': 'SANTIAGO IXTAYUTLA', '467': 'SANTIAGO JAMILTEPEC', '468': 'SANTIAGO JOCOTEPEC', '469': 'SANTIAGO JUXTLAHUACA', '470': 'SANTIAGO LACHIGUIRI', '471': 'SANTIAGO LALOPA', '472': 'SANTIAGO LAOLLAGA', '473': 'SANTIAGO LAXOPA', '474': 'SANTIAGO LLANO GRANDE', '475': 'SANTIAGO MATATLÁN', '476': 'SANTIAGO MILTEPEC', '477': 'SANTIAGO MINAS', '478': 'SANTIAGO NACALTEPEC', '479': 'SANTIAGO NEJAPILLA', '480': 'SANTIAGO NUNDICHE', '481': 'SANTIAGO NUYOÓ', '482': 'SANTIAGO PINOTEPA NACIONAL', '483': 'SANTIAGO SUCHILQUITONGO', '484': 'SANTIAGO TAMAZOLA', '485': 'SANTIAGO TAPEXTLA', '486': 'VILLA TEJÚPAM DE LA UNIÓN', '487': 'SANTIAGO TENANGO', '488': 'SANTIAGO TEPETLAPA', '489': 'SANTIAGO TETEPEC', '490': 'SANTIAGO TEXCALCINGO', '491': 'SANTIAGO TEXTITLÁN', '492': 'SANTIAGO TILANTONGO', '493': 'SANTIAGO TILLO', '494': 'SANTIAGO TLAZOYALTEPEC', '495': 'SANTIAGO XANICA', '496': 'SANTIAGO XIACUÍ', '497': 'SANTIAGO YAITEPEC', '498': 'SANTIAGO YAVEO', '499': 'SANTIAGO YOLOMÉCATL', '500': 'SANTIAGO YOSONDÚA', '501': 'SANTIAGO YUCUYACHI', '502': 'SANTIAGO ZACATEPEC', '503': 'SANTIAGO ZOOCHILA', '504': 'NUEVO ZOQUIÁPAM', '505': 'SANTO DOMINGO INGENIO', '506': 'SANTO DOMINGO ALBARRADAS', '507': 'SANTO DOMINGO ARMENTA', '508': 'SANTO DOMINGO CHIHUITÁN', '509': 'SANTO DOMINGO DE MORELOS', '510': 'SANTO DOMINGO IXCATLÁN', '511': 'SANTO DOMINGO NUXAÁ', '512': 'SANTO DOMINGO OZOLOTEPEC', '513': 'SANTO DOMINGO PETAPA', '514': 'SANTO DOMINGO ROAYAGA', '515': 'SANTO DOMINGO TEHUANTEPEC', '516': 'SANTO DOMINGO TEOJOMULCO', '517': 'SANTO DOMINGO TEPUXTEPEC', '518': 'SANTO DOMINGO TLATAYÁPAM', '519': 'SANTO DOMINGO TOMALTEPEC', '520': 'SANTO DOMINGO TONALÁ', '521': 'SANTO DOMINGO TONALTEPEC', '522': 'SANTO DOMINGO XAGACÍA', '523': 'SANTO DOMINGO YANHUITLÁN', '524': 'SANTO DOMINGO YODOHINO', '525': 'SANTO DOMINGO ZANATEPEC', '526': 'SANTOS REYES NOPALA', '527': 'SANTOS REYES PÁPALO', '528': 'SANTOS REYES TEPEJILLO', '529': 'SANTOS REYES YUCUNÁ', '530': 'SANTO TOMÁS JALIEZA', '531': 'SANTO TOMÁS MAZALTEPEC', '532': 'SANTO TOMÁS OCOTEPEC', '533': 'SANTO TOMÁS TAMAZULAPAN', '534': 'SAN VICENTE COATLÁN', '535': 'SAN VICENTE LACHIXÍO', '536': 'SAN VICENTE NUÑÚ', '537': 'SILACAYOÁPAM', '538': 'SITIO DE XITLAPEHUA', '539': 'SOLEDAD ETLA', '540': 'VILLA DE TAMAZULÁPAM DEL PROGRESO', '541': 'TANETZE DE ZARAGOZA', '542': 'TANICHE', '543': 'TATALTEPEC DE VALDÉS', '544': 'TEOCOCUILCO DE MARCOS PÉREZ', '545': 'TEOTITLÁN DE FLORES MAGÓN', '546': 'TEOTITLÁN DEL VALLE', '547': 'TEOTONGO', '548': 'TEPELMEME VILLA DE MORELOS', '549': 'HEROICA VILLA TEZOATLÁN DE SEGURA Y LUNA, CUNA DE LA INDEPENDENCIA DE OAXACA', '550': 'SAN JERÓNIMO TLACOCHAHUAYA', '551': 'TLACOLULA DE MATAMOROS', '552': 'TLACOTEPEC PLUMAS', '553': 'TLALIXTAC DE CABRERA', '554': 'TOTONTEPEC VILLA DE MORELOS', '555': 'TRINIDAD ZAACHILA', '556': 'LA TRINIDAD VISTA HERMOSA', '557': 'UNIÓN HIDALGO', '558': 'VALERIO TRUJANO', '559': 'SAN JUAN BAUTISTA VALLE NACIONAL', '560': 'VILLA DÍAZ ORDAZ', '561': 'YAXE', '562': 'MAGDALENA YODOCONO DE PORFIRIO DÍAZ', '563': 'YOGANA', '564': 'YUTANDUCHI DE GUERRERO', '565': 'VILLA DE ZAACHILA', '566': 'SAN MATEO YUCUTINDOO', '567': 'ZAPOTITLÁN LAGUNAS', '568': 'ZAPOTITLÁN PALMAS', '569': 'SANTA INÉS DE ZARAGOZA', '570': 'ZIMATLÁN DE ÁLVAREZ', '999': 'NO ESPECIFICADO', '001': 'ACAJETE', '002': 'ACATENO', '003': 'ACATLÁN', '004': 'ACATZINGO', '005': 'ACTEOPAN', '006': 'AHUACATLÁN', '007': 'AHUATLÁN', '008': 'AHUAZOTEPEC', '009': 'AHUEHUETITLA', '010': 'AJALPAN', '011': 'ALBINO ZERTUCHE', '012': 'ALJOJUCA', '013': 'ALTEPEXI', '014': 'AMIXTLÁN', '015': 'AMOZOC', '016': 'AQUIXTLA', '017': 'ATEMPAN', '018': 'ATEXCAL', '019': 'ATLIXCO', '020': 'ATOYATEMPAN', '021': 'ATZALA', '022': 'ATZITZIHUACÁN', '023': 'ATZITZINTLA', '024': 'AXUTLA', '025': 'AYOTOXCO DE GUERRERO', '026': 'CALPAN', '027': 'CALTEPEC', '028': 'CAMOCUAUTLA', '029': 'CAXHUACAN', '030': 'COATEPEC', '031': 'COATZINGO', '032': 'COHETZALA', '033': 'COHUECAN', '034': 'CORONANGO', '035': 'COXCATLÁN', '036': 'COYOMEAPAN', '037': 'COYOTEPEC', '038': 'CUAPIAXTLA DE MADERO', '039': 'CUAUTEMPAN', '040': 'CUAUTINCHÁN', '041': 'CUAUTLANCINGO', '042': 'CUAYUCA DE ANDRADE', '043': 'CUETZALAN DEL PROGRESO', '044': 'CUYOACO', '045': 'CHALCHICOMULA DE SESMA', '046': 'CHAPULCO', '047': 'CHIAUTLA', '048': 'CHIAUTZINGO', '049': 'CHICONCUAUTLA', '050': 'CHICHIQUILA', '051': 'CHIETLA', '052': 'CHIGMECATITLÁN', '053': 'CHIGNAHUAPAN', '054': 'CHIGNAUTLA', '055': 'CHILA', '056': 'CHILA DE LA SAL', '057': 'HONEY', '058': 'CHILCHOTLA', '059': 'CHINANTLA', '060': 'DOMINGO ARENAS', '061': 'ELOXOCHITLÁN', '062': 'EPATLÁN', '063': 'ESPERANZA', '064': 'FRANCISCO Z. MENA', '065': 'GENERAL FELIPE ÁNGELES', '066': 'GUADALUPE', '067': 'GUADALUPE VICTORIA', '068': 'HERMENEGILDO GALEANA', '069': 'HUAQUECHULA', '070': 'HUATLATLAUCA', '071': 'HUAUCHINANGO', '072': 'HUEHUETLA', '073': 'HUEHUETLÁN EL CHICO', '074': 'HUEJOTZINGO', '075': 'HUEYAPAN', '076': 'HUEYTAMALCO', '077': 'HUEYTLALPAN', '078': 'HUITZILAN DE SERDÁN', '079': 'HUITZILTEPEC', '080': 'ATLEQUIZAYAN', '081': 'IXCAMILPA DE GUERRERO', '082': 'IXCAQUIXTLA', '083': 'IXTACAMAXTITLÁN', '084': 'IXTEPEC', '085': 'IZÚCAR DE MATAMOROS', '086': 'JALPAN', '087': 'JOLALPAN', '088': 'JONOTLA', '089': 'JOPALA', '090': 'JUAN C. BONILLA', '091': 'JUAN GALINDO', '092': 'JUAN N. MÉNDEZ', '093': 'LAFRAGUA', '094': 'LIBRES', '095': 'LA MAGDALENA TLATLAUQUITEPEC', '096': 'MAZAPILTEPEC DE JUÁREZ', '097': 'MIXTLA', '098': 'MOLCAXAC', '099': 'CAÑADA MORELOS', '100': 'NAUPAN', '101': 'NAUZONTLA', '102': 'NEALTICAN', '103': 'NICOLÁS BRAVO', '104': 'NOPALUCAN', '105': 'OCOTEPEC', '106': 'OCOYUCAN', '107': 'OLINTLA', '108': 'ORIENTAL', '109': 'PAHUATLÁN', '110': 'PALMAR DE BRAVO', '111': 'PANTEPEC', '112': 'PETLALCINGO', '113': 'PIAXTLA', '114': 'PUEBLA', '115': 'QUECHOLAC', '116': 'QUIMIXTLÁN', '117': 'RAFAEL LARA GRAJALES', '118': 'LOS REYES DE JUÁREZ', '119': 'SAN ANDRÉS CHOLULA', '120': 'SAN ANTONIO CAÑADA', '121': 'SAN DIEGO LA MESA TOCHIMILTZINGO', '122': 'SAN FELIPE TEOTLALCINGO', '123': 'SAN FELIPE TEPATLÁN', '124': 'SAN GABRIEL CHILAC', '125': 'SAN GREGORIO ATZOMPA', '126': 'SAN JERÓNIMO TECUANIPAN', '127': 'SAN JERÓNIMO XAYACATLÁN', '128': 'SAN JOSÉ CHIAPA', '129': 'SAN JOSÉ MIAHUATLÁN', '130': 'SAN JUAN ATENCO', '131': 'SAN JUAN ATZOMPA', '132': 'SAN MARTÍN TEXMELUCAN', '133': 'SAN MARTÍN TOTOLTEPEC', '134': 'SAN MATÍAS TLALANCALECA', '135': 'SAN MIGUEL IXITLÁN', '136': 'SAN MIGUEL XOXTLA', '137': 'SAN NICOLÁS BUENOS AIRES', '138': 'SAN NICOLÁS DE LOS RANCHOS', '139': 'SAN PABLO ANICANO', '140': 'SAN PEDRO CHOLULA', '141': 'SAN PEDRO YELOIXTLAHUACA', '142': 'SAN SALVADOR EL SECO', '143': 'SAN SALVADOR EL VERDE', '144': 'SAN SALVADOR HUIXCOLOTLA', '145': 'SAN SEBASTIÁN TLACOTEPEC', '146': 'SANTA CATARINA TLALTEMPAN', '147': 'SANTA INÉS AHUATEMPAN', '148': 'SANTA ISABEL CHOLULA', '149': 'SANTIAGO MIAHUATLÁN', '150': 'HUEHUETLÁN EL GRANDE', '151': 'SANTO TOMÁS HUEYOTLIPAN', '152': 'SOLTEPEC', '153': 'TECALI DE HERRERA', '154': 'TECAMACHALCO', '155': 'TECOMATLÁN', '156': 'TEHUACÁN', '157': 'TEHUITZINGO', '158': 'TENAMPULCO', '159': 'TEOPANTLÁN', '160': 'TEOTLALCO', '161': 'TEPANCO DE LÓPEZ', '162': 'TEPANGO DE RODRÍGUEZ', '163': 'TEPATLAXCO DE HIDALGO', '164': 'TEPEACA', '165': 'TEPEMAXALCO', '166': 'TEPEOJUMA', '167': 'TEPETZINTLA', '168': 'TEPEXCO', '169': 'TEPEXI DE RODRÍGUEZ', '170': 'TEPEYAHUALCO', '171': 'TEPEYAHUALCO DE CUAUHTÉMOC', '172': 'TETELA DE OCAMPO', '173': 'TETELES DE AVILA CASTILLO', '174': 'TEZIUTLÁN', '175': 'TIANGUISMANALCO', '176': 'TILAPA', '177': 'TLACOTEPEC DE BENITO JUÁREZ', '178': 'TLACUILOTEPEC', '179': 'TLACHICHUCA', '180': 'TLAHUAPAN', '181': 'TLALTENANGO', '182': 'TLANEPANTLA', '183': 'TLAOLA', '184': 'TLAPACOYA', '185': 'TLAPANALÁ', '186': 'TLATLAUQUITEPEC', '187': 'TLAXCO', '188': 'TOCHIMILCO', '189': 'TOCHTEPEC', '190': 'TOTOLTEPEC DE GUERRERO', '191': 'TULCINGO', '192': 'TUZAMAPAN DE GALEANA', '193': 'TZICATLACOYAN', '194': 'VENUSTIANO CARRANZA', '195': 'VICENTE GUERRERO', '196': 'XAYACATLÁN DE BRAVO', '197': 'XICOTEPEC', '198': 'XICOTLÁN', '199': 'XIUTETELCO', '200': 'XOCHIAPULCO', '201': 'XOCHILTEPEC', '202': 'XOCHITLÁN DE VICENTE SUÁREZ', '203': 'XOCHITLÁN TODOS SANTOS', '204': 'YAONÁHUAC', '205': 'YEHUALTEPEC', '206': 'ZACAPALA', '207': 'ZACAPOAXTLA', '208': 'ZACATLÁN', '209': 'ZAPOTITLÁN', '210': 'ZAPOTITLÁN DE MÉNDEZ', '211': 'ZARAGOZA', '212': 'ZAUTLA', '213': 'ZIHUATEUTLA', '214': 'ZINACATEPEC', '215': 'ZONGOZOTLA', '216': 'ZOQUIAPAN', '217': 'ZOQUITLÁN', '999': 'NO ESPECIFICADO', '001': 'AMEALCO DE BONFIL', '002': 'PINAL DE AMOLES', '003': 'ARROYO SECO', '004': 'CADEREYTA DE MONTES', '005': 'COLÓN', '006': 'CORREGIDORA', '007': 'EZEQUIEL MONTES', '008': 'HUIMILPAN', '009': 'JALPAN DE SERRA', '010': 'LANDA DE MATAMOROS', '011': 'EL MARQUÉS', '012': 'PEDRO ESCOBEDO', '013': 'PEÑAMILLER', '014': 'QUERÉTARO', '015': 'SAN JOAQUÍN', '016': 'SAN JUAN DEL RÍO', '017': 'TEQUISQUIAPAN', '018': 'TOLIMÁN', '999': 'NO ESPECIFICADO', '001': 'COZUMEL', '002': 'FELIPE CARRILLO PUERTO', '003': 'ISLA MUJERES', '004': 'OTHÓN P. BLANCO', '005': 'BENITO JUÁREZ', '006': 'JOSÉ MARÍA MORELOS', '007': 'LÁZARO CÁRDENAS', '008': 'SOLIDARIDAD', '009': 'TULUM', '010': 'BACALAR', '011': 'PUERTO MORELOS', '999': 'NO ESPECIFICADO', '001': 'AHUALULCO', '002': 'ALAQUINES', '003': 'AQUISMÓN', '004': 'ARMADILLO DE LOS INFANTE', '005': 'CÁRDENAS', '006': 'CATORCE', '007': 'CEDRAL', '008': 'CERRITOS', '009': 'CERRO DE SAN PEDRO', '010': 'CIUDAD DEL MAÍZ', '011': 'CIUDAD FERNÁNDEZ', '012': 'TANCANHUITZ', '013': 'CIUDAD VALLES', '014': 'COXCATLÁN', '015': 'CHARCAS', '016': 'EBANO', '017': 'GUADALCÁZAR', '018': 'HUEHUETLÁN', '019': 'LAGUNILLAS', '020': 'MATEHUALA', '021': 'MEXQUITIC DE CARMONA', '022': 'MOCTEZUMA', '023': 'RAYÓN', '024': 'RIOVERDE', '025': 'SALINAS', '026': 'SAN ANTONIO', '027': 'SAN CIRO DE ACOSTA', '028': 'SAN LUIS POTOSÍ', '029': 'SAN MARTÍN CHALCHICUAUTLA', '030': 'SAN NICOLÁS TOLENTINO', '031': 'SANTA CATARINA', '032': 'SANTA MARÍA DEL RÍO', '033': 'SANTO DOMINGO', '034': 'SAN VICENTE TANCUAYALAB', '035': 'SOLEDAD DE GRACIANO SÁNCHEZ', '036': 'TAMASOPO', '037': 'TAMAZUNCHALE', '038': 'TAMPACÁN', '039': 'TAMPAMOLÓN CORONA', '040': 'TAMUÍN', '041': 'TANLAJÁS', '042': 'TANQUIÁN DE ESCOBEDO', '043': 'TIERRA NUEVA', '044': 'VANEGAS', '045': 'VENADO', '046': 'VILLA DE ARRIAGA', '047': 'VILLA DE GUADALUPE', '048': 'VILLA DE LA PAZ', '049': 'VILLA DE RAMOS', '050': 'VILLA DE REYES', '051': 'VILLA HIDALGO', '052': 'VILLA JUÁREZ', '053': 'AXTLA DE TERRAZAS', '054': 'XILITLA', '055': 'ZARAGOZA', '056': 'VILLA DE ARISTA', '057': 'MATLAPA', '058': 'EL NARANJO', '999': 'NO ESPECIFICADO', '001': 'AHOME', '002': 'ANGOSTURA', '003': 'BADIRAGUATO', '004': 'CONCORDIA', '005': 'COSALÁ', '006': 'CULIACÁN', '007': 'CHOIX', '008': 'ELOTA', '009': 'ESCUINAPA', '010': 'EL FUERTE', '011': 'GUASAVE', '012': 'MAZATLÁN', '013': 'MOCORITO', '014': 'ROSARIO', '015': 'SALVADOR ALVARADO', '016': 'SAN IGNACIO', '017': 'SINALOA', '018': 'NAVOLATO', '999': 'NO ESPECIFICADO', '001': 'ACONCHI', '002': 'AGUA PRIETA', '003': 'ALAMOS', '004': 'ALTAR', '005': 'ARIVECHI', '006': 'ARIZPE', '007': 'ATIL', '008': 'BACADÉHUACHI', '009': 'BACANORA', '010': 'BACERAC', '011': 'BACOACHI', '012': 'BÁCUM', '013': 'BANÁMICHI', '014': 'BAVIÁCORA', '015': 'BAVISPE', '016': 'BENJAMÍN HILL', '017': 'CABORCA', '018': 'CAJEME', '019': 'CANANEA', '020': 'CARBÓ', '021': 'LA COLORADA', '022': 'CUCURPE', '023': 'CUMPAS', '024': 'DIVISADEROS', '025': 'EMPALME', '026': 'ETCHOJOA', '027': 'FRONTERAS', '028': 'GRANADOS', '029': 'GUAYMAS', '030': 'HERMOSILLO', '031': 'HUACHINERA', '032': 'HUÁSABAS', '033': 'HUATABAMPO', '034': 'HUÉPAC', '035': 'IMURIS', '036': 'MAGDALENA', '037': 'MAZATÁN', '038': 'MOCTEZUMA', '039': 'NACO', '040': 'NÁCORI CHICO', '041': 'NACOZARI DE GARCÍA', '042': 'NAVOJOA', '043': 'NOGALES', '044': 'ONAVAS', '045': 'OPODEPE', '046': 'OQUITOA', '047': 'PITIQUITO', '048': 'PUERTO PEÑASCO', '049': 'QUIRIEGO', '050': 'RAYÓN', '051': 'ROSARIO', '052': 'SAHUARIPA', '053': 'SAN FELIPE DE JESÚS', '054': 'SAN JAVIER', '055': 'SAN LUIS RÍO COLORADO', '056': 'SAN MIGUEL DE HORCASITAS', '057': 'SAN PEDRO DE LA CUEVA', '058': 'SANTA ANA', '059': 'SANTA CRUZ', '060': 'SÁRIC', '061': 'SOYOPA', '062': 'SUAQUI GRANDE', '063': 'TEPACHE', '064': 'TRINCHERAS', '065': 'TUBUTAMA', '066': 'URES', '067': 'VILLA HIDALGO', '068': 'VILLA PESQUEIRA', '069': 'YÉCORA', '070': 'GENERAL PLUTARCO ELÍAS CALLES', '071': 'BENITO JUÁREZ', '072': 'SAN IGNACIO RÍO MUERTO', '999': 'NO ESPECIFICADO', '001': 'BALANCÁN', '002': 'CÁRDENAS', '003': 'CENTLA', '004': 'CENTRO', '005': 'COMALCALCO', '006': 'CUNDUACÁN', '007': 'EMILIANO ZAPATA', '008': 'HUIMANGUILLO', '009': 'JALAPA', '010': 'JALPA DE MÉNDEZ', '011': 'JONUTA', '012': 'MACUSPANA', '013': 'NACAJUCA', '014': 'PARAÍSO', '015': 'TACOTALPA', '016': 'TEAPA', '017': 'TENOSIQUE', '999': 'NO ESPECIFICADO', '001': 'ABASOLO', '002': 'ALDAMA', '003': 'ALTAMIRA', '004': 'ANTIGUO MORELOS', '005': 'BURGOS', '006': 'BUSTAMANTE', '007': 'CAMARGO', '008': 'CASAS', '009': 'CIUDAD MADERO', '010': 'CRUILLAS', '011': 'GÓMEZ FARÍAS', '012': 'GONZÁLEZ', '013': 'GÜÉMEZ', '014': 'GUERRERO', '015': 'GUSTAVO DÍAZ ORDAZ', '016': 'HIDALGO', '017': 'JAUMAVE', '018': 'JIMÉNEZ', '019': 'LLERA', '020': 'MAINERO', '021': 'EL MANTE', '022': 'MATAMOROS', '023': 'MÉNDEZ', '024': 'MIER', '025': 'MIGUEL ALEMÁN', '026': 'MIQUIHUANA', '027': 'NUEVO LAREDO', '028': 'NUEVO MORELOS', '029': 'OCAMPO', '030': 'PADILLA', '031': 'PALMILLAS', '032': 'REYNOSA', '033': 'RÍO BRAVO', '034': 'SAN CARLOS', '035': 'SAN FERNANDO', '036': 'SAN NICOLÁS', '037': 'SOTO LA MARINA', '038': 'TAMPICO', '039': 'TULA', '040': 'VALLE HERMOSO', '041': 'VICTORIA', '042': 'VILLAGRÁN', '043': 'XICOTÉNCATL', '999': 'NO ESPECIFICADO', '001': 'AMAXAC DE GUERRERO', '002': 'APETATITLÁN DE ANTONIO CARVAJAL', '003': 'ATLANGATEPEC', '004': 'ATLTZAYANCA', '005': 'APIZACO', '006': 'CALPULALPAN', '007': 'EL CARMEN TEQUEXQUITLA', '008': 'CUAPIAXTLA', '009': 'CUAXOMULCO', '010': 'CHIAUTEMPAN', '011': 'MUÑOZ DE DOMINGO ARENAS', '012': 'ESPAÑITA', '013': 'HUAMANTLA', '014': 'HUEYOTLIPAN', '015': 'IXTACUIXTLA DE MARIANO MATAMOROS', '016': 'IXTENCO', '017': 'MAZATECOCHCO DE JOSÉ MARÍA MORELOS', '018': 'CONTLA DE JUAN CUAMATZI', '019': 'TEPETITLA DE LARDIZÁBAL', '020': 'SANCTÓRUM DE LÁZARO CÁRDENAS', '021': 'NANACAMILPA DE MARIANO ARISTA', '022': 'ACUAMANALA DE MIGUEL HIDALGO', '023': 'NATÍVITAS', '024': 'PANOTLA', '025': 'SAN PABLO DEL MONTE', '026': 'SANTA CRUZ TLAXCALA', '027': 'TENANCINGO', '028': 'TEOLOCHOLCO', '029': 'TEPEYANCO', '030': 'TERRENATE', '031': 'TETLA DE LA SOLIDARIDAD', '032': 'TETLATLAHUCA', '033': 'TLAXCALA', '034': 'TLAXCO', '035': 'TOCATLÁN', '036': 'TOTOLAC', '037': 'ZILTLALTÉPEC DE TRINIDAD SÁNCHEZ SANTOS', '038': 'TZOMPANTEPEC', '039': 'XALOZTOC', '040': 'XALTOCAN', '041': 'PAPALOTLA DE XICOHTÉNCATL', '042': 'XICOHTZINCO', '043': 'YAUHQUEMEHCAN', '044': 'ZACATELCO', '045': 'BENITO JUÁREZ', '046': 'EMILIANO ZAPATA', '047': 'LÁZARO CÁRDENAS', '048': 'LA MAGDALENA TLALTELULCO', '049': 'SAN DAMIÁN TEXÓLOC', '050': 'SAN FRANCISCO TETLANOHCAN', '051': 'SAN JERÓNIMO ZACUALPAN', '052': 'SAN JOSÉ TEACALCO', '053': 'SAN JUAN HUACTZINCO', '054': 'SAN LORENZO AXOCOMANITLA', '055': 'SAN LUCAS TECOPILCO', '056': 'SANTA ANA NOPALUCAN', '057': 'SANTA APOLONIA TEACALCO', '058': 'SANTA CATARINA AYOMETLA', '059': 'SANTA CRUZ QUILEHTLA', '060': 'SANTA ISABEL XILOXOXTLA', '999': 'NO ESPECIFICADO', '001': 'ACAJETE', '002': 'ACATLÁN', '003': 'ACAYUCAN', '004': 'ACTOPAN', '005': 'ACULA', '006': 'ACULTZINGO', '007': 'CAMARÓN DE TEJEDA', '008': 'ALPATLÁHUAC', '009': 'ALTO LUCERO DE GUTIÉRREZ BARRIOS', '010': 'ALTOTONGA', '011': 'ALVARADO', '012': 'AMATITLÁN', '013': 'NARANJOS AMATLÁN', '014': 'AMATLÁN DE LOS REYES', '015': 'ANGEL R. CABADA', '016': 'LA ANTIGUA', '017': 'APAZAPAN', '018': 'AQUILA', '019': 'ASTACINGA', '020': 'ATLAHUILCO', '021': 'ATOYAC', '022': 'ATZACAN', '023': 'ATZALAN', '024': 'TLALTETELA', '025': 'AYAHUALULCO', '026': 'BANDERILLA', '027': 'BENITO JUÁREZ', '028': 'BOCA DEL RÍO', '029': 'CALCAHUALCO', '030': 'CAMERINO Z. MENDOZA', '031': 'CARRILLO PUERTO', '032': 'CATEMACO', '033': 'CAZONES DE HERRERA', '034': 'CERRO AZUL', '035': 'CITLALTÉPETL', '036': 'COACOATZINTLA', '037': 'COAHUITLÁN', '038': 'COATEPEC', '039': 'COATZACOALCOS', '040': 'COATZINTLA', '041': 'COETZALA', '042': 'COLIPA', '043': 'COMAPA', '044': 'CÓRDOBA', '045': 'COSAMALOAPAN DE CARPIO', '046': 'COSAUTLÁN DE CARVAJAL', '047': 'COSCOMATEPEC', '048': 'COSOLEACAQUE', '049': 'COTAXTLA', '050': 'COXQUIHUI', '051': 'COYUTLA', '052': 'CUICHAPA', '053': 'CUITLÁHUAC', '054': 'CHACALTIANGUIS', '055': 'CHALMA', '056': 'CHICONAMEL', '057': 'CHICONQUIACO', '058': 'CHICONTEPEC', '059': 'CHINAMECA', '060': 'CHINAMPA DE GOROSTIZA', '061': 'LAS CHOAPAS', '062': 'CHOCAMÁN', '063': 'CHONTLA', '064': 'CHUMATLÁN', '065': 'EMILIANO ZAPATA', '066': 'ESPINAL', '067': 'FILOMENO MATA', '068': 'FORTÍN', '069': 'GUTIÉRREZ ZAMORA', '070': 'HIDALGOTITLÁN', '071': 'HUATUSCO', '072': 'HUAYACOCOTLA', '073': 'HUEYAPAN DE OCAMPO', '074': 'HUILOAPAN DE CUAUHTÉMOC', '075': 'IGNACIO DE LA LLAVE', '076': 'ILAMATLÁN', '077': 'ISLA', '078': 'IXCATEPEC', '079': 'IXHUACÁN DE LOS REYES', '080': 'IXHUATLÁN DEL CAFÉ', '081': 'IXHUATLANCILLO', '082': 'IXHUATLÁN DEL SURESTE', '083': 'IXHUATLÁN DE MADERO', '084': 'IXMATLAHUACAN', '085': 'IXTACZOQUITLÁN', '086': 'JALACINGO', '087': 'XALAPA', '088': 'JALCOMULCO', '089': 'JÁLTIPAN', '090': 'JAMAPA', '091': 'JESÚS CARRANZA', '092': 'XICO', '093': 'JILOTEPEC', '094': 'JUAN RODRÍGUEZ CLARA', '095': 'JUCHIQUE DE FERRER', '096': 'LANDERO Y COSS', '097': 'LERDO DE TEJADA', '098': 'MAGDALENA', '099': 'MALTRATA', '100': 'MANLIO FABIO ALTAMIRANO', '101': 'MARIANO ESCOBEDO', '102': 'MARTÍNEZ DE LA TORRE', '103': 'MECATLÁN', '104': 'MECAYAPAN', '105': 'MEDELLÍN DE BRAVO', '106': 'MIAHUATLÁN', '107': 'LAS MINAS', '108': 'MINATITLÁN', '109': 'MISANTLA', '110': 'MIXTLA DE ALTAMIRANO', '111': 'MOLOACÁN', '112': 'NAOLINCO', '113': 'NARANJAL', '114': 'NAUTLA', '115': 'NOGALES', '116': 'OLUTA', '117': 'OMEALCA', '118': 'ORIZABA', '119': 'OTATITLÁN', '120': 'OTEAPAN', '121': 'OZULUAMA DE MASCAREÑAS', '122': 'PAJAPAN', '123': 'PÁNUCO', '124': 'PAPANTLA', '125': 'PASO DEL MACHO', '126': 'PASO DE OVEJAS', '127': 'LA PERLA', '128': 'PEROTE', '129': 'PLATÓN SÁNCHEZ', '130': 'PLAYA VICENTE', '131': 'POZA RICA DE HIDALGO', '132': 'LAS VIGAS DE RAMÍREZ', '133': 'PUEBLO VIEJO', '134': 'PUENTE NACIONAL', '135': 'RAFAEL DELGADO', '136': 'RAFAEL LUCIO', '137': 'LOS REYES', '138': 'RÍO BLANCO', '139': 'SALTABARRANCA', '140': 'SAN ANDRÉS TENEJAPAN', '141': 'SAN ANDRÉS TUXTLA', '142': 'SAN JUAN EVANGELISTA', '143': 'SANTIAGO TUXTLA', '144': 'SAYULA DE ALEMÁN', '145': 'SOCONUSCO', '146': 'SOCHIAPA', '147': 'SOLEDAD ATZOMPA', '148': 'SOLEDAD DE DOBLADO', '149': 'SOTEAPAN', '150': 'TAMALÍN', '151': 'TAMIAHUA', '152': 'TAMPICO ALTO', '153': 'TANCOCO', '154': 'TANTIMA', '155': 'TANTOYUCA', '156': 'TATATILA', '157': 'CASTILLO DE TEAYO', '158': 'TECOLUTLA', '159': 'TEHUIPANGO', '160': 'ÁLAMO TEMAPACHE', '161': 'TEMPOAL', '162': 'TENAMPA', '163': 'TENOCHTITLÁN', '164': 'TEOCELO', '165': 'TEPATLAXCO', '166': 'TEPETLÁN', '167': 'TEPETZINTLA', '168': 'TEQUILA', '169': 'JOSÉ AZUETA', '170': 'TEXCATEPEC', '171': 'TEXHUACÁN', '172': 'TEXISTEPEC', '173': 'TEZONAPA', '174': 'TIERRA BLANCA', '175': 'TIHUATLÁN', '176': 'TLACOJALPAN', '177': 'TLACOLULAN', '178': 'TLACOTALPAN', '179': 'TLACOTEPEC DE MEJÍA', '180': 'TLACHICHILCO', '181': 'TLALIXCOYAN', '182': 'TLALNELHUAYOCAN', '183': 'TLAPACOYAN', '184': 'TLAQUILPA', '185': 'TLILAPAN', '186': 'TOMATLÁN', '187': 'TONAYÁN', '188': 'TOTUTLA', '189': 'TUXPAN', '190': 'TUXTILLA', '191': 'URSULO GALVÁN', '192': 'VEGA DE ALATORRE', '193': 'VERACRUZ', '194': 'VILLA ALDAMA', '195': 'XOXOCOTLA', '196': 'YANGA', '197': 'YECUATLA', '198': 'ZACUALPAN', '199': 'ZARAGOZA', '200': 'ZENTLA', '201': 'ZONGOLICA', '202': 'ZONTECOMATLÁN DE LÓPEZ Y FUENTES', '203': 'ZOZOCOLCO DE HIDALGO', '204': 'AGUA DULCE', '205': 'EL HIGO', '206': 'NANCHITAL DE LÁZARO CÁRDENAS DEL RÍO', '207': 'TRES VALLES', '208': 'CARLOS A. CARRILLO', '209': 'TATAHUICAPAN DE JUÁREZ', '210': 'UXPANAPA', '211': 'SAN RAFAEL', '212': 'SANTIAGO SOCHIAPAN', '999': 'NO ESPECIFICADO', '001': 'ABALÁ', '002': 'ACANCEH', '003': 'AKIL', '004': 'BACA', '005': 'BOKOBÁ', '006': 'BUCTZOTZ', '007': 'CACALCHÉN', '008': 'CALOTMUL', '009': 'CANSAHCAB', '010': 'CANTAMAYEC', '011': 'CELESTÚN', '012': 'CENOTILLO', '013': 'CONKAL', '014': 'CUNCUNUL', '015': 'CUZAMÁ', '016': 'CHACSINKÍN', '017': 'CHANKOM', '018': 'CHAPAB', '019': 'CHEMAX', '020': 'CHICXULUB PUEBLO', '021': 'CHICHIMILÁ', '022': 'CHIKINDZONOT', '023': 'CHOCHOLÁ', '024': 'CHUMAYEL', '025': 'DZÁN', '026': 'DZEMUL', '027': 'DZIDZANTÚN', '028': 'DZILAM DE BRAVO', '029': 'DZILAM GONZÁLEZ', '030': 'DZITÁS', '031': 'DZONCAUICH', '032': 'ESPITA', '033': 'HALACHÓ', '034': 'HOCABÁ', '035': 'HOCTÚN', '036': 'HOMÚN', '037': 'HUHÍ', '038': 'HUNUCMÁ', '039': 'IXIL', '040': 'IZAMAL', '041': 'KANASÍN', '042': 'KANTUNIL', '043': 'KAUA', '044': 'KINCHIL', '045': 'KOPOMÁ', '046': 'MAMA', '047': 'MANÍ', '048': 'MAXCANÚ', '049': 'MAYAPÁN', '050': 'MÉRIDA', '051': 'MOCOCHÁ', '052': 'MOTUL', '053': 'MUNA', '054': 'MUXUPIP', '055': 'OPICHÉN', '056': 'OXKUTZCAB', '057': 'PANABÁ', '058': 'PETO', '059': 'PROGRESO', '060': 'QUINTANA ROO', '061': 'RÍO LAGARTOS', '062': 'SACALUM', '063': 'SAMAHIL', '064': 'SANAHCAT', '065': 'SAN FELIPE', '066': 'SANTA ELENA', '067': 'SEYÉ', '068': 'SINANCHÉ', '069': 'SOTUTA', '070': 'SUCILÁ', '071': 'SUDZAL', '072': 'SUMA', '073': 'TAHDZIÚ', '074': 'TAHMEK', '075': 'TEABO', '076': 'TECOH', '077': 'TEKAL DE VENEGAS', '078': 'TEKANTÓ', '079': 'TEKAX', '080': 'TEKIT', '081': 'TEKOM', '082': 'TELCHAC PUEBLO', '083': 'TELCHAC PUERTO', '084': 'TEMAX', '085': 'TEMOZÓN', '086': 'TEPAKÁN', '087': 'TETIZ', '088': 'TEYA', '089': 'TICUL', '090': 'TIMUCUY', '091': 'TINUM', '092': 'TIXCACALCUPUL', '093': 'TIXKOKOB', '094': 'TIXMEHUAC', '095': 'TIXPÉHUAL', '096': 'TIZIMÍN', '097': 'TUNKÁS', '098': 'TZUCACAB', '099': 'UAYMA', '100': 'UCÚ', '101': 'UMÁN', '102': 'VALLADOLID', '103': 'XOCCHEL', '104': 'YAXCABÁ', '105': 'YAXKUKUL', '106': 'YOBAÍN', '999': 'NO ESPECIFICADO', '001': 'APOZOL', '002': 'APULCO', '003': 'ATOLINGA', '004': 'BENITO JUÁREZ', '005': 'CALERA', '006': 'CAÑITAS DE FELIPE PESCADOR', '007': 'CONCEPCIÓN DEL ORO', '008': 'CUAUHTÉMOC', '009': 'CHALCHIHUITES', '010': 'FRESNILLO', '011': 'TRINIDAD GARCÍA DE LA CADENA', '012': 'GENARO CODINA', '013': 'GENERAL ENRIQUE ESTRADA', '014': 'GENERAL FRANCISCO R. MURGUÍA', '015': 'EL PLATEADO DE JOAQUÍN AMARO', '016': 'GENERAL PÁNFILO NATERA', '017': 'GUADALUPE', '018': 'HUANUSCO', '019': 'JALPA', '020': 'JEREZ', '021': 'JIMÉNEZ DEL TEUL', '022': 'JUAN ALDAMA', '023': 'JUCHIPILA', '024': 'LORETO', '025': 'LUIS MOYA', '026': 'MAZAPIL', '027': 'MELCHOR OCAMPO', '028': 'MEZQUITAL DEL ORO', '029': 'MIGUEL AUZA', '030': 'MOMAX', '031': 'MONTE ESCOBEDO', '032': 'MORELOS', '033': 'MOYAHUA DE ESTRADA', '034': 'NOCHISTLÁN DE MEJÍA', '035': 'NORIA DE ÁNGELES', '036': 'OJOCALIENTE', '037': 'PÁNUCO', '038': 'PINOS', '039': 'RÍO GRANDE', '040': 'SAIN ALTO', '041': 'EL SALVADOR', '042': 'SOMBRERETE', '043': 'SUSTICACÁN', '044': 'TABASCO', '045': 'TEPECHITLÁN', '046': 'TEPETONGO', '047': 'TEÚL DE GONZÁLEZ ORTEGA', '048': 'TLALTENANGO DE SÁNCHEZ ROMÁN', '049': 'VALPARAÍSO', '050': 'VETAGRANDE', '051': 'VILLA DE COS', '052': 'VILLA GARCÍA', '053': 'VILLA GONZÁLEZ ORTEGA', '054': 'VILLA HIDALGO', '055': 'VILLANUEVA', '056': 'ZACATECAS', '057': 'TRANCOSO', '058': 'SANTA MARÍA DE LA PAZ', '999': 'NO ESPECIFICADO', '999': 'NO ESPECIFICADO', '997': 'NO APLICA', '998': 'SE IGNORA', '999': 'NO ESPECIFICADO', }
dict_PACIENTE = {'1': "AMBULATORIO", '2': "HOSPITALIZADO", '99': 'NO ESPECIFICADO'}


In [160]:
dict_MUNICIPIO = {'001': 'AGUASCALIENTES', '002': 'ASIENTOS', '003': 'CALVILLO', '004': 'COSÍO', '005': 'JESÚS MARÍA', '006': 'PABELLÓN DE ARTEAGA', '007': 'RINCÓN DE ROMOS', '008': 'SAN JOSÉ DE GRACIA', '009': 'TEPEZALÁ', '010': 'EL LLANO', '011': 'SAN FRANCISCO DE LOS ROMO', '999': 'NO ESPECIFICADO', '001': 'ENSENADA', '002': 'MEXICALI', '003': 'TECATE', '004': 'TIJUANA', '005': 'PLAYAS DE ROSARITO', '999': 'NO ESPECIFICADO', '001': 'COMONDÚ', '002': 'MULEGÉ', '003': 'LA PAZ', '008': 'LOS CABOS', '009': 'LORETO', '999': 'NO ESPECIFICADO', '001': 'CALKINÍ', '002': 'CAMPECHE', '003': 'CARMEN', '004': 'CHAMPOTÓN', '005': 'HECELCHAKÁN', '006': 'HOPELCHÉN', '007': 'PALIZADA', '008': 'TENABO', '009': 'ESCÁRCEGA', '010': 'CALAKMUL', '011': 'CANDELARIA', '999': 'NO ESPECIFICADO', '001': 'ABASOLO', '002': 'ACUÑA', '003': 'ALLENDE', '004': 'ARTEAGA', '005': 'CANDELA', '006': 'CASTAÑOS', '007': 'CUATRO CIÉNEGAS', '008': 'ESCOBEDO', '009': 'FRANCISCO I. MADERO', '010': 'FRONTERA', '011': 'GENERAL CEPEDA', '012': 'GUERRERO', '013': 'HIDALGO', '014': 'JIMÉNEZ', '015': 'JUÁREZ', '016': 'LAMADRID', '017': 'MATAMOROS', '018': 'MONCLOVA', '019': 'MORELOS', '020': 'MÚZQUIZ', '021': 'NADADORES', '022': 'NAVA', '023': 'OCAMPO', '024': 'PARRAS', '025': 'PIEDRAS NEGRAS', '026': 'PROGRESO', '027': 'RAMOS ARIZPE', '028': 'SABINAS', '029': 'SACRAMENTO', '030': 'SALTILLO', '031': 'SAN BUENAVENTURA', '032': 'SAN JUAN DE SABINAS', '033': 'SAN PEDRO', '034': 'SIERRA MOJADA', '035': 'TORREÓN', '036': 'VIESCA', '037': 'VILLA UNIÓN', '038': 'ZARAGOZA', '999': 'NO ESPECIFICADO', '001': 'ARMERÍA', '002': 'COLIMA', '003': 'COMALA', '004': 'COQUIMATLÁN', '005': 'CUAUHTÉMOC', '006': 'IXTLAHUACÁN', '007': 'MANZANILLO', '008': 'MINATITLÁN', '009': 'TECOMÁN', '010': 'VILLA DE ÁLVAREZ', '999': 'NO ESPECIFICADO', '001': 'ACACOYAGUA', '002': 'ACALA', '003': 'ACAPETAHUA', '004': 'ALTAMIRANO', '005': 'AMATÁN', '006': 'AMATENANGO DE LA FRONTERA', '007': 'AMATENANGO DEL VALLE', '008': 'ANGEL ALBINO CORZO', '009': 'ARRIAGA', '010': 'BEJUCAL DE OCAMPO', '011': 'BELLA VISTA', '012': 'BERRIOZÁBAL', '013': 'BOCHIL', '014': 'EL BOSQUE', '015': 'CACAHOATÁN', '016': 'CATAZAJÁ', '017': 'CINTALAPA', '018': 'COAPILLA', '019': 'COMITÁN DE DOMÍNGUEZ', '020': 'LA CONCORDIA', '021': 'COPAINALÁ', '022': 'CHALCHIHUITÁN', '023': 'CHAMULA', '024': 'CHANAL', '025': 'CHAPULTENANGO', '026': 'CHENALHÓ', '027': 'CHIAPA DE CORZO', '028': 'CHIAPILLA', '029': 'CHICOASÉN', '030': 'CHICOMUSELO', '031': 'CHILÓN', '032': 'ESCUINTLA', '033': 'FRANCISCO LEÓN', '034': 'FRONTERA COMALAPA', '035': 'FRONTERA HIDALGO', '036': 'LA GRANDEZA', '037': 'HUEHUETÁN', '038': 'HUIXTÁN', '039': 'HUITIUPÁN', '040': 'HUIXTLA', '041': 'LA INDEPENDENCIA', '042': 'IXHUATÁN', '043': 'IXTACOMITÁN', '044': 'IXTAPA', '045': 'IXTAPANGAJOYA', '046': 'JIQUIPILAS', '047': 'JITOTOL', '048': 'JUÁREZ', '049': 'LARRÁINZAR', '050': 'LA LIBERTAD', '051': 'MAPASTEPEC', '052': 'LAS MARGARITAS', '053': 'MAZAPA DE MADERO', '054': 'MAZATÁN', '055': 'METAPA', '056': 'MITONTIC', '057': 'MOTOZINTLA', '058': 'NICOLÁS RUÍZ', '059': 'OCOSINGO', '060': 'OCOTEPEC', '061': 'OCOZOCOAUTLA DE ESPINOSA', '062': 'OSTUACÁN', '063': 'OSUMACINTA', '064': 'OXCHUC', '065': 'PALENQUE', '066': 'PANTELHÓ', '067': 'PANTEPEC', '068': 'PICHUCALCO', '069': 'PIJIJIAPAN', '070': 'EL PORVENIR', '071': 'VILLA COMALTITLÁN', '072': 'PUEBLO NUEVO SOLISTAHUACÁN', '073': 'RAYÓN', '074': 'REFORMA', '075': 'LAS ROSAS', '076': 'SABANILLA', '077': 'SALTO DE AGUA', '078': 'SAN CRISTÓBAL DE LAS CASAS', '079': 'SAN FERNANDO', '080': 'SILTEPEC', '081': 'SIMOJOVEL', '082': 'SITALÁ', '083': 'SOCOLTENANGO', '084': 'SOLOSUCHIAPA', '085': 'SOYALÓ', '086': 'SUCHIAPA', '087': 'SUCHIATE', '088': 'SUNUAPA', '089': 'TAPACHULA', '090': 'TAPALAPA', '091': 'TAPILULA', '092': 'TECPATÁN', '093': 'TENEJAPA', '094': 'TEOPISCA', '096': 'TILA', '097': 'TONALÁ', '098': 'TOTOLAPA', '099': 'LA TRINITARIA', '100': 'TUMBALÁ', '101': 'TUXTLA GUTIÉRREZ', '102': 'TUXTLA CHICO', '103': 'TUZANTÁN', '104': 'TZIMOL', '105': 'UNIÓN JUÁREZ', '106': 'VENUSTIANO CARRANZA', '107': 'VILLA CORZO', '108': 'VILLAFLORES', '109': 'YAJALÓN', '110': 'SAN LUCAS', '111': 'ZINACANTÁN', '112': 'SAN JUAN CANCUC', '113': 'ALDAMA', '114': 'BENEMÉRITO DE LAS AMÉRICAS', '115': 'MARAVILLA TENEJAPA', '116': 'MARQUÉS DE COMILLAS', '117': 'MONTECRISTO DE GUERRERO', '118': 'SAN ANDRÉS DURAZNAL', '119': 'SANTIAGO EL PINAR', '120': 'CAPITÁN LUIS ÁNGEL VIDAL', '121': 'RINCÓN CHAMULA SAN PEDRO', '122': 'EL PARRAL', '123': 'EMILIANO ZAPATA', '124': 'MEZCALAPA', '999': 'NO ESPECIFICADO', '001': 'AHUMADA', '002': 'ALDAMA', '003': 'ALLENDE', '004': 'AQUILES SERDÁN', '005': 'ASCENSIÓN', '006': 'BACHÍNIVA', '007': 'BALLEZA', '008': 'BATOPILAS DE MANUEL GÓMEZ MORÍN', '009': 'BOCOYNA', '010': 'BUENAVENTURA', '011': 'CAMARGO', '012': 'CARICHÍ', '013': 'CASAS GRANDES', '014': 'CORONADO', '015': 'COYAME DEL SOTOL', '016': 'LA CRUZ', '017': 'CUAUHTÉMOC', '018': 'CUSIHUIRIACHI', '019': 'CHIHUAHUA', '020': 'CHÍNIPAS', '021': 'DELICIAS', '022': 'DR. BELISARIO DOMÍNGUEZ', '023': 'GALEANA', '024': 'SANTA ISABEL', '025': 'GÓMEZ FARÍAS', '026': 'GRAN MORELOS', '027': 'GUACHOCHI', '028': 'GUADALUPE', '029': 'GUADALUPE Y CALVO', '030': 'GUAZAPARES', '031': 'GUERRERO', '032': 'HIDALGO DEL PARRAL', '033': 'HUEJOTITÁN', '034': 'IGNACIO ZARAGOZA', '035': 'JANOS', '036': 'JIMÉNEZ', '037': 'JUÁREZ', '038': 'JULIMES', '039': 'LÓPEZ', '040': 'MADERA', '041': 'MAGUARICHI', '042': 'MANUEL BENAVIDES', '043': 'MATACHÍ', '044': 'MATAMOROS', '045': 'MEOQUI', '046': 'MORELOS', '047': 'MORIS', '048': 'NAMIQUIPA', '049': 'NONOAVA', '050': 'NUEVO CASAS GRANDES', '051': 'OCAMPO', '052': 'OJINAGA', '053': 'PRAXEDIS G. GUERRERO', '054': 'RIVA PALACIO', '055': 'ROSALES', '056': 'ROSARIO', '057': 'SAN FRANCISCO DE BORJA', '058': 'SAN FRANCISCO DE CONCHOS', '059': 'SAN FRANCISCO DEL ORO', '060': 'SANTA BÁRBARA', '061': 'SATEVÓ', '062': 'SAUCILLO', '063': 'TEMÓSACHIC', '064': 'EL TULE', '065': 'URIQUE', '066': 'URUACHI', '067': 'VALLE DE ZARAGOZA', '999': 'NO ESPECIFICADO', '002': 'AZCAPOTZALCO', '003': 'COYOACÁN', '004': 'CUAJIMALPA DE MORELOS', '005': 'GUSTAVO A. MADERO', '006': 'IZTACALCO', '007': 'IZTAPALAPA', '008': 'LA MAGDALENA CONTRERAS', '009': 'MILPA ALTA', '010': 'ÁLVARO OBREGÓN', '011': 'TLÁHUAC', '012': 'TLALPAN', '013': 'XOCHIMILCO', '014': 'BENITO JUÁREZ', '015': 'CUAUHTÉMOC', '016': 'MIGUEL HIDALGO', '017': 'VENUSTIANO CARRANZA', '999': 'NO ESPECIFICADO', '001': 'CANATLÁN', '002': 'CANELAS', '003': 'CONETO DE COMONFORT', '004': 'CUENCAMÉ', '005': 'DURANGO', '006': 'GENERAL SIMÓN BOLÍVAR', '007': 'GÓMEZ PALACIO', '008': 'GUADALUPE VICTORIA', '009': 'GUANACEVÍ', '010': 'HIDALGO', '011': 'INDÉ', '012': 'LERDO', '013': 'MAPIMÍ', '014': 'MEZQUITAL', '015': 'NAZAS', '016': 'NOMBRE DE DIOS', '017': 'OCAMPO', '018': 'EL ORO', '019': 'OTÁEZ', '020': 'PÁNUCO DE CORONADO', '021': 'PEÑÓN BLANCO', '022': 'POANAS', '023': 'PUEBLO NUEVO', '024': 'RODEO', '025': 'SAN BERNARDO', '026': 'SAN DIMAS', '027': 'SAN JUAN DE GUADALUPE', '028': 'SAN JUAN DEL RÍO', '029': 'SAN LUIS DEL CORDERO', '030': 'SAN PEDRO DEL GALLO', '031': 'SANTA CLARA', '032': 'SANTIAGO PAPASQUIARO', '033': 'SÚCHIL', '034': 'TAMAZULA', '035': 'TEPEHUANES', '036': 'TLAHUALILO', '037': 'TOPIA', '038': 'VICENTE GUERRERO', '039': 'NUEVO IDEAL', '999': 'NO ESPECIFICADO', '001': 'ABASOLO', '002': 'ACÁMBARO', '003': 'SAN MIGUEL DE ALLENDE', '004': 'APASEO EL ALTO', '005': 'APASEO EL GRANDE', '006': 'ATARJEA', '007': 'CELAYA', '008': 'MANUEL DOBLADO', '009': 'COMONFORT', '010': 'CORONEO', '011': 'CORTAZAR', '012': 'CUERÁMARO', '013': 'DOCTOR MORA', '014': 'DOLORES HIDALGO CUNA DE LA INDEPENDENCIA NACIONAL', '015': 'GUANAJUATO', '016': 'HUANÍMARO', '017': 'IRAPUATO', '018': 'JARAL DEL PROGRESO', '019': 'JERÉCUARO', '020': 'LEÓN', '021': 'MOROLEÓN', '022': 'OCAMPO', '023': 'PÉNJAMO', '024': 'PUEBLO NUEVO', '025': 'PURÍSIMA DEL RINCÓN', '026': 'ROMITA', '027': 'SALAMANCA', '028': 'SALVATIERRA', '029': 'SAN DIEGO DE LA UNIÓN', '030': 'SAN FELIPE', '031': 'SAN FRANCISCO DEL RINCÓN', '032': 'SAN JOSÉ ITURBIDE', '033': 'SAN LUIS DE LA PAZ', '034': 'SANTA CATARINA', '035': 'SANTA CRUZ DE JUVENTINO ROSAS', '036': 'SANTIAGO MARAVATÍO', '037': 'SILAO DE LA VICTORIA', '038': 'TARANDACUAO', '039': 'TARIMORO', '040': 'TIERRA BLANCA', '041': 'URIANGATO', '042': 'VALLE DE SANTIAGO', '043': 'VICTORIA', '044': 'VILLAGRÁN', '045': 'XICHÚ', '046': 'YURIRIA', '999': 'NO ESPECIFICADO', '001': 'ACAPULCO DE JUÁREZ', '002': 'AHUACUOTZINGO', '003': 'AJUCHITLÁN DEL PROGRESO', '004': 'ALCOZAUCA DE GUERRERO', '005': 'ALPOYECA', '006': 'APAXTLA', '007': 'ARCELIA', '008': 'ATENANGO DEL RÍO', '009': 'ATLAMAJALCINGO DEL MONTE', '010': 'ATLIXTAC', '011': 'ATOYAC DE ÁLVAREZ', '012': 'AYUTLA DE LOS LIBRES', '013': 'AZOYÚ', '014': 'BENITO JUÁREZ', '015': 'BUENAVISTA DE CUÉLLAR', '016': 'COAHUAYUTLA DE JOSÉ MARÍA IZAZAGA', '017': 'COCULA', '018': 'COPALA', '019': 'COPALILLO', '020': 'COPANATOYAC', '021': 'COYUCA DE BENÍTEZ', '022': 'COYUCA DE CATALÁN', '023': 'CUAJINICUILAPA', '024': 'CUALÁC', '025': 'CUAUTEPEC', '026': 'CUETZALA DEL PROGRESO', '027': 'CUTZAMALA DE PINZÓN', '028': 'CHILAPA DE ÁLVAREZ', '029': 'CHILPANCINGO DE LOS BRAVO', '030': 'FLORENCIO VILLARREAL', '031': 'GENERAL CANUTO A. NERI', '032': 'GENERAL HELIODORO CASTILLO', '033': 'HUAMUXTITLÁN', '034': 'HUITZUCO DE LOS FIGUEROA', '035': 'IGUALA DE LA INDEPENDENCIA', '036': 'IGUALAPA', '037': 'IXCATEOPAN DE CUAUHTÉMOC', '038': 'ZIHUATANEJO DE AZUETA', '039': 'JUAN R. ESCUDERO', '040': 'LEONARDO BRAVO', '041': 'MALINALTEPEC', '042': 'MÁRTIR DE CUILAPAN', '043': 'METLATÓNOC', '044': 'MOCHITLÁN', '045': 'OLINALÁ', '046': 'OMETEPEC', '047': 'PEDRO ASCENCIO ALQUISIRAS', '048': 'PETATLÁN', '049': 'PILCAYA', '050': 'PUNGARABATO', '051': 'QUECHULTENANGO', '052': 'SAN LUIS ACATLÁN', '053': 'SAN MARCOS', '054': 'SAN MIGUEL TOTOLAPAN', '055': 'TAXCO DE ALARCÓN', '056': 'TECOANAPA', '057': 'TÉCPAN DE GALEANA', '058': 'TELOLOAPAN', '059': 'TEPECOACUILCO DE TRUJANO', '060': 'TETIPAC', '061': 'TIXTLA DE GUERRERO', '062': 'TLACOACHISTLAHUACA', '063': 'TLACOAPA', '064': 'TLALCHAPA', '065': 'TLALIXTAQUILLA DE MALDONADO', '066': 'TLAPA DE COMONFORT', '067': 'TLAPEHUALA', '068': 'LA UNIÓN DE ISIDORO MONTES DE OCA', '069': 'XALPATLÁHUAC', '070': 'XOCHIHUEHUETLÁN', '071': 'XOCHISTLAHUACA', '072': 'ZAPOTITLÁN TABLAS', '073': 'ZIRÁNDARO', '074': 'ZITLALA', '075': 'EDUARDO NERI', '076': 'ACATEPEC', '077': 'MARQUELIA', '078': 'COCHOAPA EL GRANDE', '079': 'JOSÉ JOAQUÍN DE HERRERA', '080': 'JUCHITÁN', '081': 'ILIATENCO', '999': 'NO ESPECIFICADO', '001': 'ACATLÁN', '002': 'ACAXOCHITLÁN', '003': 'ACTOPAN', '004': 'AGUA BLANCA DE ITURBIDE', '005': 'AJACUBA', '006': 'ALFAJAYUCAN', '007': 'ALMOLOYA', '008': 'APAN', '009': 'EL ARENAL', '010': 'ATITALAQUIA', '011': 'ATLAPEXCO', '012': 'ATOTONILCO EL GRANDE', '013': 'ATOTONILCO DE TULA', '014': 'CALNALI', '015': 'CARDONAL', '016': 'CUAUTEPEC DE HINOJOSA', '017': 'CHAPANTONGO', '018': 'CHAPULHUACÁN', '019': 'CHILCUAUTLA', '020': 'ELOXOCHITLÁN', '021': 'EMILIANO ZAPATA', '022': 'EPAZOYUCAN', '023': 'FRANCISCO I. MADERO', '024': 'HUASCA DE OCAMPO', '025': 'HUAUTLA', '026': 'HUAZALINGO', '027': 'HUEHUETLA', '028': 'HUEJUTLA DE REYES', '029': 'HUICHAPAN', '030': 'IXMIQUILPAN', '031': 'JACALA DE LEDEZMA', '032': 'JALTOCÁN', '033': 'JUÁREZ HIDALGO', '034': 'LOLOTLA', '035': 'METEPEC', '036': 'SAN AGUSTÍN METZQUITITLÁN', '037': 'METZTITLÁN', '038': 'MINERAL DEL CHICO', '039': 'MINERAL DEL MONTE', '040': 'LA MISIÓN', '041': 'MIXQUIAHUALA DE JUÁREZ', '042': 'MOLANGO DE ESCAMILLA', '043': 'NICOLÁS FLORES', '044': 'NOPALA DE VILLAGRÁN', '045': 'OMITLÁN DE JUÁREZ', '046': 'SAN FELIPE ORIZATLÁN', '047': 'PACULA', '048': 'PACHUCA DE SOTO', '049': 'PISAFLORES', '050': 'PROGRESO DE OBREGÓN', '051': 'MINERAL DE LA REFORMA', '052': 'SAN AGUSTÍN TLAXIACA', '053': 'SAN BARTOLO TUTOTEPEC', '054': 'SAN SALVADOR', '055': 'SANTIAGO DE ANAYA', '056': 'SANTIAGO TULANTEPEC DE LUGO GUERRERO', '057': 'SINGUILUCAN', '058': 'TASQUILLO', '059': 'TECOZAUTLA', '060': 'TENANGO DE DORIA', '061': 'TEPEAPULCO', '062': 'TEPEHUACÁN DE GUERRERO', '063': 'TEPEJI DEL RÍO DE OCAMPO', '064': 'TEPETITLÁN', '065': 'TETEPANGO', '066': 'VILLA DE TEZONTEPEC', '067': 'TEZONTEPEC DE ALDAMA', '068': 'TIANGUISTENGO', '069': 'TIZAYUCA', '070': 'TLAHUELILPAN', '071': 'TLAHUILTEPA', '072': 'TLANALAPA', '073': 'TLANCHINOL', '074': 'TLAXCOAPAN', '075': 'TOLCAYUCA', '076': 'TULA DE ALLENDE', '077': 'TULANCINGO DE BRAVO', '078': 'XOCHIATIPAN', '079': 'XOCHICOATLÁN', '080': 'YAHUALICA', '081': 'ZACUALTIPÁN DE ÁNGELES', '082': 'ZAPOTLÁN DE JUÁREZ', '083': 'ZEMPOALA', '084': 'ZIMAPÁN', '999': 'NO ESPECIFICADO', '001': 'ACATIC', '002': 'ACATLÁN DE JUÁREZ', '003': 'AHUALULCO DE MERCADO', '004': 'AMACUECA', '005': 'AMATITÁN', '006': 'AMECA', '007': 'SAN JUANITO DE ESCOBEDO', '008': 'ARANDAS', '009': 'EL ARENAL', '010': 'ATEMAJAC DE BRIZUELA', '011': 'ATENGO', '012': 'ATENGUILLO', '013': 'ATOTONILCO EL ALTO', '014': 'ATOYAC', '015': 'AUTLÁN DE NAVARRO', '016': 'AYOTLÁN', '017': 'AYUTLA', '018': 'LA BARCA', '019': 'BOLAÑOS', '020': 'CABO CORRIENTES', '021': 'CASIMIRO CASTILLO', '022': 'CIHUATLÁN', '023': 'ZAPOTLÁN EL GRANDE', '024': 'COCULA', '025': 'COLOTLÁN', '026': 'CONCEPCIÓN DE BUENOS AIRES', '027': 'CUAUTITLÁN DE GARCÍA BARRAGÁN', '028': 'CUAUTLA', '029': 'CUQUÍO', '030': 'CHAPALA', '031': 'CHIMALTITÁN', '032': 'CHIQUILISTLÁN', '033': 'DEGOLLADO', '034': 'EJUTLA', '035': 'ENCARNACIÓN DE DÍAZ', '036': 'ETZATLÁN', '037': 'EL GRULLO', '038': 'GUACHINANGO', '039': 'GUADALAJARA', '040': 'HOSTOTIPAQUILLO', '041': 'HUEJÚCAR', '042': 'HUEJUQUILLA EL ALTO', '043': 'LA HUERTA', '044': 'IXTLAHUACÁN DE LOS MEMBRILLOS', '045': 'IXTLAHUACÁN DEL RÍO', '046': 'JALOSTOTITLÁN', '047': 'JAMAY', '048': 'JESÚS MARÍA', '049': 'JILOTLÁN DE LOS DOLORES', '050': 'JOCOTEPEC', '051': 'JUANACATLÁN', '052': 'JUCHITLÁN', '053': 'LAGOS DE MORENO', '054': 'EL LIMÓN', '055': 'MAGDALENA', '056': 'SANTA MARÍA DEL ORO', '057': 'LA MANZANILLA DE LA PAZ', '058': 'MASCOTA', '059': 'MAZAMITLA', '060': 'MEXTICACÁN', '061': 'MEZQUITIC', '062': 'MIXTLÁN', '063': 'OCOTLÁN', '064': 'OJUELOS DE JALISCO', '065': 'PIHUAMO', '066': 'PONCITLÁN', '067': 'PUERTO VALLARTA', '068': 'VILLA PURIFICACIÓN', '069': 'QUITUPAN', '070': 'EL SALTO', '071': 'SAN CRISTÓBAL DE LA BARRANCA', '072': 'SAN DIEGO DE ALEJANDRÍA', '073': 'SAN JUAN DE LOS LAGOS', '074': 'SAN JULIÁN', '075': 'SAN MARCOS', '076': 'SAN MARTÍN DE BOLAÑOS', '077': 'SAN MARTÍN HIDALGO', '078': 'SAN MIGUEL EL ALTO', '079': 'GÓMEZ FARÍAS', '080': 'SAN SEBASTIÁN DEL OESTE', '081': 'SANTA MARÍA DE LOS ÁNGELES', '082': 'SAYULA', '083': 'TALA', '084': 'TALPA DE ALLENDE', '085': 'TAMAZULA DE GORDIANO', '086': 'TAPALPA', '087': 'TECALITLÁN', '088': 'TECOLOTLÁN', '089': 'TECHALUTA DE MONTENEGRO', '090': 'TENAMAXTLÁN', '091': 'TEOCALTICHE', '092': 'TEOCUITATLÁN DE CORONA', '093': 'TEPATITLÁN DE MORELOS', '094': 'TEQUILA', '095': 'TEUCHITLÁN', '096': 'TIZAPÁN EL ALTO', '097': 'TLAJOMULCO DE ZÚÑIGA', '098': 'SAN PEDRO TLAQUEPAQUE', '099': 'TOLIMÁN', '100': 'TOMATLÁN', '101': 'TONALÁ', '102': 'TONAYA', '103': 'TONILA', '104': 'TOTATICHE', '105': 'TOTOTLÁN', '106': 'TUXCACUESCO', '107': 'TUXCUECA', '108': 'TUXPAN', '109': 'UNIÓN DE SAN ANTONIO', '110': 'UNIÓN DE TULA', '111': 'VALLE DE GUADALUPE', '112': 'VALLE DE JUÁREZ', '113': 'SAN GABRIEL', '114': 'VILLA CORONA', '115': 'VILLA GUERRERO', '116': 'VILLA HIDALGO', '117': 'CAÑADAS DE OBREGÓN', '118': 'YAHUALICA DE GONZÁLEZ GALLO', '119': 'ZACOALCO DE TORRES', '120': 'ZAPOPAN', '121': 'ZAPOTILTIC', '122': 'ZAPOTITLÁN DE VADILLO', '123': 'ZAPOTLÁN DEL REY', '124': 'ZAPOTLANEJO', '125': 'SAN IGNACIO CERRO GORDO', '999': 'NO ESPECIFICADO', '001': 'ACAMBAY DE RUÍZ CASTAÑEDA', '002': 'ACOLMAN', '003': 'ACULCO', '004': 'ALMOLOYA DE ALQUISIRAS', '005': 'ALMOLOYA DE JUÁREZ', '006': 'ALMOLOYA DEL RÍO', '007': 'AMANALCO', '008': 'AMATEPEC', '009': 'AMECAMECA', '010': 'APAXCO', '011': 'ATENCO', '012': 'ATIZAPÁN', '013': 'ATIZAPÁN DE ZARAGOZA', '014': 'ATLACOMULCO', '015': 'ATLAUTLA', '016': 'AXAPUSCO', '017': 'AYAPANGO', '018': 'CALIMAYA', '019': 'CAPULHUAC', '020': 'COACALCO DE BERRIOZÁBAL', '021': 'COATEPEC HARINAS', '022': 'COCOTITLÁN', '023': 'COYOTEPEC', '024': 'CUAUTITLÁN', '025': 'CHALCO', '026': 'CHAPA DE MOTA', '027': 'CHAPULTEPEC', '028': 'CHIAUTLA', '029': 'CHICOLOAPAN', '030': 'CHICONCUAC', '031': 'CHIMALHUACÁN', '032': 'DONATO GUERRA', '033': 'ECATEPEC DE MORELOS', '034': 'ECATZINGO', '035': 'HUEHUETOCA', '036': 'HUEYPOXTLA', '037': 'HUIXQUILUCAN', '038': 'ISIDRO FABELA', '039': 'IXTAPALUCA', '040': 'IXTAPAN DE LA SAL', '041': 'IXTAPAN DEL ORO', '042': 'IXTLAHUACA', '043': 'XALATLACO', '044': 'JALTENCO', '045': 'JILOTEPEC', '046': 'JILOTZINGO', '047': 'JIQUIPILCO', '048': 'JOCOTITLÁN', '049': 'JOQUICINGO', '050': 'JUCHITEPEC', '051': 'LERMA', '052': 'MALINALCO', '053': 'MELCHOR OCAMPO', '054': 'METEPEC', '055': 'MEXICALTZINGO', '056': 'MORELOS', '057': 'NAUCALPAN DE JUÁREZ', '058': 'NEZAHUALCÓYOTL', '059': 'NEXTLALPAN', '060': 'NICOLÁS ROMERO', '061': 'NOPALTEPEC', '062': 'OCOYOACAC', '063': 'OCUILAN', '064': 'EL ORO', '065': 'OTUMBA', '066': 'OTZOLOAPAN', '067': 'OTZOLOTEPEC', '068': 'OZUMBA', '069': 'PAPALOTLA', '070': 'LA PAZ', '071': 'POLOTITLÁN', '072': 'RAYÓN', '073': 'SAN ANTONIO LA ISLA', '074': 'SAN FELIPE DEL PROGRESO', '075': 'SAN MARTÍN DE LAS PIRÁMIDES', '076': 'SAN MATEO ATENCO', '077': 'SAN SIMÓN DE GUERRERO', '078': 'SANTO TOMÁS', '079': 'SOYANIQUILPAN DE JUÁREZ', '080': 'SULTEPEC', '081': 'TECÁMAC', '082': 'TEJUPILCO', '083': 'TEMAMATLA', '084': 'TEMASCALAPA', '085': 'TEMASCALCINGO', '086': 'TEMASCALTEPEC', '087': 'TEMOAYA', '088': 'TENANCINGO', '089': 'TENANGO DEL AIRE', '090': 'TENANGO DEL VALLE', '091': 'TEOLOYUCAN', '092': 'TEOTIHUACÁN', '093': 'TEPETLAOXTOC', '094': 'TEPETLIXPA', '095': 'TEPOTZOTLÁN', '096': 'TEQUIXQUIAC', '097': 'TEXCALTITLÁN', '098': 'TEXCALYACAC', '099': 'TEXCOCO', '100': 'TEZOYUCA', '101': 'TIANGUISTENCO', '102': 'TIMILPAN', '103': 'TLALMANALCO', '104': 'TLALNEPANTLA DE BAZ', '105': 'TLATLAYA', '106': 'TOLUCA', '107': 'TONATICO', '108': 'TULTEPEC', '109': 'TULTITLÁN', '110': 'VALLE DE BRAVO', '111': 'VILLA DE ALLENDE', '112': 'VILLA DEL CARBÓN', '113': 'VILLA GUERRERO', '114': 'VILLA VICTORIA', '115': 'XONACATLÁN', '116': 'ZACAZONAPAN', '117': 'ZACUALPAN', '118': 'ZINACANTEPEC', '119': 'ZUMPAHUACÁN', '120': 'ZUMPANGO', '121': 'CUAUTITLÁN IZCALLI', '122': 'VALLE DE CHALCO SOLIDARIDAD', '123': 'LUVIANOS', '124': 'SAN JOSÉ DEL RINCÓN', '125': 'TONANITLA', '999': 'NO ESPECIFICADO', '001': 'ACUITZIO', '002': 'AGUILILLA', '003': 'ÁLVARO OBREGÓN', '004': 'ANGAMACUTIRO', '005': 'ANGANGUEO', '006': 'APATZINGÁN', '007': 'APORO', '008': 'AQUILA', '009': 'ARIO', '010': 'ARTEAGA', '011': 'BRISEÑAS', '012': 'BUENAVISTA', '013': 'CARÁCUARO', '014': 'COAHUAYANA', '015': 'COALCOMÁN DE VÁZQUEZ PALLARES', '016': 'COENEO', '017': 'CONTEPEC', '018': 'COPÁNDARO', '019': 'COTIJA', '020': 'CUITZEO', '021': 'CHARAPAN', '022': 'CHARO', '023': 'CHAVINDA', '024': 'CHERÁN', '025': 'CHILCHOTA', '026': 'CHINICUILA', '027': 'CHUCÁNDIRO', '028': 'CHURINTZIO', '029': 'CHURUMUCO', '030': 'ECUANDUREO', '031': 'EPITACIO HUERTA', '032': 'ERONGARÍCUARO', '033': 'GABRIEL ZAMORA', '034': 'HIDALGO', '035': 'LA HUACANA', '036': 'HUANDACAREO', '037': 'HUANIQUEO', '038': 'HUETAMO', '039': 'HUIRAMBA', '040': 'INDAPARAPEO', '041': 'IRIMBO', '042': 'IXTLÁN', '043': 'JACONA', '044': 'JIMÉNEZ', '045': 'JIQUILPAN', '046': 'JUÁREZ', '047': 'JUNGAPEO', '048': 'LAGUNILLAS', '049': 'MADERO', '050': 'MARAVATÍO', '051': 'MARCOS CASTELLANOS', '052': 'LÁZARO CÁRDENAS', '053': 'MORELIA', '054': 'MORELOS', '055': 'MÚGICA', '056': 'NAHUATZEN', '057': 'NOCUPÉTARO', '058': 'NUEVO PARANGARICUTIRO', '059': 'NUEVO URECHO', '060': 'NUMARÁN', '061': 'OCAMPO', '062': 'PAJACUARÁN', '063': 'PANINDÍCUARO', '064': 'PARÁCUARO', '065': 'PARACHO', '066': 'PÁTZCUARO', '067': 'PENJAMILLO', '068': 'PERIBÁN', '069': 'LA PIEDAD', '070': 'PURÉPERO', '071': 'PURUÁNDIRO', '072': 'QUERÉNDARO', '073': 'QUIROGA', '074': 'COJUMATLÁN DE RÉGULES', '075': 'LOS REYES', '076': 'SAHUAYO', '077': 'SAN LUCAS', '078': 'SANTA ANA MAYA', '079': 'SALVADOR ESCALANTE', '080': 'SENGUIO', '081': 'SUSUPUATO', '082': 'TACÁMBARO', '083': 'TANCÍTARO', '084': 'TANGAMANDAPIO', '085': 'TANGANCÍCUARO', '086': 'TANHUATO', '087': 'TARETAN', '088': 'TARÍMBARO', '089': 'TEPALCATEPEC', '090': 'TINGAMBATO', '091': 'TINGÜINDÍN', '092': 'TIQUICHEO DE NICOLÁS ROMERO', '093': 'TLALPUJAHUA', '094': 'TLAZAZALCA', '095': 'TOCUMBO', '096': 'TUMBISCATÍO', '097': 'TURICATO', '098': 'TUXPAN', '099': 'TUZANTLA', '100': 'TZINTZUNTZAN', '101': 'TZITZIO', '102': 'URUAPAN', '103': 'VENUSTIANO CARRANZA', '104': 'VILLAMAR', '105': 'VISTA HERMOSA', '106': 'YURÉCUARO', '107': 'ZACAPU', '108': 'ZAMORA', '109': 'ZINÁPARO', '110': 'ZINAPÉCUARO', '111': 'ZIRACUARETIRO', '112': 'ZITÁCUARO', '113': 'JOSÉ SIXTO VERDUZCO', '999': 'NO ESPECIFICADO', '001': 'AMACUZAC', '002': 'ATLATLAHUCAN', '003': 'AXOCHIAPAN', '004': 'AYALA', '005': 'COATLÁN DEL RÍO', '006': 'CUAUTLA', '007': 'CUERNAVACA', '008': 'EMILIANO ZAPATA', '009': 'HUITZILAC', '010': 'JANTETELCO', '011': 'JIUTEPEC', '012': 'JOJUTLA', '013': 'JONACATEPEC DE LEANDRO VALLE', '014': 'MAZATEPEC', '015': 'MIACATLÁN', '016': 'OCUITUCO', '017': 'PUENTE DE IXTLA', '018': 'TEMIXCO', '019': 'TEPALCINGO', '020': 'TEPOZTLÁN', '021': 'TETECALA', '022': 'TETELA DEL VOLCÁN', '023': 'TLALNEPANTLA', '024': 'TLALTIZAPÁN DE ZAPATA', '025': 'TLAQUILTENANGO', '026': 'TLAYACAPAN', '027': 'TOTOLAPAN', '028': 'XOCHITEPEC', '029': 'YAUTEPEC', '030': 'YECAPIXTLA', '031': 'ZACATEPEC', '032': 'ZACUALPAN DE AMILPAS', '033': 'TEMOAC', '034': 'COATETELCO', '035': 'XOXOCOTLA', '999': 'NO ESPECIFICADO', '001': 'ACAPONETA', '002': 'AHUACATLÁN', '003': 'AMATLÁN DE CAÑAS', '004': 'COMPOSTELA', '005': 'HUAJICORI', '006': 'IXTLÁN DEL RÍO', '007': 'JALA', '008': 'XALISCO', '009': 'DEL NAYAR', '010': 'ROSAMORADA', '011': 'RUÍZ', '012': 'SAN BLAS', '013': 'SAN PEDRO LAGUNILLAS', '014': 'SANTA MARÍA DEL ORO', '015': 'SANTIAGO IXCUINTLA', '016': 'TECUALA', '017': 'TEPIC', '018': 'TUXPAN', '019': 'LA YESCA', '020': 'BAHÍA DE BANDERAS', '999': 'NO ESPECIFICADO', '001': 'ABASOLO', '002': 'AGUALEGUAS', '003': 'LOS ALDAMAS', '004': 'ALLENDE', '005': 'ANÁHUAC', '006': 'APODACA', '007': 'ARAMBERRI', '008': 'BUSTAMANTE', '009': 'CADEREYTA JIMÉNEZ', '010': 'EL CARMEN', '011': 'CERRALVO', '012': 'CIÉNEGA DE FLORES', '013': 'CHINA', '014': 'DOCTOR ARROYO', '015': 'DOCTOR COSS', '016': 'DOCTOR GONZÁLEZ', '017': 'GALEANA', '018': 'GARCÍA', '019': 'SAN PEDRO GARZA GARCÍA', '020': 'GENERAL BRAVO', '021': 'GENERAL ESCOBEDO', '022': 'GENERAL TERÁN', '023': 'GENERAL TREVIÑO', '024': 'GENERAL ZARAGOZA', '025': 'GENERAL ZUAZUA', '026': 'GUADALUPE', '027': 'LOS HERRERAS', '028': 'HIGUERAS', '029': 'HUALAHUISES', '030': 'ITURBIDE', '031': 'JUÁREZ', '032': 'LAMPAZOS DE NARANJO', '033': 'LINARES', '034': 'MARÍN', '035': 'MELCHOR OCAMPO', '036': 'MIER Y NORIEGA', '037': 'MINA', '038': 'MONTEMORELOS', '039': 'MONTERREY', '040': 'PARÁS', '041': 'PESQUERÍA', '042': 'LOS RAMONES', '043': 'RAYONES', '044': 'SABINAS HIDALGO', '045': 'SALINAS VICTORIA', '046': 'SAN NICOLÁS DE LOS GARZA', '047': 'HIDALGO', '048': 'SANTA CATARINA', '049': 'SANTIAGO', '050': 'VALLECILLO', '051': 'VILLALDAMA', '999': 'NO ESPECIFICADO', '001': 'ABEJONES', '002': 'ACATLÁN DE PÉREZ FIGUEROA', '003': 'ASUNCIÓN CACALOTEPEC', '004': 'ASUNCIÓN CUYOTEPEJI', '005': 'ASUNCIÓN IXTALTEPEC', '006': 'ASUNCIÓN NOCHIXTLÁN', '007': 'ASUNCIÓN OCOTLÁN', '008': 'ASUNCIÓN TLACOLULITA', '009': 'AYOTZINTEPEC', '010': 'EL BARRIO DE LA SOLEDAD', '011': 'CALIHUALÁ', '012': 'CANDELARIA LOXICHA', '013': 'CIÉNEGA DE ZIMATLÁN', '014': 'CIUDAD IXTEPEC', '015': 'COATECAS ALTAS', '016': 'COICOYÁN DE LAS FLORES', '017': 'LA COMPAÑÍA', '018': 'CONCEPCIÓN BUENAVISTA', '019': 'CONCEPCIÓN PÁPALO', '020': 'CONSTANCIA DEL ROSARIO', '021': 'COSOLAPA', '022': 'COSOLTEPEC', '023': 'CUILÁPAM DE GUERRERO', '024': 'CUYAMECALCO VILLA DE ZARAGOZA', '025': 'CHAHUITES', '026': 'CHALCATONGO DE HIDALGO', '027': 'CHIQUIHUITLÁN DE BENITO JUÁREZ', '028': 'HEROICA CIUDAD DE EJUTLA DE CRESPO', '029': 'ELOXOCHITLÁN DE FLORES MAGÓN', '030': 'EL ESPINAL', '031': 'TAMAZULÁPAM DEL ESPÍRITU SANTO', '032': 'FRESNILLO DE TRUJANO', '033': 'GUADALUPE ETLA', '034': 'GUADALUPE DE RAMÍREZ', '035': 'GUELATAO DE JUÁREZ', '036': 'GUEVEA DE HUMBOLDT', '037': 'MESONES HIDALGO', '038': 'VILLA HIDALGO', '039': 'HEROICA CIUDAD DE HUAJUAPAN DE LEÓN', '040': 'HUAUTEPEC', '041': 'HUAUTLA DE JIMÉNEZ', '042': 'IXTLÁN DE JUÁREZ', '043': 'JUCHITÁN DE ZARAGOZA', '044': 'LOMA BONITA', '045': 'MAGDALENA APASCO', '046': 'MAGDALENA JALTEPEC', '047': 'SANTA MAGDALENA JICOTLÁN', '048': 'MAGDALENA MIXTEPEC', '049': 'MAGDALENA OCOTLÁN', '050': 'MAGDALENA PEÑASCO', '051': 'MAGDALENA TEITIPAC', '052': 'MAGDALENA TEQUISISTLÁN', '053': 'MAGDALENA TLACOTEPEC', '054': 'MAGDALENA ZAHUATLÁN', '055': 'MARISCALA DE JUÁREZ', '056': 'MÁRTIRES DE TACUBAYA', '057': 'MATÍAS ROMERO AVENDAÑO', '058': 'MAZATLÁN VILLA DE FLORES', '059': 'MIAHUATLÁN DE PORFIRIO DÍAZ', '060': 'MIXISTLÁN DE LA REFORMA', '061': 'MONJAS', '062': 'NATIVIDAD', '063': 'NAZARENO ETLA', '064': 'NEJAPA DE MADERO', '065': 'IXPANTEPEC NIEVES', '066': 'SANTIAGO NILTEPEC', '067': 'OAXACA DE JUÁREZ', '068': 'OCOTLÁN DE MORELOS', '069': 'LA PE', '070': 'PINOTEPA DE DON LUIS', '071': 'PLUMA HIDALGO', '072': 'SAN JOSÉ DEL PROGRESO', '073': 'PUTLA VILLA DE GUERRERO', '074': 'SANTA CATARINA QUIOQUITANI', '075': 'REFORMA DE PINEDA', '076': 'LA REFORMA', '077': 'REYES ETLA', '078': 'ROJAS DE CUAUHTÉMOC', '079': 'SALINA CRUZ', '080': 'SAN AGUSTÍN AMATENGO', '081': 'SAN AGUSTÍN ATENANGO', '082': 'SAN AGUSTÍN CHAYUCO', '083': 'SAN AGUSTÍN DE LAS JUNTAS', '084': 'SAN AGUSTÍN ETLA', '085': 'SAN AGUSTÍN LOXICHA', '086': 'SAN AGUSTÍN TLACOTEPEC', '087': 'SAN AGUSTÍN YATARENI', '088': 'SAN ANDRÉS CABECERA NUEVA', '089': 'SAN ANDRÉS DINICUITI', '090': 'SAN ANDRÉS HUAXPALTEPEC', '091': 'SAN ANDRÉS HUAYÁPAM', '092': 'SAN ANDRÉS IXTLAHUACA', '093': 'SAN ANDRÉS LAGUNAS', '094': 'SAN ANDRÉS NUXIÑO', '095': 'SAN ANDRÉS PAXTLÁN', '096': 'SAN ANDRÉS SINAXTLA', '097': 'SAN ANDRÉS SOLAGA', '098': 'SAN ANDRÉS TEOTILÁLPAM', '099': 'SAN ANDRÉS TEPETLAPA', '100': 'SAN ANDRÉS YAÁ', '101': 'SAN ANDRÉS ZABACHE', '102': 'SAN ANDRÉS ZAUTLA', '103': 'SAN ANTONINO CASTILLO VELASCO', '104': 'SAN ANTONINO EL ALTO', '105': 'SAN ANTONINO MONTE VERDE', '106': 'SAN ANTONIO ACUTLA', '107': 'SAN ANTONIO DE LA CAL', '108': 'SAN ANTONIO HUITEPEC', '109': 'SAN ANTONIO NANAHUATÍPAM', '110': 'SAN ANTONIO SINICAHUA', '111': 'SAN ANTONIO TEPETLAPA', '112': 'SAN BALTAZAR CHICHICÁPAM', '113': 'SAN BALTAZAR LOXICHA', '114': 'SAN BALTAZAR YATZACHI EL BAJO', '115': 'SAN BARTOLO COYOTEPEC', '116': 'SAN BARTOLOMÉ AYAUTLA', '117': 'SAN BARTOLOMÉ LOXICHA', '118': 'SAN BARTOLOMÉ QUIALANA', '119': 'SAN BARTOLOMÉ YUCUAÑE', '120': 'SAN BARTOLOMÉ ZOOGOCHO', '121': 'SAN BARTOLO SOYALTEPEC', '122': 'SAN BARTOLO YAUTEPEC', '123': 'SAN BERNARDO MIXTEPEC', '124': 'SAN BLAS ATEMPA', '125': 'SAN CARLOS YAUTEPEC', '126': 'SAN CRISTÓBAL AMATLÁN', '127': 'SAN CRISTÓBAL AMOLTEPEC', '128': 'SAN CRISTÓBAL LACHIRIOAG', '129': 'SAN CRISTÓBAL SUCHIXTLAHUACA', '130': 'SAN DIONISIO DEL MAR', '131': 'SAN DIONISIO OCOTEPEC', '132': 'SAN DIONISIO OCOTLÁN', '133': 'SAN ESTEBAN ATATLAHUCA', '134': 'SAN FELIPE JALAPA DE DÍAZ', '135': 'SAN FELIPE TEJALÁPAM', '136': 'SAN FELIPE USILA', '137': 'SAN FRANCISCO CAHUACUÁ', '138': 'SAN FRANCISCO CAJONOS', '139': 'SAN FRANCISCO CHAPULAPA', '140': 'SAN FRANCISCO CHINDÚA', '141': 'SAN FRANCISCO DEL MAR', '142': 'SAN FRANCISCO HUEHUETLÁN', '143': 'SAN FRANCISCO IXHUATÁN', '144': 'SAN FRANCISCO JALTEPETONGO', '145': 'SAN FRANCISCO LACHIGOLÓ', '146': 'SAN FRANCISCO LOGUECHE', '147': 'SAN FRANCISCO NUXAÑO', '148': 'SAN FRANCISCO OZOLOTEPEC', '149': 'SAN FRANCISCO SOLA', '150': 'SAN FRANCISCO TELIXTLAHUACA', '151': 'SAN FRANCISCO TEOPAN', '152': 'SAN FRANCISCO TLAPANCINGO', '153': 'SAN GABRIEL MIXTEPEC', '154': 'SAN ILDEFONSO AMATLÁN', '155': 'SAN ILDEFONSO SOLA', '156': 'SAN ILDEFONSO VILLA ALTA', '157': 'SAN JACINTO AMILPAS', '158': 'SAN JACINTO TLACOTEPEC', '159': 'SAN JERÓNIMO COATLÁN', '160': 'SAN JERÓNIMO SILACAYOAPILLA', '161': 'SAN JERÓNIMO SOSOLA', '162': 'SAN JERÓNIMO TAVICHE', '163': 'SAN JERÓNIMO TECÓATL', '164': 'SAN JORGE NUCHITA', '165': 'SAN JOSÉ AYUQUILA', '166': 'SAN JOSÉ CHILTEPEC', '167': 'SAN JOSÉ DEL PEÑASCO', '168': 'SAN JOSÉ ESTANCIA GRANDE', '169': 'SAN JOSÉ INDEPENDENCIA', '170': 'SAN JOSÉ LACHIGUIRI', '171': 'SAN JOSÉ TENANGO', '172': 'SAN JUAN ACHIUTLA', '173': 'SAN JUAN ATEPEC', '174': 'ÁNIMAS TRUJANO', '175': 'SAN JUAN BAUTISTA ATATLAHUCA', '176': 'SAN JUAN BAUTISTA COIXTLAHUACA', '177': 'SAN JUAN BAUTISTA CUICATLÁN', '178': 'SAN JUAN BAUTISTA GUELACHE', '179': 'SAN JUAN BAUTISTA JAYACATLÁN', '180': 'SAN JUAN BAUTISTA LO DE SOTO', '181': 'SAN JUAN BAUTISTA SUCHITEPEC', '182': 'SAN JUAN BAUTISTA TLACOATZINTEPEC', '183': 'SAN JUAN BAUTISTA TLACHICHILCO', '184': 'SAN JUAN BAUTISTA TUXTEPEC', '185': 'SAN JUAN CACAHUATEPEC', '186': 'SAN JUAN CIENEGUILLA', '187': 'SAN JUAN COATZÓSPAM', '188': 'SAN JUAN COLORADO', '189': 'SAN JUAN COMALTEPEC', '190': 'SAN JUAN COTZOCÓN', '191': 'SAN JUAN CHICOMEZÚCHIL', '192': 'SAN JUAN CHILATECA', '193': 'SAN JUAN DEL ESTADO', '194': 'SAN JUAN DEL RÍO', '195': 'SAN JUAN DIUXI', '196': 'SAN JUAN EVANGELISTA ANALCO', '197': 'SAN JUAN GUELAVÍA', '198': 'SAN JUAN GUICHICOVI', '199': 'SAN JUAN IHUALTEPEC', '200': 'SAN JUAN JUQUILA MIXES', '201': 'SAN JUAN JUQUILA VIJANOS', '202': 'SAN JUAN LACHAO', '203': 'SAN JUAN LACHIGALLA', '204': 'SAN JUAN LAJARCIA', '205': 'SAN JUAN LALANA', '206': 'SAN JUAN DE LOS CUÉS', '207': 'SAN JUAN MAZATLÁN', '208': 'SAN JUAN MIXTEPEC', '209': 'SAN JUAN MIXTEPEC', '210': 'SAN JUAN ÑUMÍ', '211': 'SAN JUAN OZOLOTEPEC', '212': 'SAN JUAN PETLAPA', '213': 'SAN JUAN QUIAHIJE', '214': 'SAN JUAN QUIOTEPEC', '215': 'SAN JUAN SAYULTEPEC', '216': 'SAN JUAN TABAÁ', '217': 'SAN JUAN TAMAZOLA', '218': 'SAN JUAN TEITA', '219': 'SAN JUAN TEITIPAC', '220': 'SAN JUAN TEPEUXILA', '221': 'SAN JUAN TEPOSCOLULA', '222': 'SAN JUAN YAEÉ', '223': 'SAN JUAN YATZONA', '224': 'SAN JUAN YUCUITA', '225': 'SAN LORENZO', '226': 'SAN LORENZO ALBARRADAS', '227': 'SAN LORENZO CACAOTEPEC', '228': 'SAN LORENZO CUAUNECUILTITLA', '229': 'SAN LORENZO TEXMELÚCAN', '230': 'SAN LORENZO VICTORIA', '231': 'SAN LUCAS CAMOTLÁN', '232': 'SAN LUCAS OJITLÁN', '233': 'SAN LUCAS QUIAVINÍ', '234': 'SAN LUCAS ZOQUIÁPAM', '235': 'SAN LUIS AMATLÁN', '236': 'SAN MARCIAL OZOLOTEPEC', '237': 'SAN MARCOS ARTEAGA', '238': 'SAN MARTÍN DE LOS CANSECOS', '239': 'SAN MARTÍN HUAMELÚLPAM', '240': 'SAN MARTÍN ITUNYOSO', '241': 'SAN MARTÍN LACHILÁ', '242': 'SAN MARTÍN PERAS', '243': 'SAN MARTÍN TILCAJETE', '244': 'SAN MARTÍN TOXPALAN', '245': 'SAN MARTÍN ZACATEPEC', '246': 'SAN MATEO CAJONOS', '247': 'CAPULÁLPAM DE MÉNDEZ', '248': 'SAN MATEO DEL MAR', '249': 'SAN MATEO YOLOXOCHITLÁN', '250': 'SAN MATEO ETLATONGO', '251': 'SAN MATEO NEJÁPAM', '252': 'SAN MATEO PEÑASCO', '253': 'SAN MATEO PIÑAS', '254': 'SAN MATEO RÍO HONDO', '255': 'SAN MATEO SINDIHUI', '256': 'SAN MATEO TLAPILTEPEC', '257': 'SAN MELCHOR BETAZA', '258': 'SAN MIGUEL ACHIUTLA', '259': 'SAN MIGUEL AHUEHUETITLÁN', '260': 'SAN MIGUEL ALOÁPAM', '261': 'SAN MIGUEL AMATITLÁN', '262': 'SAN MIGUEL AMATLÁN', '263': 'SAN MIGUEL COATLÁN', '264': 'SAN MIGUEL CHICAHUA', '265': 'SAN MIGUEL CHIMALAPA', '266': 'SAN MIGUEL DEL PUERTO', '267': 'SAN MIGUEL DEL RÍO', '268': 'SAN MIGUEL EJUTLA', '269': 'SAN MIGUEL EL GRANDE', '270': 'SAN MIGUEL HUAUTLA', '271': 'SAN MIGUEL MIXTEPEC', '272': 'SAN MIGUEL PANIXTLAHUACA', '273': 'SAN MIGUEL PERAS', '274': 'SAN MIGUEL PIEDRAS', '275': 'SAN MIGUEL QUETZALTEPEC', '276': 'SAN MIGUEL SANTA FLOR', '277': 'VILLA SOLA DE VEGA', '278': 'SAN MIGUEL SOYALTEPEC', '279': 'SAN MIGUEL SUCHIXTEPEC', '280': 'VILLA TALEA DE CASTRO', '281': 'SAN MIGUEL TECOMATLÁN', '282': 'SAN MIGUEL TENANGO', '283': 'SAN MIGUEL TEQUIXTEPEC', '284': 'SAN MIGUEL TILQUIÁPAM', '285': 'SAN MIGUEL TLACAMAMA', '286': 'SAN MIGUEL TLACOTEPEC', '287': 'SAN MIGUEL TULANCINGO', '288': 'SAN MIGUEL YOTAO', '289': 'SAN NICOLÁS', '290': 'SAN NICOLÁS HIDALGO', '291': 'SAN PABLO COATLÁN', '292': 'SAN PABLO CUATRO VENADOS', '293': 'SAN PABLO ETLA', '294': 'SAN PABLO HUITZO', '295': 'SAN PABLO HUIXTEPEC', '296': 'SAN PABLO MACUILTIANGUIS', '297': 'SAN PABLO TIJALTEPEC', '298': 'SAN PABLO VILLA DE MITLA', '299': 'SAN PABLO YAGANIZA', '300': 'SAN PEDRO AMUZGOS', '301': 'SAN PEDRO APÓSTOL', '302': 'SAN PEDRO ATOYAC', '303': 'SAN PEDRO CAJONOS', '304': 'SAN PEDRO COXCALTEPEC CÁNTAROS', '305': 'SAN PEDRO COMITANCILLO', '306': 'SAN PEDRO EL ALTO', '307': 'SAN PEDRO HUAMELULA', '308': 'SAN PEDRO HUILOTEPEC', '309': 'SAN PEDRO IXCATLÁN', '310': 'SAN PEDRO IXTLAHUACA', '311': 'SAN PEDRO JALTEPETONGO', '312': 'SAN PEDRO JICAYÁN', '313': 'SAN PEDRO JOCOTIPAC', '314': 'SAN PEDRO JUCHATENGO', '315': 'SAN PEDRO MÁRTIR', '316': 'SAN PEDRO MÁRTIR QUIECHAPA', '317': 'SAN PEDRO MÁRTIR YUCUXACO', '318': 'SAN PEDRO MIXTEPEC', '319': 'SAN PEDRO MIXTEPEC', '320': 'SAN PEDRO MOLINOS', '321': 'SAN PEDRO NOPALA', '322': 'SAN PEDRO OCOPETATILLO', '323': 'SAN PEDRO OCOTEPEC', '324': 'SAN PEDRO POCHUTLA', '325': 'SAN PEDRO QUIATONI', '326': 'SAN PEDRO SOCHIÁPAM', '327': 'SAN PEDRO TAPANATEPEC', '328': 'SAN PEDRO TAVICHE', '329': 'SAN PEDRO TEOZACOALCO', '330': 'SAN PEDRO TEUTILA', '331': 'SAN PEDRO TIDAÁ', '332': 'SAN PEDRO TOPILTEPEC', '333': 'SAN PEDRO TOTOLÁPAM', '334': 'VILLA DE TUTUTEPEC', '335': 'SAN PEDRO YANERI', '336': 'SAN PEDRO YÓLOX', '337': 'SAN PEDRO Y SAN PABLO AYUTLA', '338': 'VILLA DE ETLA', '339': 'SAN PEDRO Y SAN PABLO TEPOSCOLULA', '340': 'SAN PEDRO Y SAN PABLO TEQUIXTEPEC', '341': 'SAN PEDRO YUCUNAMA', '342': 'SAN RAYMUNDO JALPAN', '343': 'SAN SEBASTIÁN ABASOLO', '344': 'SAN SEBASTIÁN COATLÁN', '345': 'SAN SEBASTIÁN IXCAPA', '346': 'SAN SEBASTIÁN NICANANDUTA', '347': 'SAN SEBASTIÁN RÍO HONDO', '348': 'SAN SEBASTIÁN TECOMAXTLAHUACA', '349': 'SAN SEBASTIÁN TEITIPAC', '350': 'SAN SEBASTIÁN TUTLA', '351': 'SAN SIMÓN ALMOLONGAS', '352': 'SAN SIMÓN ZAHUATLÁN', '353': 'SANTA ANA', '354': 'SANTA ANA ATEIXTLAHUACA', '355': 'SANTA ANA CUAUHTÉMOC', '356': 'SANTA ANA DEL VALLE', '357': 'SANTA ANA TAVELA', '358': 'SANTA ANA TLAPACOYAN', '359': 'SANTA ANA YARENI', '360': 'SANTA ANA ZEGACHE', '361': 'SANTA CATALINA QUIERÍ', '362': 'SANTA CATARINA CUIXTLA', '363': 'SANTA CATARINA IXTEPEJI', '364': 'SANTA CATARINA JUQUILA', '365': 'SANTA CATARINA LACHATAO', '366': 'SANTA CATARINA LOXICHA', '367': 'SANTA CATARINA MECHOACÁN', '368': 'SANTA CATARINA MINAS', '369': 'SANTA CATARINA QUIANÉ', '370': 'SANTA CATARINA TAYATA', '371': 'SANTA CATARINA TICUÁ', '372': 'SANTA CATARINA YOSONOTÚ', '373': 'SANTA CATARINA ZAPOQUILA', '374': 'SANTA CRUZ ACATEPEC', '375': 'SANTA CRUZ AMILPAS', '376': 'SANTA CRUZ DE BRAVO', '377': 'SANTA CRUZ ITUNDUJIA', '378': 'SANTA CRUZ MIXTEPEC', '379': 'SANTA CRUZ NUNDACO', '380': 'SANTA CRUZ PAPALUTLA', '381': 'SANTA CRUZ TACACHE DE MINA', '382': 'SANTA CRUZ TACAHUA', '383': 'SANTA CRUZ TAYATA', '384': 'SANTA CRUZ XITLA', '385': 'SANTA CRUZ XOXOCOTLÁN', '386': 'SANTA CRUZ ZENZONTEPEC', '387': 'SANTA GERTRUDIS', '388': 'SANTA INÉS DEL MONTE', '389': 'SANTA INÉS YATZECHE', '390': 'SANTA LUCÍA DEL CAMINO', '391': 'SANTA LUCÍA MIAHUATLÁN', '392': 'SANTA LUCÍA MONTEVERDE', '393': 'SANTA LUCÍA OCOTLÁN', '394': 'SANTA MARÍA ALOTEPEC', '395': 'SANTA MARÍA APAZCO', '396': 'SANTA MARÍA LA ASUNCIÓN', '397': 'HEROICA CIUDAD DE TLAXIACO', '398': 'AYOQUEZCO DE ALDAMA', '399': 'SANTA MARÍA ATZOMPA', '400': 'SANTA MARÍA CAMOTLÁN', '401': 'SANTA MARÍA COLOTEPEC', '402': 'SANTA MARÍA CORTIJO', '403': 'SANTA MARÍA COYOTEPEC', '404': 'SANTA MARÍA CHACHOÁPAM', '405': 'VILLA DE CHILAPA DE DÍAZ', '406': 'SANTA MARÍA CHILCHOTLA', '407': 'SANTA MARÍA CHIMALAPA', '408': 'SANTA MARÍA DEL ROSARIO', '409': 'SANTA MARÍA DEL TULE', '410': 'SANTA MARÍA ECATEPEC', '411': 'SANTA MARÍA GUELACÉ', '412': 'SANTA MARÍA GUIENAGATI', '413': 'SANTA MARÍA HUATULCO', '414': 'SANTA MARÍA HUAZOLOTITLÁN', '415': 'SANTA MARÍA IPALAPA', '416': 'SANTA MARÍA IXCATLÁN', '417': 'SANTA MARÍA JACATEPEC', '418': 'SANTA MARÍA JALAPA DEL MARQUÉS', '419': 'SANTA MARÍA JALTIANGUIS', '420': 'SANTA MARÍA LACHIXÍO', '421': 'SANTA MARÍA MIXTEQUILLA', '422': 'SANTA MARÍA NATIVITAS', '423': 'SANTA MARÍA NDUAYACO', '424': 'SANTA MARÍA OZOLOTEPEC', '425': 'SANTA MARÍA PÁPALO', '426': 'SANTA MARÍA PEÑOLES', '427': 'SANTA MARÍA PETAPA', '428': 'SANTA MARÍA QUIEGOLANI', '429': 'SANTA MARÍA SOLA', '430': 'SANTA MARÍA TATALTEPEC', '431': 'SANTA MARÍA TECOMAVACA', '432': 'SANTA MARÍA TEMAXCALAPA', '433': 'SANTA MARÍA TEMAXCALTEPEC', '434': 'SANTA MARÍA TEOPOXCO', '435': 'SANTA MARÍA TEPANTLALI', '436': 'SANTA MARÍA TEXCATITLÁN', '437': 'SANTA MARÍA TLAHUITOLTEPEC', '438': 'SANTA MARÍA TLALIXTAC', '439': 'SANTA MARÍA TONAMECA', '440': 'SANTA MARÍA TOTOLAPILLA', '441': 'SANTA MARÍA XADANI', '442': 'SANTA MARÍA YALINA', '443': 'SANTA MARÍA YAVESÍA', '444': 'SANTA MARÍA YOLOTEPEC', '445': 'SANTA MARÍA YOSOYÚA', '446': 'SANTA MARÍA YUCUHITI', '447': 'SANTA MARÍA ZACATEPEC', '448': 'SANTA MARÍA ZANIZA', '449': 'SANTA MARÍA ZOQUITLÁN', '450': 'SANTIAGO AMOLTEPEC', '451': 'SANTIAGO APOALA', '452': 'SANTIAGO APÓSTOL', '453': 'SANTIAGO ASTATA', '454': 'SANTIAGO ATITLÁN', '455': 'SANTIAGO AYUQUILILLA', '456': 'SANTIAGO CACALOXTEPEC', '457': 'SANTIAGO CAMOTLÁN', '458': 'SANTIAGO COMALTEPEC', '459': 'VILLA DE SANTIAGO CHAZUMBA', '460': 'SANTIAGO CHOÁPAM', '461': 'SANTIAGO DEL RÍO', '462': 'SANTIAGO HUAJOLOTITLÁN', '463': 'SANTIAGO HUAUCLILLA', '464': 'SANTIAGO IHUITLÁN PLUMAS', '465': 'SANTIAGO IXCUINTEPEC', '466': 'SANTIAGO IXTAYUTLA', '467': 'SANTIAGO JAMILTEPEC', '468': 'SANTIAGO JOCOTEPEC', '469': 'SANTIAGO JUXTLAHUACA', '470': 'SANTIAGO LACHIGUIRI', '471': 'SANTIAGO LALOPA', '472': 'SANTIAGO LAOLLAGA', '473': 'SANTIAGO LAXOPA', '474': 'SANTIAGO LLANO GRANDE', '475': 'SANTIAGO MATATLÁN', '476': 'SANTIAGO MILTEPEC', '477': 'SANTIAGO MINAS', '478': 'SANTIAGO NACALTEPEC', '479': 'SANTIAGO NEJAPILLA', '480': 'SANTIAGO NUNDICHE', '481': 'SANTIAGO NUYOÓ', '482': 'SANTIAGO PINOTEPA NACIONAL', '483': 'SANTIAGO SUCHILQUITONGO', '484': 'SANTIAGO TAMAZOLA', '485': 'SANTIAGO TAPEXTLA', '486': 'VILLA TEJÚPAM DE LA UNIÓN', '487': 'SANTIAGO TENANGO', '488': 'SANTIAGO TEPETLAPA', '489': 'SANTIAGO TETEPEC', '490': 'SANTIAGO TEXCALCINGO', '491': 'SANTIAGO TEXTITLÁN', '492': 'SANTIAGO TILANTONGO', '493': 'SANTIAGO TILLO', '494': 'SANTIAGO TLAZOYALTEPEC', '495': 'SANTIAGO XANICA', '496': 'SANTIAGO XIACUÍ', '497': 'SANTIAGO YAITEPEC', '498': 'SANTIAGO YAVEO', '499': 'SANTIAGO YOLOMÉCATL', '500': 'SANTIAGO YOSONDÚA', '501': 'SANTIAGO YUCUYACHI', '502': 'SANTIAGO ZACATEPEC', '503': 'SANTIAGO ZOOCHILA', '504': 'NUEVO ZOQUIÁPAM', '505': 'SANTO DOMINGO INGENIO', '506': 'SANTO DOMINGO ALBARRADAS', '507': 'SANTO DOMINGO ARMENTA', '508': 'SANTO DOMINGO CHIHUITÁN', '509': 'SANTO DOMINGO DE MORELOS', '510': 'SANTO DOMINGO IXCATLÁN', '511': 'SANTO DOMINGO NUXAÁ', '512': 'SANTO DOMINGO OZOLOTEPEC', '513': 'SANTO DOMINGO PETAPA', '514': 'SANTO DOMINGO ROAYAGA', '515': 'SANTO DOMINGO TEHUANTEPEC', '516': 'SANTO DOMINGO TEOJOMULCO', '517': 'SANTO DOMINGO TEPUXTEPEC', '518': 'SANTO DOMINGO TLATAYÁPAM', '519': 'SANTO DOMINGO TOMALTEPEC', '520': 'SANTO DOMINGO TONALÁ', '521': 'SANTO DOMINGO TONALTEPEC', '522': 'SANTO DOMINGO XAGACÍA', '523': 'SANTO DOMINGO YANHUITLÁN', '524': 'SANTO DOMINGO YODOHINO', '525': 'SANTO DOMINGO ZANATEPEC', '526': 'SANTOS REYES NOPALA', '527': 'SANTOS REYES PÁPALO', '528': 'SANTOS REYES TEPEJILLO', '529': 'SANTOS REYES YUCUNÁ', '530': 'SANTO TOMÁS JALIEZA', '531': 'SANTO TOMÁS MAZALTEPEC', '532': 'SANTO TOMÁS OCOTEPEC', '533': 'SANTO TOMÁS TAMAZULAPAN', '534': 'SAN VICENTE COATLÁN', '535': 'SAN VICENTE LACHIXÍO', '536': 'SAN VICENTE NUÑÚ', '537': 'SILACAYOÁPAM', '538': 'SITIO DE XITLAPEHUA', '539': 'SOLEDAD ETLA', '540': 'VILLA DE TAMAZULÁPAM DEL PROGRESO', '541': 'TANETZE DE ZARAGOZA', '542': 'TANICHE', '543': 'TATALTEPEC DE VALDÉS', '544': 'TEOCOCUILCO DE MARCOS PÉREZ', '545': 'TEOTITLÁN DE FLORES MAGÓN', '546': 'TEOTITLÁN DEL VALLE', '547': 'TEOTONGO', '548': 'TEPELMEME VILLA DE MORELOS', 549	HEROICA VILLA TEZOATLÁN DE SEGURA Y LUNA, CUNA DE LA INDEPENDENCIA 'DE': 'OAXACA', '550': 'SAN JERÓNIMO TLACOCHAHUAYA', '551': 'TLACOLULA DE MATAMOROS', '552': 'TLACOTEPEC PLUMAS', '553': 'TLALIXTAC DE CABRERA', '554': 'TOTONTEPEC VILLA DE MORELOS', '555': 'TRINIDAD ZAACHILA', '556': 'LA TRINIDAD VISTA HERMOSA', '557': 'UNIÓN HIDALGO', '558': 'VALERIO TRUJANO', '559': 'SAN JUAN BAUTISTA VALLE NACIONAL', '560': 'VILLA DÍAZ ORDAZ', '561': 'YAXE', '562': 'MAGDALENA YODOCONO DE PORFIRIO DÍAZ', '563': 'YOGANA', '564': 'YUTANDUCHI DE GUERRERO', '565': 'VILLA DE ZAACHILA', '566': 'SAN MATEO YUCUTINDOO', '567': 'ZAPOTITLÁN LAGUNAS', '568': 'ZAPOTITLÁN PALMAS', '569': 'SANTA INÉS DE ZARAGOZA', '570': 'ZIMATLÁN DE ÁLVAREZ', '999': 'NO ESPECIFICADO', '001': 'ACAJETE', '002': 'ACATENO', '003': 'ACATLÁN', '004': 'ACATZINGO', '005': 'ACTEOPAN', '006': 'AHUACATLÁN', '007': 'AHUATLÁN', '008': 'AHUAZOTEPEC', '009': 'AHUEHUETITLA', '010': 'AJALPAN', '011': 'ALBINO ZERTUCHE', '012': 'ALJOJUCA', '013': 'ALTEPEXI', '014': 'AMIXTLÁN', '015': 'AMOZOC', '016': 'AQUIXTLA', '017': 'ATEMPAN', '018': 'ATEXCAL', '019': 'ATLIXCO', '020': 'ATOYATEMPAN', '021': 'ATZALA', '022': 'ATZITZIHUACÁN', '023': 'ATZITZINTLA', '024': 'AXUTLA', '025': 'AYOTOXCO DE GUERRERO', '026': 'CALPAN', '027': 'CALTEPEC', '028': 'CAMOCUAUTLA', '029': 'CAXHUACAN', '030': 'COATEPEC', '031': 'COATZINGO', '032': 'COHETZALA', '033': 'COHUECAN', '034': 'CORONANGO', '035': 'COXCATLÁN', '036': 'COYOMEAPAN', '037': 'COYOTEPEC', '038': 'CUAPIAXTLA DE MADERO', '039': 'CUAUTEMPAN', '040': 'CUAUTINCHÁN', '041': 'CUAUTLANCINGO', '042': 'CUAYUCA DE ANDRADE', '043': 'CUETZALAN DEL PROGRESO', '044': 'CUYOACO', '045': 'CHALCHICOMULA DE SESMA', '046': 'CHAPULCO', '047': 'CHIAUTLA', '048': 'CHIAUTZINGO', '049': 'CHICONCUAUTLA', '050': 'CHICHIQUILA', '051': 'CHIETLA', '052': 'CHIGMECATITLÁN', '053': 'CHIGNAHUAPAN', '054': 'CHIGNAUTLA', '055': 'CHILA', '056': 'CHILA DE LA SAL', '057': 'HONEY', '058': 'CHILCHOTLA', '059': 'CHINANTLA', '060': 'DOMINGO ARENAS', '061': 'ELOXOCHITLÁN', '062': 'EPATLÁN', '063': 'ESPERANZA', '064': 'FRANCISCO Z. MENA', '065': 'GENERAL FELIPE ÁNGELES', '066': 'GUADALUPE', '067': 'GUADALUPE VICTORIA', '068': 'HERMENEGILDO GALEANA', '069': 'HUAQUECHULA', '070': 'HUATLATLAUCA', '071': 'HUAUCHINANGO', '072': 'HUEHUETLA', '073': 'HUEHUETLÁN EL CHICO', '074': 'HUEJOTZINGO', '075': 'HUEYAPAN', '076': 'HUEYTAMALCO', '077': 'HUEYTLALPAN', '078': 'HUITZILAN DE SERDÁN', '079': 'HUITZILTEPEC', '080': 'ATLEQUIZAYAN', '081': 'IXCAMILPA DE GUERRERO', '082': 'IXCAQUIXTLA', '083': 'IXTACAMAXTITLÁN', '084': 'IXTEPEC', '085': 'IZÚCAR DE MATAMOROS', '086': 'JALPAN', '087': 'JOLALPAN', '088': 'JONOTLA', '089': 'JOPALA', '090': 'JUAN C. BONILLA', '091': 'JUAN GALINDO', '092': 'JUAN N. MÉNDEZ', '093': 'LAFRAGUA', '094': 'LIBRES', '095': 'LA MAGDALENA TLATLAUQUITEPEC', '096': 'MAZAPILTEPEC DE JUÁREZ', '097': 'MIXTLA', '098': 'MOLCAXAC', '099': 'CAÑADA MORELOS', '100': 'NAUPAN', '101': 'NAUZONTLA', '102': 'NEALTICAN', '103': 'NICOLÁS BRAVO', '104': 'NOPALUCAN', '105': 'OCOTEPEC', '106': 'OCOYUCAN', '107': 'OLINTLA', '108': 'ORIENTAL', '109': 'PAHUATLÁN', '110': 'PALMAR DE BRAVO', '111': 'PANTEPEC', '112': 'PETLALCINGO', '113': 'PIAXTLA', '114': 'PUEBLA', '115': 'QUECHOLAC', '116': 'QUIMIXTLÁN', '117': 'RAFAEL LARA GRAJALES', '118': 'LOS REYES DE JUÁREZ', '119': 'SAN ANDRÉS CHOLULA', '120': 'SAN ANTONIO CAÑADA', '121': 'SAN DIEGO LA MESA TOCHIMILTZINGO', '122': 'SAN FELIPE TEOTLALCINGO', '123': 'SAN FELIPE TEPATLÁN', '124': 'SAN GABRIEL CHILAC', '125': 'SAN GREGORIO ATZOMPA', '126': 'SAN JERÓNIMO TECUANIPAN', '127': 'SAN JERÓNIMO XAYACATLÁN', '128': 'SAN JOSÉ CHIAPA', '129': 'SAN JOSÉ MIAHUATLÁN', '130': 'SAN JUAN ATENCO', '131': 'SAN JUAN ATZOMPA', '132': 'SAN MARTÍN TEXMELUCAN', '133': 'SAN MARTÍN TOTOLTEPEC', '134': 'SAN MATÍAS TLALANCALECA', '135': 'SAN MIGUEL IXITLÁN', '136': 'SAN MIGUEL XOXTLA', '137': 'SAN NICOLÁS BUENOS AIRES', '138': 'SAN NICOLÁS DE LOS RANCHOS', '139': 'SAN PABLO ANICANO', '140': 'SAN PEDRO CHOLULA', '141': 'SAN PEDRO YELOIXTLAHUACA', '142': 'SAN SALVADOR EL SECO', '143': 'SAN SALVADOR EL VERDE', '144': 'SAN SALVADOR HUIXCOLOTLA', '145': 'SAN SEBASTIÁN TLACOTEPEC', '146': 'SANTA CATARINA TLALTEMPAN', '147': 'SANTA INÉS AHUATEMPAN', '148': 'SANTA ISABEL CHOLULA', '149': 'SANTIAGO MIAHUATLÁN', '150': 'HUEHUETLÁN EL GRANDE', '151': 'SANTO TOMÁS HUEYOTLIPAN', '152': 'SOLTEPEC', '153': 'TECALI DE HERRERA', '154': 'TECAMACHALCO', '155': 'TECOMATLÁN', '156': 'TEHUACÁN', '157': 'TEHUITZINGO', '158': 'TENAMPULCO', '159': 'TEOPANTLÁN', '160': 'TEOTLALCO', '161': 'TEPANCO DE LÓPEZ', '162': 'TEPANGO DE RODRÍGUEZ', '163': 'TEPATLAXCO DE HIDALGO', '164': 'TEPEACA', '165': 'TEPEMAXALCO', '166': 'TEPEOJUMA', '167': 'TEPETZINTLA', '168': 'TEPEXCO', '169': 'TEPEXI DE RODRÍGUEZ', '170': 'TEPEYAHUALCO', '171': 'TEPEYAHUALCO DE CUAUHTÉMOC', '172': 'TETELA DE OCAMPO', '173': 'TETELES DE AVILA CASTILLO', '174': 'TEZIUTLÁN', '175': 'TIANGUISMANALCO', '176': 'TILAPA', '177': 'TLACOTEPEC DE BENITO JUÁREZ', '178': 'TLACUILOTEPEC', '179': 'TLACHICHUCA', '180': 'TLAHUAPAN', '181': 'TLALTENANGO', '182': 'TLANEPANTLA', '183': 'TLAOLA', '184': 'TLAPACOYA', '185': 'TLAPANALÁ', '186': 'TLATLAUQUITEPEC', '187': 'TLAXCO', '188': 'TOCHIMILCO', '189': 'TOCHTEPEC', '190': 'TOTOLTEPEC DE GUERRERO', '191': 'TULCINGO', '192': 'TUZAMAPAN DE GALEANA', '193': 'TZICATLACOYAN', '194': 'VENUSTIANO CARRANZA', '195': 'VICENTE GUERRERO', '196': 'XAYACATLÁN DE BRAVO', '197': 'XICOTEPEC', '198': 'XICOTLÁN', '199': 'XIUTETELCO', '200': 'XOCHIAPULCO', '201': 'XOCHILTEPEC', '202': 'XOCHITLÁN DE VICENTE SUÁREZ', '203': 'XOCHITLÁN TODOS SANTOS', '204': 'YAONÁHUAC', '205': 'YEHUALTEPEC', '206': 'ZACAPALA', '207': 'ZACAPOAXTLA', '208': 'ZACATLÁN', '209': 'ZAPOTITLÁN', '210': 'ZAPOTITLÁN DE MÉNDEZ', '211': 'ZARAGOZA', '212': 'ZAUTLA', '213': 'ZIHUATEUTLA', '214': 'ZINACATEPEC', '215': 'ZONGOZOTLA', '216': 'ZOQUIAPAN', '217': 'ZOQUITLÁN', '999': 'NO ESPECIFICADO', '001': 'AMEALCO DE BONFIL', '002': 'PINAL DE AMOLES', '003': 'ARROYO SECO', '004': 'CADEREYTA DE MONTES', '005': 'COLÓN', '006': 'CORREGIDORA', '007': 'EZEQUIEL MONTES', '008': 'HUIMILPAN', '009': 'JALPAN DE SERRA', '010': 'LANDA DE MATAMOROS', '011': 'EL MARQUÉS', '012': 'PEDRO ESCOBEDO', '013': 'PEÑAMILLER', '014': 'QUERÉTARO', '015': 'SAN JOAQUÍN', '016': 'SAN JUAN DEL RÍO', '017': 'TEQUISQUIAPAN', '018': 'TOLIMÁN', '999': 'NO ESPECIFICADO', '001': 'COZUMEL', '002': 'FELIPE CARRILLO PUERTO', '003': 'ISLA MUJERES', '004': 'OTHÓN P. BLANCO', '005': 'BENITO JUÁREZ', '006': 'JOSÉ MARÍA MORELOS', '007': 'LÁZARO CÁRDENAS', '008': 'SOLIDARIDAD', '009': 'TULUM', '010': 'BACALAR', '011': 'PUERTO MORELOS', '999': 'NO ESPECIFICADO', '001': 'AHUALULCO', '002': 'ALAQUINES', '003': 'AQUISMÓN', '004': 'ARMADILLO DE LOS INFANTE', '005': 'CÁRDENAS', '006': 'CATORCE', '007': 'CEDRAL', '008': 'CERRITOS', '009': 'CERRO DE SAN PEDRO', '010': 'CIUDAD DEL MAÍZ', '011': 'CIUDAD FERNÁNDEZ', '012': 'TANCANHUITZ', '013': 'CIUDAD VALLES', '014': 'COXCATLÁN', '015': 'CHARCAS', '016': 'EBANO', '017': 'GUADALCÁZAR', '018': 'HUEHUETLÁN', '019': 'LAGUNILLAS', '020': 'MATEHUALA', '021': 'MEXQUITIC DE CARMONA', '022': 'MOCTEZUMA', '023': 'RAYÓN', '024': 'RIOVERDE', '025': 'SALINAS', '026': 'SAN ANTONIO', '027': 'SAN CIRO DE ACOSTA', '028': 'SAN LUIS POTOSÍ', '029': 'SAN MARTÍN CHALCHICUAUTLA', '030': 'SAN NICOLÁS TOLENTINO', '031': 'SANTA CATARINA', '032': 'SANTA MARÍA DEL RÍO', '033': 'SANTO DOMINGO', '034': 'SAN VICENTE TANCUAYALAB', '035': 'SOLEDAD DE GRACIANO SÁNCHEZ', '036': 'TAMASOPO', '037': 'TAMAZUNCHALE', '038': 'TAMPACÁN', '039': 'TAMPAMOLÓN CORONA', '040': 'TAMUÍN', '041': 'TANLAJÁS', '042': 'TANQUIÁN DE ESCOBEDO', '043': 'TIERRA NUEVA', '044': 'VANEGAS', '045': 'VENADO', '046': 'VILLA DE ARRIAGA', '047': 'VILLA DE GUADALUPE', '048': 'VILLA DE LA PAZ', '049': 'VILLA DE RAMOS', '050': 'VILLA DE REYES', '051': 'VILLA HIDALGO', '052': 'VILLA JUÁREZ', '053': 'AXTLA DE TERRAZAS', '054': 'XILITLA', '055': 'ZARAGOZA', '056': 'VILLA DE ARISTA', '057': 'MATLAPA', '058': 'EL NARANJO', '999': 'NO ESPECIFICADO', '001': 'AHOME', '002': 'ANGOSTURA', '003': 'BADIRAGUATO', '004': 'CONCORDIA', '005': 'COSALÁ', '006': 'CULIACÁN', '007': 'CHOIX', '008': 'ELOTA', '009': 'ESCUINAPA', '010': 'EL FUERTE', '011': 'GUASAVE', '012': 'MAZATLÁN', '013': 'MOCORITO', '014': 'ROSARIO', '015': 'SALVADOR ALVARADO', '016': 'SAN IGNACIO', '017': 'SINALOA', '018': 'NAVOLATO', '999': 'NO ESPECIFICADO', '001': 'ACONCHI', '002': 'AGUA PRIETA', '003': 'ALAMOS', '004': 'ALTAR', '005': 'ARIVECHI', '006': 'ARIZPE', '007': 'ATIL', '008': 'BACADÉHUACHI', '009': 'BACANORA', '010': 'BACERAC', '011': 'BACOACHI', '012': 'BÁCUM', '013': 'BANÁMICHI', '014': 'BAVIÁCORA', '015': 'BAVISPE', '016': 'BENJAMÍN HILL', '017': 'CABORCA', '018': 'CAJEME', '019': 'CANANEA', '020': 'CARBÓ', '021': 'LA COLORADA', '022': 'CUCURPE', '023': 'CUMPAS', '024': 'DIVISADEROS', '025': 'EMPALME', '026': 'ETCHOJOA', '027': 'FRONTERAS', '028': 'GRANADOS', '029': 'GUAYMAS', '030': 'HERMOSILLO', '031': 'HUACHINERA', '032': 'HUÁSABAS', '033': 'HUATABAMPO', '034': 'HUÉPAC', '035': 'IMURIS', '036': 'MAGDALENA', '037': 'MAZATÁN', '038': 'MOCTEZUMA', '039': 'NACO', '040': 'NÁCORI CHICO', '041': 'NACOZARI DE GARCÍA', '042': 'NAVOJOA', '043': 'NOGALES', '044': 'ONAVAS', '045': 'OPODEPE', '046': 'OQUITOA', '047': 'PITIQUITO', '048': 'PUERTO PEÑASCO', '049': 'QUIRIEGO', '050': 'RAYÓN', '051': 'ROSARIO', '052': 'SAHUARIPA', '053': 'SAN FELIPE DE JESÚS', '054': 'SAN JAVIER', '055': 'SAN LUIS RÍO COLORADO', '056': 'SAN MIGUEL DE HORCASITAS', '057': 'SAN PEDRO DE LA CUEVA', '058': 'SANTA ANA', '059': 'SANTA CRUZ', '060': 'SÁRIC', '061': 'SOYOPA', '062': 'SUAQUI GRANDE', '063': 'TEPACHE', '064': 'TRINCHERAS', '065': 'TUBUTAMA', '066': 'URES', '067': 'VILLA HIDALGO', '068': 'VILLA PESQUEIRA', '069': 'YÉCORA', '070': 'GENERAL PLUTARCO ELÍAS CALLES', '071': 'BENITO JUÁREZ', '072': 'SAN IGNACIO RÍO MUERTO', '999': 'NO ESPECIFICADO', '001': 'BALANCÁN', '002': 'CÁRDENAS', '003': 'CENTLA', '004': 'CENTRO', '005': 'COMALCALCO', '006': 'CUNDUACÁN', '007': 'EMILIANO ZAPATA', '008': 'HUIMANGUILLO', '009': 'JALAPA', '010': 'JALPA DE MÉNDEZ', '011': 'JONUTA', '012': 'MACUSPANA', '013': 'NACAJUCA', '014': 'PARAÍSO', '015': 'TACOTALPA', '016': 'TEAPA', '017': 'TENOSIQUE', '999': 'NO ESPECIFICADO', '001': 'ABASOLO', '002': 'ALDAMA', '003': 'ALTAMIRA', '004': 'ANTIGUO MORELOS', '005': 'BURGOS', '006': 'BUSTAMANTE', '007': 'CAMARGO', '008': 'CASAS', '009': 'CIUDAD MADERO', '010': 'CRUILLAS', '011': 'GÓMEZ FARÍAS', '012': 'GONZÁLEZ', '013': 'GÜÉMEZ', '014': 'GUERRERO', '015': 'GUSTAVO DÍAZ ORDAZ', '016': 'HIDALGO', '017': 'JAUMAVE', '018': 'JIMÉNEZ', '019': 'LLERA', '020': 'MAINERO', '021': 'EL MANTE', '022': 'MATAMOROS', '023': 'MÉNDEZ', '024': 'MIER', '025': 'MIGUEL ALEMÁN', '026': 'MIQUIHUANA', '027': 'NUEVO LAREDO', '028': 'NUEVO MORELOS', '029': 'OCAMPO', '030': 'PADILLA', '031': 'PALMILLAS', '032': 'REYNOSA', '033': 'RÍO BRAVO', '034': 'SAN CARLOS', '035': 'SAN FERNANDO', '036': 'SAN NICOLÁS', '037': 'SOTO LA MARINA', '038': 'TAMPICO', '039': 'TULA', '040': 'VALLE HERMOSO', '041': 'VICTORIA', '042': 'VILLAGRÁN', '043': 'XICOTÉNCATL', '999': 'NO ESPECIFICADO', '001': 'AMAXAC DE GUERRERO', '002': 'APETATITLÁN DE ANTONIO CARVAJAL', '003': 'ATLANGATEPEC', '004': 'ATLTZAYANCA', '005': 'APIZACO', '006': 'CALPULALPAN', '007': 'EL CARMEN TEQUEXQUITLA', '008': 'CUAPIAXTLA', '009': 'CUAXOMULCO', '010': 'CHIAUTEMPAN', '011': 'MUÑOZ DE DOMINGO ARENAS', '012': 'ESPAÑITA', '013': 'HUAMANTLA', '014': 'HUEYOTLIPAN', '015': 'IXTACUIXTLA DE MARIANO MATAMOROS', '016': 'IXTENCO', '017': 'MAZATECOCHCO DE JOSÉ MARÍA MORELOS', '018': 'CONTLA DE JUAN CUAMATZI', '019': 'TEPETITLA DE LARDIZÁBAL', '020': 'SANCTÓRUM DE LÁZARO CÁRDENAS', '021': 'NANACAMILPA DE MARIANO ARISTA', '022': 'ACUAMANALA DE MIGUEL HIDALGO', '023': 'NATÍVITAS', '024': 'PANOTLA', '025': 'SAN PABLO DEL MONTE', '026': 'SANTA CRUZ TLAXCALA', '027': 'TENANCINGO', '028': 'TEOLOCHOLCO', '029': 'TEPEYANCO', '030': 'TERRENATE', '031': 'TETLA DE LA SOLIDARIDAD', '032': 'TETLATLAHUCA', '033': 'TLAXCALA', '034': 'TLAXCO', '035': 'TOCATLÁN', '036': 'TOTOLAC', '037': 'ZILTLALTÉPEC DE TRINIDAD SÁNCHEZ SANTOS', '038': 'TZOMPANTEPEC', '039': 'XALOZTOC', '040': 'XALTOCAN', '041': 'PAPALOTLA DE XICOHTÉNCATL', '042': 'XICOHTZINCO', '043': 'YAUHQUEMEHCAN', '044': 'ZACATELCO', '045': 'BENITO JUÁREZ', '046': 'EMILIANO ZAPATA', '047': 'LÁZARO CÁRDENAS', '048': 'LA MAGDALENA TLALTELULCO', '049': 'SAN DAMIÁN TEXÓLOC', '050': 'SAN FRANCISCO TETLANOHCAN', '051': 'SAN JERÓNIMO ZACUALPAN', '052': 'SAN JOSÉ TEACALCO', '053': 'SAN JUAN HUACTZINCO', '054': 'SAN LORENZO AXOCOMANITLA', '055': 'SAN LUCAS TECOPILCO', '056': 'SANTA ANA NOPALUCAN', '057': 'SANTA APOLONIA TEACALCO', '058': 'SANTA CATARINA AYOMETLA', '059': 'SANTA CRUZ QUILEHTLA', '060': 'SANTA ISABEL XILOXOXTLA', '999': 'NO ESPECIFICADO', '001': 'ACAJETE', '002': 'ACATLÁN', '003': 'ACAYUCAN', '004': 'ACTOPAN', '005': 'ACULA', '006': 'ACULTZINGO', '007': 'CAMARÓN DE TEJEDA', '008': 'ALPATLÁHUAC', '009': 'ALTO LUCERO DE GUTIÉRREZ BARRIOS', '010': 'ALTOTONGA', '011': 'ALVARADO', '012': 'AMATITLÁN', '013': 'NARANJOS AMATLÁN', '014': 'AMATLÁN DE LOS REYES', '015': 'ANGEL R. CABADA', '016': 'LA ANTIGUA', '017': 'APAZAPAN', '018': 'AQUILA', '019': 'ASTACINGA', '020': 'ATLAHUILCO', '021': 'ATOYAC', '022': 'ATZACAN', '023': 'ATZALAN', '024': 'TLALTETELA', '025': 'AYAHUALULCO', '026': 'BANDERILLA', '027': 'BENITO JUÁREZ', '028': 'BOCA DEL RÍO', '029': 'CALCAHUALCO', '030': 'CAMERINO Z. MENDOZA', '031': 'CARRILLO PUERTO', '032': 'CATEMACO', '033': 'CAZONES DE HERRERA', '034': 'CERRO AZUL', '035': 'CITLALTÉPETL', '036': 'COACOATZINTLA', '037': 'COAHUITLÁN', '038': 'COATEPEC', '039': 'COATZACOALCOS', '040': 'COATZINTLA', '041': 'COETZALA', '042': 'COLIPA', '043': 'COMAPA', '044': 'CÓRDOBA', '045': 'COSAMALOAPAN DE CARPIO', '046': 'COSAUTLÁN DE CARVAJAL', '047': 'COSCOMATEPEC', '048': 'COSOLEACAQUE', '049': 'COTAXTLA', '050': 'COXQUIHUI', '051': 'COYUTLA', '052': 'CUICHAPA', '053': 'CUITLÁHUAC', '054': 'CHACALTIANGUIS', '055': 'CHALMA', '056': 'CHICONAMEL', '057': 'CHICONQUIACO', '058': 'CHICONTEPEC', '059': 'CHINAMECA', '060': 'CHINAMPA DE GOROSTIZA', '061': 'LAS CHOAPAS', '062': 'CHOCAMÁN', '063': 'CHONTLA', '064': 'CHUMATLÁN', '065': 'EMILIANO ZAPATA', '066': 'ESPINAL', '067': 'FILOMENO MATA', '068': 'FORTÍN', '069': 'GUTIÉRREZ ZAMORA', '070': 'HIDALGOTITLÁN', '071': 'HUATUSCO', '072': 'HUAYACOCOTLA', '073': 'HUEYAPAN DE OCAMPO', '074': 'HUILOAPAN DE CUAUHTÉMOC', '075': 'IGNACIO DE LA LLAVE', '076': 'ILAMATLÁN', '077': 'ISLA', '078': 'IXCATEPEC', '079': 'IXHUACÁN DE LOS REYES', '080': 'IXHUATLÁN DEL CAFÉ', '081': 'IXHUATLANCILLO', '082': 'IXHUATLÁN DEL SURESTE', '083': 'IXHUATLÁN DE MADERO', '084': 'IXMATLAHUACAN', '085': 'IXTACZOQUITLÁN', '086': 'JALACINGO', '087': 'XALAPA', '088': 'JALCOMULCO', '089': 'JÁLTIPAN', '090': 'JAMAPA', '091': 'JESÚS CARRANZA', '092': 'XICO', '093': 'JILOTEPEC', '094': 'JUAN RODRÍGUEZ CLARA', '095': 'JUCHIQUE DE FERRER', '096': 'LANDERO Y COSS', '097': 'LERDO DE TEJADA', '098': 'MAGDALENA', '099': 'MALTRATA', '100': 'MANLIO FABIO ALTAMIRANO', '101': 'MARIANO ESCOBEDO', '102': 'MARTÍNEZ DE LA TORRE', '103': 'MECATLÁN', '104': 'MECAYAPAN', '105': 'MEDELLÍN DE BRAVO', '106': 'MIAHUATLÁN', '107': 'LAS MINAS', '108': 'MINATITLÁN', '109': 'MISANTLA', '110': 'MIXTLA DE ALTAMIRANO', '111': 'MOLOACÁN', '112': 'NAOLINCO', '113': 'NARANJAL', '114': 'NAUTLA', '115': 'NOGALES', '116': 'OLUTA', '117': 'OMEALCA', '118': 'ORIZABA', '119': 'OTATITLÁN', '120': 'OTEAPAN', '121': 'OZULUAMA DE MASCAREÑAS', '122': 'PAJAPAN', '123': 'PÁNUCO', '124': 'PAPANTLA', '125': 'PASO DEL MACHO', '126': 'PASO DE OVEJAS', '127': 'LA PERLA', '128': 'PEROTE', '129': 'PLATÓN SÁNCHEZ', '130': 'PLAYA VICENTE', '131': 'POZA RICA DE HIDALGO', '132': 'LAS VIGAS DE RAMÍREZ', '133': 'PUEBLO VIEJO', '134': 'PUENTE NACIONAL', '135': 'RAFAEL DELGADO', '136': 'RAFAEL LUCIO', '137': 'LOS REYES', '138': 'RÍO BLANCO', '139': 'SALTABARRANCA', '140': 'SAN ANDRÉS TENEJAPAN', '141': 'SAN ANDRÉS TUXTLA', '142': 'SAN JUAN EVANGELISTA', '143': 'SANTIAGO TUXTLA', '144': 'SAYULA DE ALEMÁN', '145': 'SOCONUSCO', '146': 'SOCHIAPA', '147': 'SOLEDAD ATZOMPA', '148': 'SOLEDAD DE DOBLADO', '149': 'SOTEAPAN', '150': 'TAMALÍN', '151': 'TAMIAHUA', '152': 'TAMPICO ALTO', '153': 'TANCOCO', '154': 'TANTIMA', '155': 'TANTOYUCA', '156': 'TATATILA', '157': 'CASTILLO DE TEAYO', '158': 'TECOLUTLA', '159': 'TEHUIPANGO', '160': 'ÁLAMO TEMAPACHE', '161': 'TEMPOAL', '162': 'TENAMPA', '163': 'TENOCHTITLÁN', '164': 'TEOCELO', '165': 'TEPATLAXCO', '166': 'TEPETLÁN', '167': 'TEPETZINTLA', '168': 'TEQUILA', '169': 'JOSÉ AZUETA', '170': 'TEXCATEPEC', '171': 'TEXHUACÁN', '172': 'TEXISTEPEC', '173': 'TEZONAPA', '174': 'TIERRA BLANCA', '175': 'TIHUATLÁN', '176': 'TLACOJALPAN', '177': 'TLACOLULAN', '178': 'TLACOTALPAN', '179': 'TLACOTEPEC DE MEJÍA', '180': 'TLACHICHILCO', '181': 'TLALIXCOYAN', '182': 'TLALNELHUAYOCAN', '183': 'TLAPACOYAN', '184': 'TLAQUILPA', '185': 'TLILAPAN', '186': 'TOMATLÁN', '187': 'TONAYÁN', '188': 'TOTUTLA', '189': 'TUXPAN', '190': 'TUXTILLA', '191': 'URSULO GALVÁN', '192': 'VEGA DE ALATORRE', '193': 'VERACRUZ', '194': 'VILLA ALDAMA', '195': 'XOXOCOTLA', '196': 'YANGA', '197': 'YECUATLA', '198': 'ZACUALPAN', '199': 'ZARAGOZA', '200': 'ZENTLA', '201': 'ZONGOLICA', '202': 'ZONTECOMATLÁN DE LÓPEZ Y FUENTES', '203': 'ZOZOCOLCO DE HIDALGO', '204': 'AGUA DULCE', '205': 'EL HIGO', '206': 'NANCHITAL DE LÁZARO CÁRDENAS DEL RÍO', '207': 'TRES VALLES', '208': 'CARLOS A. CARRILLO', '209': 'TATAHUICAPAN DE JUÁREZ', '210': 'UXPANAPA', '211': 'SAN RAFAEL', '212': 'SANTIAGO SOCHIAPAN', '999': 'NO ESPECIFICADO', '001': 'ABALÁ', '002': 'ACANCEH', '003': 'AKIL', '004': 'BACA', '005': 'BOKOBÁ', '006': 'BUCTZOTZ', '007': 'CACALCHÉN', '008': 'CALOTMUL', '009': 'CANSAHCAB', '010': 'CANTAMAYEC', '011': 'CELESTÚN', '012': 'CENOTILLO', '013': 'CONKAL', '014': 'CUNCUNUL', '015': 'CUZAMÁ', '016': 'CHACSINKÍN', '017': 'CHANKOM', '018': 'CHAPAB', '019': 'CHEMAX', '020': 'CHICXULUB PUEBLO', '021': 'CHICHIMILÁ', '022': 'CHIKINDZONOT', '023': 'CHOCHOLÁ', '024': 'CHUMAYEL', '025': 'DZÁN', '026': 'DZEMUL', '027': 'DZIDZANTÚN', '028': 'DZILAM DE BRAVO', '029': 'DZILAM GONZÁLEZ', '030': 'DZITÁS', '031': 'DZONCAUICH', '032': 'ESPITA', '033': 'HALACHÓ', '034': 'HOCABÁ', '035': 'HOCTÚN', '036': 'HOMÚN', '037': 'HUHÍ', '038': 'HUNUCMÁ', '039': 'IXIL', '040': 'IZAMAL', '041': 'KANASÍN', '042': 'KANTUNIL', '043': 'KAUA', '044': 'KINCHIL', '045': 'KOPOMÁ', '046': 'MAMA', '047': 'MANÍ', '048': 'MAXCANÚ', '049': 'MAYAPÁN', '050': 'MÉRIDA', '051': 'MOCOCHÁ', '052': 'MOTUL', '053': 'MUNA', '054': 'MUXUPIP', '055': 'OPICHÉN', '056': 'OXKUTZCAB', '057': 'PANABÁ', '058': 'PETO', '059': 'PROGRESO', '060': 'QUINTANA ROO', '061': 'RÍO LAGARTOS', '062': 'SACALUM', '063': 'SAMAHIL', '064': 'SANAHCAT', '065': 'SAN FELIPE', '066': 'SANTA ELENA', '067': 'SEYÉ', '068': 'SINANCHÉ', '069': 'SOTUTA', '070': 'SUCILÁ', '071': 'SUDZAL', '072': 'SUMA', '073': 'TAHDZIÚ', '074': 'TAHMEK', '075': 'TEABO', '076': 'TECOH', '077': 'TEKAL DE VENEGAS', '078': 'TEKANTÓ', '079': 'TEKAX', '080': 'TEKIT', '081': 'TEKOM', '082': 'TELCHAC PUEBLO', '083': 'TELCHAC PUERTO', '084': 'TEMAX', '085': 'TEMOZÓN', '086': 'TEPAKÁN', '087': 'TETIZ', '088': 'TEYA', '089': 'TICUL', '090': 'TIMUCUY', '091': 'TINUM', '092': 'TIXCACALCUPUL', '093': 'TIXKOKOB', '094': 'TIXMEHUAC', '095': 'TIXPÉHUAL', '096': 'TIZIMÍN', '097': 'TUNKÁS', '098': 'TZUCACAB', '099': 'UAYMA', '100': 'UCÚ', '101': 'UMÁN', '102': 'VALLADOLID', '103': 'XOCCHEL', '104': 'YAXCABÁ', '105': 'YAXKUKUL', '106': 'YOBAÍN', '999': 'NO ESPECIFICADO', '001': 'APOZOL', '002': 'APULCO', '003': 'ATOLINGA', '004': 'BENITO JUÁREZ', '005': 'CALERA', '006': 'CAÑITAS DE FELIPE PESCADOR', '007': 'CONCEPCIÓN DEL ORO', '008': 'CUAUHTÉMOC', '009': 'CHALCHIHUITES', '010': 'FRESNILLO', '011': 'TRINIDAD GARCÍA DE LA CADENA', '012': 'GENARO CODINA', '013': 'GENERAL ENRIQUE ESTRADA', '014': 'GENERAL FRANCISCO R. MURGUÍA', '015': 'EL PLATEADO DE JOAQUÍN AMARO', '016': 'GENERAL PÁNFILO NATERA', '017': 'GUADALUPE', '018': 'HUANUSCO', '019': 'JALPA', '020': 'JEREZ', '021': 'JIMÉNEZ DEL TEUL', '022': 'JUAN ALDAMA', '023': 'JUCHIPILA', '024': 'LORETO', '025': 'LUIS MOYA', '026': 'MAZAPIL', '027': 'MELCHOR OCAMPO', '028': 'MEZQUITAL DEL ORO', '029': 'MIGUEL AUZA', '030': 'MOMAX', '031': 'MONTE ESCOBEDO', '032': 'MORELOS', '033': 'MOYAHUA DE ESTRADA', '034': 'NOCHISTLÁN DE MEJÍA', '035': 'NORIA DE ÁNGELES', '036': 'OJOCALIENTE', '037': 'PÁNUCO', '038': 'PINOS', '039': 'RÍO GRANDE', '040': 'SAIN ALTO', '041': 'EL SALVADOR', '042': 'SOMBRERETE', '043': 'SUSTICACÁN', '044': 'TABASCO', '045': 'TEPECHITLÁN', '046': 'TEPETONGO', '047': 'TEÚL DE GONZÁLEZ ORTEGA', '048': 'TLALTENANGO DE SÁNCHEZ ROMÁN', '049': 'VALPARAÍSO', '050': 'VETAGRANDE', '051': 'VILLA DE COS', '052': 'VILLA GARCÍA', '053': 'VILLA GONZÁLEZ ORTEGA', '054': 'VILLA HIDALGO', '055': 'VILLANUEVA', '056': 'ZACATECAS', '057': 'TRANCOSO', '058': 'SANTA MARÍA DE LA PAZ', '999': 'NO ESPECIFICADO', '999': 'NO ESPECIFICADO', '997': 'NO APLICA', '998': 'SE IGNORA', '999': 'NO ESPECIFICADO', }

SyntaxError: invalid syntax (<ipython-input-160-345bee464189>, line 1)

In [152]:
dfA.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-07-05,13a406,2,3,25,1,25,25,006,2,...,2,2,2,2,99,1,99,MÃ©xico,99,2
1,2020-07-05,093267,2,4,02,2,02,02,004,2,...,2,1,2,2,99,1,99,MÃ©xico,99,2
2,2020-07-05,1e2219,2,4,09,2,09,09,005,1,...,2,2,2,2,99,1,99,MÃ©xico,99,97
3,2020-07-05,087250,2,4,09,1,15,15,109,2,...,1,2,1,2,99,1,99,MÃ©xico,99,2
4,2020-07-05,1d75fa,2,4,09,1,09,09,014,1,...,2,2,2,2,99,1,99,MÃ©xico,99,97


In [158]:
dfA.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [154]:
# df['col1'].map(di)
dfA['ORIGEN'].replace(di, inplace=True)


In [155]:
dfA.ORIGEN.unique()

array(['FUERA DE USMER', 'USMER'], dtype=object)

## Load data

In [2]:

# load the csv from the src folder
strpath = '../src/'
m = []
i =0
if not os.path.isfile('fulldb1.csv'):
    for filename in glob.glob(strpath+"*"):
        if filename.find('.csv')>0:
            date = '20'+filename[7:9] +'-' + filename[11:13] +'-' +filename[13:15]
            #print(date)
            df = pd.read_csv(filename, encoding = "ISO-8859-1",dtype =str)
            df['FechaRegistro'] = date
            df.FECHA_INGRESO = pd.to_datetime(df['FECHA_INGRESO'],)
            df.loc[df.ORIGEN=='1':'ORIGEN'] = 'USMER'
            df.loc[df.ORIGEN=='2':'ORIGEN'] = 'NO USMER'
            if i== 0:
                df0=df.copy()
                i=1
            else:
                frame = [df0,df]
                df0 = pd.concat(frame)
            del df
else:
    df0 = pd.read_csv('fulldb.csv')
    df0.FechaRegistro = pd.to_datetime(df0['FechaRegistro'], format='%Y-%m-%d')
    lastdateRecord = df0.FechaRegistro.max()
    deltaT = datetime.datetime.now() - lastdateRecord
    for i in range(1,deltaT.days+1) :
        filename = (lastdateRecord + datetime.timedelta(days=1)).strftime("%Y%m%d")+'.csv'
        df = pd.read_csv(strpath+filename, encoding = "ISO-8859-1",dtype =str)
        df['FechaRegistro'] = (lastdateRecord + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
        df.FECHA_INGRESO = pd.to_datetime(df['FECHA_INGRESO'],)

len(df0)

22840271

In [141]:
df.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [3]:
# we sorted the value by 
# df0 = df0.sort_values(by='FechaRegistro', ascending=False)
df0.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,FechaRegistro
0,2020-06-04,1632b6,2,4,14,2,15,14,120,1,...,2,2,2,99,1,99,MÃ©xico,99,97,2020-06-04
1,2020-06-04,022691,2,4,09,2,09,09,015,2,...,1,2,2,99,1,99,MÃ©xico,99,2,2020-06-04
2,2020-06-04,10413d,2,4,15,1,15,15,091,2,...,2,2,2,99,1,99,MÃ©xico,99,2,2020-06-04
3,2020-06-04,0c15eb,2,4,11,2,15,11,020,1,...,2,2,2,99,1,99,MÃ©xico,99,97,2020-06-04
4,2020-06-04,12944a,2,4,15,2,15,15,060,1,...,2,1,2,99,1,99,MÃ©xico,99,97,2020-06-04


In [4]:
md("We have {0} records of covid evolution cases in Mexico until today {1}".format(df0.shape[0], datetime.datetime.now()) )


We have 22840271 records of covid evolution cases in Mexico until today 2020-07-06 22:20:28.608644

In [5]:
df0.FechaRegistro = pd.to_datetime(df0['FechaRegistro'], format='%Y-%m-%d')

In [150]:
#today = (datetime.datetime.now()).strftime("%Y%m%d")
today = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y%m%d")
yesterday = (datetime.datetime.now() - datetime.timedelta(days=2)).strftime("%Y%m%d")
ereyesterday = (datetime.datetime.now() - datetime.timedelta(days=3)).strftime("%Y%m%d")
df= pd.read_csv( '../src/'+today       +'.csv', encoding = "ISO-8859-1",dtype =str)
dfA= pd.read_csv('../src/'+yesterday   +'.csv', encoding = "ISO-8859-1",dtype =str)
dfB= pd.read_csv('../src/'+ereyesterday+'.csv', encoding = "ISO-8859-1",dtype =str)

## 1 How many cases we have in the last report vs all reports?

In [7]:
# we get unique cases globaly and from the last csv 
global_ids = df0.ID_REGISTRO.unique().tolist()
last_ids  = df.ID_REGISTRO.unique().tolist()
# we get the cases that are present in previous reports but they doesn't show in the last report
untracket_cases = set(global_ids)- set(last_ids)
tracket_cases = set(global_ids)- set(untracket_cases)

In [8]:
df0.FECHA_SINTOMAS = pd.to_datetime(df0['FECHA_SINTOMAS'], format ='%Y-%m-%d')

In [78]:
# we extract the records that does not exist in the last report
df1 = df0[df0.ID_REGISTRO.isin(untracket_cases)]
df1 = df1.copy()

md("we have {0} register that are not tracket".format(df1.ID_REGISTRO.nunique()) )

we have 37114 register that are not tracket

1. How many days the records exist
2. Is there some kind or relation between the records?

In [79]:
# df0.FechaRegistro = pd.to_datetime(df0['FechaRegistro'], format='%Y-%m-%d')
df1.FECHA_SINTOMAS = pd.to_datetime(df1['FECHA_SINTOMAS'], format ='%Y-%m-%d')
df1.FECHA_ACTUALIZACION = pd.to_datetime(df1['FECHA_ACTUALIZACION'], format ='%Y-%m-%d')
df1.FechaRegistro = pd.to_datetime(df1['FechaRegistro'], format ='%Y-%m-%d')


In [98]:
# we get the duration of cases between records
df1['LastDay']  = df1.groupby('ID_REGISTRO')['FechaRegistro'].transform('max')
df1['FirstDay'] = df1.groupby('ID_REGISTRO')['FechaRegistro'].transform('min')
df1['Duration'] = (df1.LastDay - df1.FirstDay).dt.days

In [106]:
df3.groupby(['RESULTADO','Duration'])['Duration'].count().to_frame(name = 'count').reset_index()

,RESULTADO,Duration,count
0,1,0,347
1,1,1,47
2,1,2,45
3,1,3,51
4,1,4,59
...,...,...,...
211,3,63,1
212,3,65,3
213,3,69,1
214,3,81,2


In [116]:
# We keep only one case to get plot correctly the cases
df3 = df1.drop_duplicates(keep='first')
df3 = df3.drop_duplicates('ID_REGISTRO',keep='first')
df_3 = df3.groupby(['RESULTADO','Duration'])['Duration'].count().to_frame(name = 'count').reset_index()
# sns.lineplot(x='Duration', y='count', data=df_3, hue='RESULTADO');
df_3

,RESULTADO,Duration,count
0,1,0,347
1,1,1,47
2,1,2,45
3,1,3,51
4,1,4,59
...,...,...,...
211,3,63,1
212,3,65,3
213,3,69,1
214,3,81,2


In [127]:
sns.lineplot(x='Duration', y='count', data=df_3, style='RESULTADO')#, palette=sns.color_palette("Set1", 3))

In [128]:
df3['EDAD_int'] = df3.EDAD.astype(int)

In [129]:
df3.groupby('EDAD_int')['EDAD_int'].count().plot()

In [130]:
df1.FechaRegistro = pd.to_datetime(df1['FechaRegistro'], format='%Y-%m-%d')
df1 = df1.sort_values(by='FechaRegistro', ascending=False)

In [131]:
df1.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,FechaRegistro,LastDay,FirstDay,Duration
623343,2020-07-06,00c5ef,1,12,05,1,15,05,002,1,...,2,3,99,MÃ©xico,99,97,2020-07-06,2020-07-06,2020-07-06,0
627280,2020-07-06,05b2da,2,12,14,1,09,14,098,1,...,1,3,99,MÃ©xico,99,97,2020-07-06,2020-07-06,2020-07-06,0
626759,2020-07-06,08ee44,2,12,11,1,11,11,031,1,...,1,3,99,MÃ©xico,99,97,2020-07-06,2020-07-06,2020-07-06,0
626760,2020-07-06,19cd96,2,12,12,1,12,12,029,1,...,2,3,99,MÃ©xico,99,97,2020-07-06,2020-07-06,2020-07-06,0
626761,2020-07-06,175bbc,2,12,11,2,11,11,026,1,...,1,3,99,MÃ©xico,99,97,2020-07-06,2020-07-06,2020-07-06,0


In [132]:
print(len(df1))
df3 = df1.drop_duplicates(keep='first')
# print(len(df3))
df3 = df3.drop_duplicates('ID_REGISTRO',keep='first')
# print(len(df3))
print(len(df1))

280457
280457


In [133]:
df3.groupby(['FechaRegistro'])['FechaRegistro'].count().sort_values(ascending=False).head(19)

FechaRegistro
2020-07-06    11323
2020-04-16     2370
2020-04-20     1395
2020-05-19      775
2020-07-02      731
2020-05-21      594
2020-05-12      558
2020-05-07      550
2020-04-30      481
2020-06-10      439
2020-05-08      439
2020-06-03      437
2020-06-26      433
2020-06-02      420
2020-06-22      408
2020-04-17      404
2020-05-13      404
2020-06-30      400
2020-06-16      374
Name: FechaRegistro, dtype: int64

In [134]:
df3.groupby('RESULTADO')['RESULTADO'].count()

RESULTADO
1     2541
2     4432
3    30141
Name: RESULTADO, dtype: int64

In [135]:
df3.groupby(['FechaRegistro'])['FechaRegistro'].count().plot()

In [136]:
df3.groupby(['FECHA_SINTOMAS'])['FECHA_SINTOMAS'].count().plot()

In [139]:
df3.groupby(['FECHA_ACTUALIZACION'])['FECHA_ACTUALIZACION'].count().plot()

In [138]:
df3[df3.FECHA_SINTOMAS<'2020-02-28'].groupby('RESULTADO')['RESULTADO'].count()

RESULTADO
1     4
2    71
3    10
Name: RESULTADO, dtype: int64

# personas que siguen en el sistema

In [28]:
# we extract the records that does not live in the last report
dfO1 = df0[df0.ID_REGISTRO.isin(tracket_cases)]
dfO1 = dfO1.copy()
dfO1.shape


(22559814, 36)

In [29]:
# df0.FechaRegistro = pd.to_datetime(df0['FechaRegistro'], format='%Y-%m-%d')
dfO1.FECHA_SINTOMAS = pd.to_datetime(dfO1['FECHA_SINTOMAS'], format ='%Y-%m-%d')
dfO1.FECHA_ACTUALIZACION = pd.to_datetime(dfO1['FECHA_ACTUALIZACION'], format ='%Y-%m-%d')
dfO1.FechaRegistro = pd.to_datetime(dfO1['FechaRegistro'], format ='%Y-%m-%d')
dfO1['delta_lastRecord'] = (dfO1.FECHA_ACTUALIZACION-dfO1.FechaRegistro ).dt.days
dfO1['deltaReporte'] = (dfO1.FechaRegistro-dfO1.FECHA_SINTOMAS).dt.days
dfO1[dfO1.FECHA_SINTOMAS<'2020-02-29'].groupby('RESULTADO')['RESULTADO'].count()



RESULTADO
1      1545
2    155506
3       336
Name: RESULTADO, dtype: int64

In [30]:
# sort_values(by='FechaRegistro', ascending=False).head()

dfO13 = dfO1.drop_duplicates(['ID_REGISTRO', 
       'FECHA_INGRESO', 'FECHA_SINTOMAS', ],keep='first')
dfO13 = dfO13.copy()

In [31]:
dfO13.groupby('ID_REGISTRO')['FECHA_ACTUALIZACION'].count().sort_values( ascending=False)

ID_REGISTRO
0a2098    4
113be4    4
168506    3
0aa887    3
139532    3
         ..
1452d1    1
1452cc    1
1452c3    1
1452c2    1
000002    1
Name: FECHA_ACTUALIZACION, Length: 641142, dtype: int64

In [32]:
dfO131 = dfO13[dfO13.FECHA_SINTOMAS<'2020-02-29']
dfO131.groupby('ID_REGISTRO')['FECHA_ACTUALIZACION'].count().sort_values( ascending=False).head(10)

ID_REGISTRO
0304ae    2
02a0df    2
0226ad    2
13fb66    2
0632c1    2
1732a3    2
09ab34    1
09a8f7    1
09a8af    1
09a72f    1
Name: FECHA_ACTUALIZACION, dtype: int64

In [33]:
dfO131[dfO131.ID_REGISTRO.isin(['0304ae','02a0df','0226ad','13fb66','0632c1','1732a3',])][['ID_REGISTRO', 
        'FECHA_SINTOMAS','FECHA_INGRESO',]].sort_values(['ID_REGISTRO','FECHA_SINTOMAS'], ascending=False)

,ID_REGISTRO,FECHA_SINTOMAS,FECHA_INGRESO
45837,1732a3,2020-02-06,2020-04-14
55559,1732a3,2020-02-06,2020-04-07
224695,13fb66,2020-01-09,2020-01-09
34894,13fb66,2020-01-02,2020-01-04
118033,0632c1,2020-02-12,2020-02-15
6076,0632c1,2020-02-12,2020-03-15
245193,0304ae,2020-01-28,2020-01-28
27433,0304ae,2020-01-27,2020-01-28
265357,02a0df,2020-02-20,2020-02-21
2333,02a0df,2020-02-20,2020-03-31


In [34]:
dfO131['deltaCorrección'] = (dfO131.FECHA_INGRESO-dfO131.FECHA_SINTOMAS).dt.days

/Users/jorge.urteaga/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
dfO131.groupby('ID_REGISTRO')['deltaCorrección'].count().sort_values( ascending=False).head(50)

ID_REGISTRO
0304ae    2
02a0df    2
0226ad    2
13fb66    2
0632c1    2
1732a3    2
09ab34    1
09a8f7    1
09a8af    1
09a72f    1
1e844c    1
099bfc    1
09ab8c    1
098ca1    1
0989cb    1
09870e    1
098532    1
0992c5    1
09b305    1
09ad90    1
097ee0    1
09b30e    1
09bc06    1
09c726    1
09cd5a    1
09e264    1
09e545    1
09eabf    1
09f464    1
09f651    1
09fa2d    1
09fd9f    1
09ffe7    1
098054    1
097a8f    1
0a0f7f    1
093d6c    1
091b0e    1
091e6b    1
092a9c    1
092f75    1
093097    1
093164    1
093400    1
093599    1
09364d    1
093896    1
093931    1
093c64    1
093d26    1
Name: deltaCorrección, dtype: int64

In [36]:
df0['FECHA_SINTOMAS'].min()

Timestamp('2020-01-01 00:00:00')

In [37]:
df0[df0['FECHA_SINTOMAS']=='2020-01-01']

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,FechaRegistro
106885,2020-06-04,0c745e,2,4,09,1,09,09,008,1,...,2,2,2,99,2,99,MÃ©xico,99,97,2020-06-04
109525,2020-06-04,0bc511,2,4,09,2,09,09,002,1,...,2,2,2,99,2,99,MÃ©xico,99,97,2020-06-04
109548,2020-06-04,082032,2,4,14,2,14,14,039,1,...,2,2,2,99,2,99,MÃ©xico,99,97,2020-06-04
110590,2020-06-04,031621,2,4,09,2,09,09,002,2,...,2,2,2,99,2,99,MÃ©xico,99,2,2020-06-04
111805,2020-06-04,073494,2,4,23,2,07,23,005,1,...,2,2,2,99,2,99,MÃ©xico,99,97,2020-06-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377920,2020-06-21,044d0d,2,4,09,1,09,09,015,1,...,2,2,2,99,2,99,MÃ©xico,99,97,2020-06-21
381950,2020-06-21,1dc68d,2,4,14,1,14,14,101,2,...,2,2,2,99,2,99,MÃ©xico,99,2,2020-06-21
397140,2020-06-21,033221,2,12,05,1,28,05,030,1,...,2,2,2,1,2,99,MÃ©xico,99,97,2020-06-21
405742,2020-06-21,0bdf57,2,4,09,1,09,15,104,1,...,2,2,2,99,2,99,MÃ©xico,99,97,2020-06-21


In [38]:
first= pd.read_csv( '../src/'+'20200412'       +'.csv', encoding = "ISO-8859-1",dtype =str)


In [39]:
first.FECHA_SINTOMAS = pd.to_datetime(first['FECHA_SINTOMAS'], format ='%Y-%m-%d')

In [40]:
first.FECHA_SINTOMAS.min()

Timestamp('2020-01-01 00:00:00')

At the end of 2019, the world saw the born of the Sars-cov-2 virus which genereate a disease called COVID19 (COrona VIrus disease). The high risk that we face with this virus is that the sympthons are related to a typical flu but generate a high damage un the lungs decreasing the oxygen level. The disminution of oxygen generate a high demand of intensive beds.

From WHO files

According to the WHO the incubation period for COVID-19, is on average 5-6 days, but can be extended to 14 days. However there some evidence that shows that exist different types of transmition, such as symptomatic, pre-symptomatic and a symptomatic.

The typical transmition is the symptomatic, this ocurrs when a person shown COVID symptomatic and can spread the virus conciently or inconciently respiratory droplest over thier enviroment. However, some cases has shown that a person can transmit COVID-19 without symptoms, (3 or 2 days) before present the typical symptoms. Finally there are some concern about the asyptomatic transmission according to the WHO: Asymptomatic transmission refers to transmission of the virus from a person, who does not develop symptoms. There are few reports of laboratory-confirmed cases who are truly asymptomatic, and to date, there has been no documente asymptomatic transmission. This does not exclude the possibility that it may occur. Asymptomatic cases have been reported as part of contact tracing efforts in some countries. Source https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200402-sitrep-73-covid-19.pdf?sfvrsn=5ae25bc7_2#:~:text=An%20asymptomatic%20laboratory%2Dconfirmed,more%20information%20becomes%20available.


The goal of this notebook is listed below:
* Understand the evolution of the covi19 in Mexico, there are a rate of 10% of deads though a visualization analysis


Key elements

* Incubation period can be expresed a GAMMA.
* Is most common the transmision after show symptoms.
* How we can modeled (explain pre-symtomatic transmition)
* how we can modeled asyntomatic transmition
